In [1]:
import numpy as np
from operator import itemgetter
# web scraping imports
# from CNN import *
# from Reuters import *
# from SeekingAlpha import *
# from CNBC import *

# install nltk, selenium, genism and bs4
# install chromedriver and add to path

# ntlk imports
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

# prettyprint
import pprint

# genism imports
from gensim import corpora,models
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS

stopwords = []  # user defined stopwords

def process_article(text):
# for single article (text string), tokenize and lemmatize data, return list of word stems
    sents = sent_tokenize(text)
    sents = '. '.join([s.strip().replace("\n", "") for s in sents])
    words = [word for word in sents.lower().split()
             if word not in STOPWORDS and word.isalnum() and word not in stopwords]
    wordnet_lemmatizer = WordNetLemmatizer()
    words = [wordnet_lemmatizer.lemmatize(i) for i in words]
    p_stemmer = PorterStemmer()
    words = [p_stemmer.stem(i) for i in words]
    return words


def get_corpus(text):
# for single article (text string), tokenize data and return corpus (list of lists)
    a_list = [process_article(text)]
    dictionary = corpora.Dictionary(a_list)
    return [dictionary.doc2bow(a) for a in a_list]


#https://radimrehurek.com/gensim/models/tfidfmodel.html
# models.TfidfModel: normalize=True by default,normalize document vectors to unit euclidean length.You can also inject your own function into normalize.
#slope=0.65 by default: Parameter required by pivoted document length normalization which determines the slope to which the old normalization can be tilted. This parameter only works when pivot is defined.
def tf_idf(corpus):
    average_count=[]
    for item in corpus:
        if len(item) != 0:
            average_count.append(np.mean(item,axis=0)[1])
        else:
            average_count.append(0)
    pivot = np.mean(average_count)    
    tfidf = models.TfidfModel(corpus,pivot=pivot)
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf


def get_lda(news_list, num_topics, num_passes):
# for list of articles ([date,text string] list), generate lda model
    texts = [process_article(a[1]) for a in news_list]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(a) for a in texts]
    corpus_tfidf = tf_idf(corpus)
    lda = LdaModel(corpus_tfidf,  # list of lists containing tuples (word index, word freq)
                   id2word=dictionary,  # change nums back to words
                   num_topics=num_topics,  # need to set num topics
                   passes=num_passes)
    return lda,corpus_tfidf
 

def print_lda(lda, num_words=8):
# prints lda model coefficients, user can specify number of words to include for each topic
    pp = pprint.PrettyPrinter(indent=4)
    # create prettyprint obj, 8 words for each topic
    pp.pprint(lda.print_topics(num_words=8))


def get_topic(article_number,corpus_tfidf):
# for a single article (text string), given the order of the article on the loaded news list, returns list of relevant topics ordered by likelihood
    #single_corpus = get_corpus(article) 
#     lda.get_document_topics(new_a[0],minimum_probability=0.05,per_word_topics=False)
    return sorted(lda.get_document_topics(corpus_tfidf[article_number],minimum_probability=0, per_word_topics=False),
                  key=itemgetter(1), reverse=True)



In [2]:
import csv
with open('tfidf.txt') as csvfile:
    l=[]
    readCSV = csv.reader(csvfile, delimiter='\n')
    for i in readCSV:
        l.append(i)
        

In [12]:
print (l[0][0][0])

[


In [9]:
import os
import datetime

def get_news(ticker,days):
# retreives news from websites, may take a few minutes
    #get_cnn(ticker,days)
    #get_reuters(ticker,days)
    #get_seekingalpha(ticker,days)
    get_cnbc(ticker,days)

def get_date(filename,current_dir):
# for given filename and relative directory, return date (datetime format)
    try:
        date = datetime.datetime.strptime(filename[:19],'%Y-%m-%d_%H.%M.%S')
        return date
    except:
        pass
    
def get_file(filename,current_dir):
# for given filename and relative directory, return file content (text string)
    date = get_date(filename,current_dir)
    if date:
        with open(current_dir+"/"+filename,"r") as f:
            text = f.read()
        return text

def load_news(ticker,days=3):
# for given ticker, loads news from database and returns a list of [date,articles]
# default parameter: load articles from recent three days
    l = []
    for news_source in os.listdir("news_test/"+ticker):
        current_dir = "news_test/"+ticker+"/"+news_source
        for doc in os.listdir(current_dir):
            date = get_date(doc,current_dir) 
            if date and date>=(datetime.datetime.now()-datetime.timedelta(days)):
                l.append([date,get_file(doc,current_dir)])
    return l

In [10]:
def load_dict():
# load negative and positive dictionaries
    import os
    def get_dict_words(dict_dir):
        with open(dict_dir,'r') as f:
            words = []
            for line in f:
                words.append(line.replace('\n','').lower())
        return words
    
    neg = get_dict_words(os.getcwd() + '\\dictionaries\\negative.txt')
    pos = get_dict_words(os.getcwd() + '\\dictionaries\\positive.txt')
    return neg,pos

def emotion_analysis(text):
# input: text (string)
# output: negative and positive score
    neg,pos = load_dict()
    from nltk import word_tokenize
    results = dict()
    c1 = c2 = 0
    for word in word_tokenize(text):
        word = word.lower()
        if word in neg:
            c1 += 1
        if word in pos:
            c2 += 1
    n = len(word_tokenize(text))
    results['negative'] = c1/n
    results['positive'] = c2/n
    return results

def emotion_analyzer(text_list):
# input: list of texts (list)
# output: dataframe containing negative and positive scores of each text
    import pandas as pd
    df = pd.DataFrame(columns=['negative','positive'])
    count = 1
    for text in text_list:
        a = emotion_analysis(text[1])
        df.loc[count] = [a['negative'],a['positive']]
        count += 1
    return df

In [11]:
def get_lda_noun(nouns_list, num_topics, num_passes):
# input: list of nouns (list), number of topics (int), number of passes (int)
# output: lda model and corpus_tfidf
    texts = [process_article_noun(a) for a in nouns_list]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(a) for a in texts]
    corpus_tfidf = tf_idf(corpus)
    lda = LdaModel(corpus_tfidf,  # list of lists containing tuples (word index, word freq)
                   id2word=dictionary,  # change nums back to words
                   num_topics=num_topics,  # need to set num topics
                   passes=num_passes)
    return lda,corpus_tfidf

def process_article_noun(text):
# for single article (text string), tokenize and lemmatize data, return list of word stems
# for topic analysis on nouns
    sents = sent_tokenize(text)
    sents = '. '.join([s.strip().replace("\n", "") for s in sents])
    words = [word for word in sents.lower().split()
             if word not in STOPWORDS and word.isalnum() and word not in stopwords]
    wordnet_lemmatizer = WordNetLemmatizer()
    words = [wordnet_lemmatizer.lemmatize(i) for i in words]
    #p_stemmer = PorterStemmer()
    #words = [p_stemmer.stem(i) for i in words]
    return words

def pick_nouns(doc,stopwords):
# input: doc (str) and stopwords (list)
# output: list of nouns (list)
    import nltk
    stopwords = [x.upper() for x in stopwords]
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sens = sent_detector.tokenize(doc)
    wpos = []
    for i in sens:
        word_list = nltk.word_tokenize(i)
        wpos.extend(nltk.pos_tag(word_list))
    result = []
    for i in wpos:
        if i[1] == 'NNP':
            if i[0].upper() not in stopwords:
                result.append(i[0])
    return result

def noun_lda(doc_list,stopwords,numTopic,numPass):
# input: list of news (list), stopwords(case does not matter) (list) , number of topics (int) and number of passes (int)
# output: lda model and corpus_tfidf
    nouns = []
    for i in text:
        nouns.append(' '.join(pick_nouns(i,stopwords)))
    lda,corpus_tfidf = get_lda_noun(nouns, numTopic, numPass)
    return lda,corpus_tfidf

In [12]:
def sort_text(news):
#turn news into a list of text, ordered from the newest to the latest
#input: news: list of (datetime,news) from load_news()
    sorted_news = sorted(news, key=lambda article: article[0],reverse=True)
    texts_list = list(map(itemgetter(1),sorted_news))
    return texts_list
    

def document_term_matrix(texts_list,num_topics,lda,stopwords,type= 'topic_terms'):
# input: list of texts (list), number of topics (int), lda model, list of stopwords (list), type of matrix generated (string)
# output: matrix (list)
    texts = [process_article_noun(a) for a in texts_list]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(a) for a in texts]
    
    nouns = []
    for i in texts_list:
        nouns.append(' '.join(pick_nouns(i,stopwords)))
    texts = [process_article_noun(a) for a in nouns]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(a) for a in texts]
    corpus_tfidf = tf_idf(corpus)
    
    if type == 'document_topics':
        dt = []
        for i in range(len(texts_list)):
            dt.append(get_topic(i,corpus_tfidf))
        return dt
    if type == 'topic_terms':
        tt = []
        for k,v in dictionary.items():
            pass
        word_map = dictionary.id2token
        for i in range(num_topics):
            tt.append(list(map(lambda x: (word_map[x[0]],x[1]),lda.get_topic_terms(i))))
        return tt
    
def exp_smooth(alpha,text_list):
# input: alpha in the exponential smoothing equation (double), list of texts (list)
# output: conslidated sentimental scores (tuple of doubles)
    neg = []
    pos = []
    for d in text_list:
        neg.append(emotion_analysis(d)['negative'])
        pos.append(emotion_analysis(d)['positive'])
    for i in range(len(text_list)):
        if i == 0:
            neg_s = neg[len(pos)-i-1]
            pos_s = pos[len(pos)-i-1]
        else:
            neg_s = alpha*neg[len(pos)-i-1] + (1-alpha)*neg_s
            pos_s = alpha*pos[len(pos)-i-1] + (1-alpha)*pos_s
    return neg_s,pos_s

In [6]:
get_news("fb",5)

NameError: name 'get_cnbc' is not defined

In [21]:
news = load_news("FB",30)

In [7]:
news

[[datetime.datetime(2019, 4, 24, 16, 11),
  'show chapters\n\n\n\n\n\n \n\n\n\n   \n\n\n\n\t\t\t\t    \t                 \n            Facebook beat on top and bottomHere\'s what six experts are watching now020202\n\t\t\t                        3 Hours Ago |  04:57 \n\n\n\n\n\nFacebook took a $3 billion charge due to the Federal Trade Commission\'s inquiry into its business, the company disclosed in its first quarter 2019 earnings report. Facebook estimated the final charge could be as much as $5 billion for the company.\n\n"The matter remains unresolved, and there can be no assurance as to the timing or the terms of any final outcome," the company wrote in its release.\nThe FTC launched an investigation into Facebook after the Cambridge Analytica scandal broke, probing whether Facebook violated a 2011 agreement it made with the agency that required it to gain explicit consent to share users data. Although the FTC has previously declined to confirm the status of the investigation, The 

In [23]:
corpus = [[(0, 1), (1, 3), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 9), (8, 1), (9, 2), (10, 1), (11, 2), (12, 3), (13, 6), (14, 1), (15, 2), (16, 1), (17, 1), (18, 2), (19, 8)], [(4, 1), (10, 2), (20, 1), (21, 1), (22, 3), (23, 4), (24, 3), (25, 1), (26, 2), (27, 1), (28, 2), (29, 2), (30, 14), (31, 2), (32, 2), (33, 2), (34, 5), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 7), (41, 1), (42, 1), (43, 2)], [(5, 1), (8, 1), (11, 2), (13, 1), (14, 7), (15, 2), (19, 2), (20, 4), (22, 27), (24, 7), (30, 1), (38, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 3), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 3), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 2), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 3), (80, 1), (81, 1), (82, 1), (83, 4), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1)], [(10, 2), (17, 1), (20, 1), (22, 5), (24, 3), (31, 1), (52, 1), (56, 2), (69, 1), (79, 1), (90, 1), (91, 1), (92, 1), (93, 3), (94, 1), (95, 1), (96, 2), (97, 1)], [(10, 2), (20, 1), (22, 32), (31, 1), (38, 1), (52, 4), (69, 1), (72, 1), (79, 3), (90, 12), (95, 13), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1)], [(10, 1), (20, 1), (22, 9), (24, 3), (56, 1), (61, 1), (65, 1), (70, 1), (73, 1), (78, 1), (83, 3), (87, 2), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1)], [(0, 1), (1, 3), (3, 1), (4, 5), (5, 1), (8, 2), (11, 3), (13, 3), (14, 7), (15, 3), (17, 1), (19, 6), (20, 1), (22, 1), (24, 4), (44, 3), (45, 1), (49, 1), (53, 1), (54, 1), (62, 1), (71, 4), (72, 4), (74, 1), (84, 1), (89, 1), (97, 1), (110, 3), (111, 1), (112, 6), (113, 6), (114, 1), (115, 1), (116, 1), (117, 2), (118, 4), (119, 1), (120, 1), (121, 1), (122, 1), (123, 1), (124, 1), (125, 2), (126, 1), (127, 1), (128, 1)], [(3, 1), (10, 1), (20, 1), (22, 2), (24, 10), (26, 1), (69, 2), (73, 1), (83, 3), (84, 1), (85, 1), (129, 1), (130, 1), (131, 1), (132, 1)], [(0, 1), (3, 1), (8, 1), (13, 6), (14, 8), (17, 1), (18, 1), (19, 7), (20, 1), (22, 3), (24, 8), (55, 1), (70, 1), (78, 1), (83, 2), (84, 1), (85, 3), (97, 1), (130, 1), (131, 1), (133, 1), (134, 1), (135, 2), (136, 2), (137, 1), (138, 1), (139, 4), (140, 1), (141, 1), (142, 1), (143, 2), (144, 1), (145, 1), (146, 1), (147, 1), (148, 2), (149, 1), (150, 1), (151, 1), (152, 1), (153, 15), (154, 2), (155, 1), (156, 1), (157, 1), (158, 1), (159, 3), (160, 4), (161, 1), (162, 1), (163, 1), (164, 1), (165, 1), (166, 1), (167, 1), (168, 1), (169, 1), (170, 2), (171, 2), (172, 1), (173, 1), (174, 1), (175, 2), (176, 2), (177, 4), (178, 4), (179, 3), (180, 2), (181, 1), (182, 1), (183, 1), (184, 1), (185, 1), (186, 1), (187, 3), (188, 1), (189, 1), (190, 4), (191, 1)], [(3, 1), (8, 1), (13, 1), (14, 2), (19, 1), (20, 1), (22, 2), (24, 3), (73, 1), (83, 1), (84, 1), (141, 1), (143, 2), (165, 1), (167, 1), (192, 1), (193, 1), (194, 3), (195, 1), (196, 1), (197, 2), (198, 1)], [(3, 1), (8, 1), (11, 2), (13, 1), (14, 2), (15, 2), (19, 1), (20, 1), (22, 2), (24, 3), (73, 1), (83, 1), (84, 2), (89, 1), (141, 1), (143, 2), (165, 1), (167, 1), (192, 1), (193, 1), (194, 3), (195, 1), (196, 1), (197, 2), (198, 1), (199, 1), (200, 1), (201, 1), (202, 1)], [(0, 1), (3, 2), (8, 1), (10, 3), (11, 6), (13, 2), (14, 4), (15, 6), (18, 1), (19, 2), (20, 1), (22, 1), (24, 9), (31, 1), (56, 1), (69, 3), (73, 2), (74, 1), (81, 2), (83, 2), (85, 5), (89, 1), (129, 1), (130, 1), (131, 1), (141, 1), (143, 1), (145, 1), (146, 1), (156, 1), (160, 2), (165, 1), (178, 1), (179, 1), (184, 1), (203, 1), (204, 1), (205, 1), (206, 1), (207, 1), (208, 1), (209, 1), (210, 2), (211, 1), (212, 1), (213, 1), (214, 1), (215, 1), (216, 2), (217, 1), (218, 3), (219, 2), (220, 1), (221, 1), (222, 1), (223, 1), (224, 2), (225, 1), (226, 1), (227, 1), (228, 1), (229, 1), (230, 1), (231, 1), (232, 1), (233, 1), (234, 1), (235, 1), (236, 1), (237, 1)], [(10, 1), (20, 1), (22, 26), (44, 2), (74, 1), (90, 6), (238, 13), (239, 1), (240, 3), (241, 1), (242, 1), (243, 1), (244, 1), (245, 2)], [(22, 22), (38, 1), (46, 1), (52, 3), (55, 1), (66, 4), (69, 2), (73, 1), (79, 2), (83, 3), (90, 6), (95, 6), (101, 1), (107, 1), (112, 1), (123, 1), (246, 2), (247, 1), (248, 1), (249, 1), (250, 1), (251, 2), (252, 2), (253, 1), (254, 1), (255, 1), (256, 1), (257, 1), (258, 1), (259, 2), (260, 1), (261, 1), (262, 1), (263, 1), (264, 1)], [(22, 22), (38, 1), (46, 1), (52, 3), (55, 1), (66, 4), (69, 2), (73, 1), (79, 2), (83, 3), (90, 6), (95, 6), (101, 1), (107, 1), (112, 1), (123, 1), (246, 2), (247, 1), (248, 1), (249, 1), (250, 1), (251, 2), (252, 2), (253, 1), (254, 1), (255, 1), (256, 1), (257, 1), (258, 1), (259, 2), (260, 1), (261, 1), (262, 1), (263, 1), (264, 1)], [(22, 16), (35, 1), (62, 1), (69, 1), (71, 3), (90, 1), (95, 1), (125, 1), (240, 1), (242, 1), (243, 1), (244, 1), (245, 3), (265, 1), (266, 1), (267, 1), (268, 1), (269, 1), (270, 1), (271, 1), (272, 1), (273, 2), (274, 1), (275, 1), (276, 1), (277, 1), (278, 1), (279, 1), (280, 1)], [(22, 2), (52, 1), (79, 1), (258, 1), (263, 2), (281, 1), (282, 1), (283, 1), (284, 1), (285, 1), (286, 1), (287, 1), (288, 1), (289, 1), (290, 1), (291, 1), (292, 1), (293, 1), (294, 2), (295, 1), (296, 1), (297, 1)], [(0, 4), (1, 2), (4, 1), (5, 4), (13, 9), (14, 1), (16, 1), (17, 2), (18, 3), (19, 15), (97, 2), (165, 1), (298, 5), (299, 1), (300, 1), (301, 1), (302, 1), (303, 1), (304, 1), (305, 1), (306, 1), (307, 1), (308, 1), (309, 1), (310, 1)], [(0, 3), (11, 2), (13, 5), (14, 4), (15, 2), (18, 1), (19, 5), (89, 1), (148, 2), (159, 2), (302, 2), (310, 1), (311, 1), (312, 1), (313, 1), (314, 1), (315, 1), (316, 1), (317, 1), (318, 1), (319, 1), (320, 1), (321, 1), (322, 1), (323, 1), (324, 3), (325, 3), (326, 1), (327, 1), (328, 1), (329, 1), (330, 1), (331, 1)], [(52, 1), (84, 1), (195, 1), (249, 1), (332, 1), (333, 2), (334, 2)], [(69, 1), (333, 2), (335, 1), (336, 1), (337, 1), (338, 1), (339, 1), (340, 1), (341, 2), (342, 1), (343, 1), (344, 1)], [(0, 2), (1, 1), (5, 2), (11, 2), (13, 4), (14, 4), (15, 2), (16, 1), (18, 1), (19, 5), (89, 1), (130, 1), (148, 6), (195, 3), (308, 1), (313, 1), (345, 1), (346, 3), (347, 1), (348, 1), (349, 1), (350, 1), (351, 1), (352, 3), (353, 1), (354, 2), (355, 1), (356, 1), (357, 2), (358, 1), (359, 3), (360, 1)], [(31, 1), (165, 1), (331, 1), (355, 1), (361, 1), (362, 2), (363, 1), (364, 1)], [(17, 1), (49, 1), (97, 1), (197, 1), (301, 1), (331, 1), (365, 1), (366, 1), (367, 1), (368, 1), (369, 1), (370, 1), (371, 2), (372, 1)], [(26, 1), (31, 2), (186, 1), (373, 3), (374, 1), (375, 10), (376, 9), (377, 1), (378, 1), (379, 1), (380, 1), (381, 2), (382, 1), (383, 8), (384, 1), (385, 4), (386, 1), (387, 1), (388, 1), (389, 5), (390, 1), (391, 1), (392, 1), (393, 1), (394, 1)], [(10, 1), (31, 2), (49, 1), (148, 1), (192, 1), (197, 1), (377, 2), (385, 1), (395, 4), (396, 1), (397, 1), (398, 1), (399, 1), (400, 1), (401, 5), (402, 1), (403, 4), (404, 1), (405, 1), (406, 1), (407, 6), (408, 1), (409, 2), (410, 1), (411, 3), (412, 1), (413, 1), (414, 5), (415, 2), (416, 1), (417, 1), (418, 1), (419, 1), (420, 1), (421, 1), (422, 1), (423, 1), (424, 1), (425, 2), (426, 1), (427, 1), (428, 2), (429, 1), (430, 1), (431, 2)], [(55, 2), (69, 1), (73, 4), (83, 2), (85, 1), (100, 1), (426, 2), (432, 1), (433, 1), (434, 1), (435, 3), (436, 1), (437, 1), (438, 2), (439, 8), (440, 1), (441, 2), (442, 1), (443, 1), (444, 2), (445, 1), (446, 1), (447, 3), (448, 5), (449, 2), (450, 1)], [(0, 2), (8, 1), (11, 2), (13, 5), (14, 4), (15, 2), (18, 1), (19, 4), (85, 1), (89, 1), (310, 1), (313, 1), (451, 2), (452, 1), (453, 3), (454, 1), (455, 1), (456, 1), (457, 2), (458, 1), (459, 2), (460, 1), (461, 1), (462, 1), (463, 2), (464, 1), (465, 1), (466, 3), (467, 2), (468, 1), (469, 1), (470, 1), (471, 1)], [(0, 2), (1, 3), (4, 1), (5, 3), (8, 1), (9, 3), (11, 6), (13, 11), (14, 5), (15, 6), (18, 1), (19, 20), (85, 2), (89, 1), (137, 1), (313, 1), (354, 3), (357, 1), (399, 1), (456, 1), (462, 1), (471, 1), (472, 1), (473, 1), (474, 1), (475, 1), (476, 1), (477, 1), (478, 1), (479, 1), (480, 1), (481, 1), (482, 1), (483, 1), (484, 1), (485, 2), (486, 2), (487, 2), (488, 3), (489, 1), (490, 1), (491, 3), (492, 1), (493, 1), (494, 1), (495, 1), (496, 1), (497, 1), (498, 5), (499, 1), (500, 1), (501, 1), (502, 1), (503, 1), (504, 1), (505, 1), (506, 1), (507, 1), (508, 2), (509, 1), (510, 1), (511, 1), (512, 1), (513, 2), (514, 1)], [(10, 1), (11, 1), (12, 1), (15, 1), (17, 2), (26, 1), (31, 2), (97, 2), (112, 1), (301, 1), (393, 1), (402, 1), (446, 5), (455, 6), (456, 2), (515, 1), (516, 1), (517, 1), (518, 1), (519, 1), (520, 1), (521, 1), (522, 1), (523, 1), (524, 5), (525, 1), (526, 1), (527, 1), (528, 1), (529, 1), (530, 1), (531, 1), (532, 1), (533, 1), (534, 1)], [(24, 1), (84, 1), (85, 1), (138, 2), (163, 1), (255, 1), (267, 2), (442, 1), (455, 3), (462, 2), (535, 1), (536, 1), (537, 2), (538, 1), (539, 6), (540, 1), (541, 4), (542, 1), (543, 1), (544, 1), (545, 4), (546, 1), (547, 1)], [(24, 1), (84, 1), (85, 1), (138, 2), (163, 1), (255, 1), (267, 2), (442, 1), (455, 3), (462, 2), (535, 1), (536, 1), (537, 2), (538, 1), (539, 6), (540, 1), (541, 4), (542, 1), (543, 1), (544, 1), (545, 4), (546, 1), (547, 1)], [(25, 1), (69, 1), (82, 1), (84, 1), (86, 2), (122, 1), (140, 1), (192, 7), (193, 1), (197, 3), (210, 2), (219, 1), (225, 1), (318, 9), (337, 2), (376, 1), (445, 1), (460, 1), (509, 2), (526, 1), (548, 2), (549, 1), (550, 1), (551, 1), (552, 1), (553, 1), (554, 1), (555, 1), (556, 1), (557, 2), (558, 2), (559, 1), (560, 1), (561, 1), (562, 1), (563, 1), (564, 1), (565, 2), (566, 1), (567, 1), (568, 1), (569, 1), (570, 1), (571, 2), (572, 1), (573, 1), (574, 2), (575, 2), (576, 1), (577, 1), (578, 1), (579, 2), (580, 1), (581, 1), (582, 1), (583, 1), (584, 2), (585, 1), (586, 1), (587, 1), (588, 2), (589, 1), (590, 1), (591, 1), (592, 1), (593, 1), (594, 1), (595, 2), (596, 1), (597, 1), (598, 1), (599, 1), (600, 1), (601, 2), (602, 1), (603, 2), (604, 1), (605, 1), (606, 2), (607, 1), (608, 1), (609, 5), (610, 1), (611, 1), (612, 1), (613, 1), (614, 2), (615, 1), (616, 2), (617, 1), (618, 1), (619, 1), (620, 1), (621, 1), (622, 1), (623, 2), (624, 2), (625, 1), (626, 1), (627, 1), (628, 1), (629, 2), (630, 1), (631, 1), (632, 1), (633, 2), (634, 1), (635, 2), (636, 1), (637, 1), (638, 1), (639, 1), (640, 4), (641, 1), (642, 1), (643, 1), (644, 1), (645, 1), (646, 1), (647, 1), (648, 1), (649, 1), (650, 1), (651, 1), (652, 1), (653, 1), (654, 1), (655, 1), (656, 2), (657, 1), (658, 1), (659, 1), (660, 1), (661, 1), (662, 1), (663, 1), (664, 1), (665, 1), (666, 1), (667, 1), (668, 1), (669, 2), (670, 1), (671, 1), (672, 1), (673, 1), (674, 1), (675, 1), (676, 1), (677, 1), (678, 1), (679, 3), (680, 1), (681, 1), (682, 1), (683, 1), (684, 1), (685, 1), (686, 1), (687, 1), (688, 1), (689, 1), (690, 1), (691, 2), (692, 1), (693, 1), (694, 1), (695, 1), (696, 1), (697, 1), (698, 2)], [(3, 4), (255, 3), (264, 2), (318, 2), (324, 1), (536, 1), (598, 1), (699, 2), (700, 1), (701, 1), (702, 1), (703, 2), (704, 1), (705, 1), (706, 5), (707, 1), (708, 1), (709, 2), (710, 1), (711, 1), (712, 1), (713, 2), (714, 7), (715, 1)], [(49, 1), (84, 1), (85, 1), (246, 2), (255, 2), (261, 1), (264, 1), (337, 1), (420, 1), (526, 1), (587, 1), (644, 1), (716, 7), (717, 1), (718, 1), (719, 3), (720, 1), (721, 1), (722, 3), (723, 1), (724, 1), (725, 1), (726, 1), (727, 1), (728, 1), (729, 1), (730, 1), (731, 1), (732, 1)], [(85, 1), (255, 4), (640, 2), (733, 1), (734, 1), (735, 1), (736, 4), (737, 1), (738, 1), (739, 1), (740, 2), (741, 1), (742, 1), (743, 3), (744, 2), (745, 1), (746, 1), (747, 1)], [(29, 1), (76, 1), (156, 1), (169, 1), (192, 1), (197, 4), (225, 1), (246, 1), (255, 1), (260, 1), (262, 2), (263, 3), (264, 1), (314, 1), (318, 4), (337, 7), (372, 15), (402, 2), (410, 2), (422, 2), (424, 2), (442, 1), (553, 1), (575, 1), (584, 3), (598, 1), (609, 6), (624, 1), (669, 1), (670, 11), (678, 7), (727, 2), (748, 2), (749, 2), (750, 2), (751, 1), (752, 1), (753, 1), (754, 2), (755, 1), (756, 1), (757, 2), (758, 1), (759, 1), (760, 2), (761, 2), (762, 8), (763, 3), (764, 2), (765, 2), (766, 2), (767, 1), (768, 1), (769, 1), (770, 1), (771, 2), (772, 2), (773, 3), (774, 1), (775, 1), (776, 1), (777, 1), (778, 1), (779, 2), (780, 1), (781, 1), (782, 1), (783, 1), (784, 1), (785, 1), (786, 1), (787, 1), (788, 1), (789, 1), (790, 2), (791, 1), (792, 1), (793, 1), (794, 1), (795, 1), (796, 3), (797, 1), (798, 1), (799, 1), (800, 1), (801, 1), (802, 1), (803, 2), (804, 1), (805, 1), (806, 1), (807, 1), (808, 5), (809, 2), (810, 2), (811, 2), (812, 1), (813, 2), (814, 1), (815, 1), (816, 3), (817, 1), (818, 2), (819, 2), (820, 1), (821, 1), (822, 2), (823, 1)], [(31, 1), (54, 20), (152, 6), (566, 1), (824, 1), (825, 1), (826, 2)], [(10, 2), (31, 2), (54, 2), (71, 16), (72, 1), (125, 4), (225, 1), (240, 2), (266, 3), (827, 1), (828, 1), (829, 1), (830, 1), (831, 2)], [(3, 1), (10, 1), (11, 1), (15, 1), (24, 1), (26, 1), (31, 2), (54, 3), (62, 8), (69, 1), (73, 1), (83, 1), (102, 1), (119, 1), (129, 1), (130, 1), (131, 1), (266, 8), (318, 1), (827, 1), (828, 1), (832, 1), (833, 1), (834, 3), (835, 2), (836, 1), (837, 1)], [(10, 1), (31, 2), (49, 1), (53, 1), (54, 15), (62, 15), (84, 1), (125, 3), (148, 1), (210, 1), (266, 11), (318, 1), (566, 1), (824, 1), (827, 1), (828, 3), (837, 1), (838, 1), (839, 1), (840, 1)], [(10, 1), (31, 1), (49, 1), (54, 14), (62, 15), (125, 4), (210, 2), (213, 1), (266, 11), (313, 1), (566, 1), (578, 1), (824, 1), (827, 1), (828, 1), (841, 1), (842, 2)], [(31, 1), (54, 17), (62, 1), (86, 2), (165, 1), (197, 1), (210, 5), (578, 1), (685, 2), (843, 1), (844, 2), (845, 1), (846, 1), (847, 1), (848, 1), (849, 1), (850, 1), (851, 1), (852, 1), (853, 1), (854, 1), (855, 2), (856, 1), (857, 1)], [(54, 3), (84, 2), (86, 7), (122, 1), (240, 1), (258, 1), (333, 1), (366, 1), (555, 1), (566, 1), (594, 1), (685, 7), (824, 1), (837, 1), (855, 3), (858, 1), (859, 3), (860, 1), (861, 2), (862, 1), (863, 1), (864, 1), (865, 1)], [(10, 1), (22, 1), (54, 9), (62, 13), (71, 3), (74, 1), (84, 1), (108, 1), (125, 2), (210, 1), (255, 1), (264, 1), (266, 9), (267, 1), (310, 1), (318, 1), (366, 1), (464, 1), (566, 1), (824, 1), (828, 4), (837, 1), (860, 1), (866, 1), (867, 1), (868, 1), (869, 1), (870, 1), (871, 1), (872, 1)], [(10, 1), (22, 1), (54, 9), (62, 13), (71, 3), (74, 1), (84, 1), (108, 1), (125, 2), (210, 1), (255, 1), (264, 1), (266, 9), (267, 1), (310, 1), (318, 1), (366, 1), (464, 1), (566, 1), (824, 1), (828, 4), (837, 1), (860, 1), (866, 1), (867, 1), (868, 1), (869, 1), (870, 1), (871, 1), (872, 1)], [(25, 1), (52, 1), (54, 1), (79, 1), (84, 2), (267, 1), (415, 1), (837, 1), (838, 3), (873, 2), (874, 1), (875, 1), (876, 1), (877, 1), (878, 1), (879, 1), (880, 1)], [(54, 2), (62, 1), (266, 3), (267, 1), (318, 1), (837, 1)], [(25, 2), (31, 1), (54, 1), (415, 1), (704, 1), (838, 6), (873, 2), (874, 1), (876, 1), (877, 1), (879, 2), (881, 1), (882, 1), (883, 1)], [(53, 1), (54, 1), (62, 3), (266, 4), (827, 1), (828, 1), (884, 1), (885, 1), (886, 1)], [(10, 1), (31, 2), (54, 1), (62, 5), (106, 1), (210, 2), (266, 3), (827, 1), (831, 1), (839, 1), (840, 2), (887, 1)], [(24, 15), (85, 9), (86, 1), (137, 5), (143, 2), (144, 2), (187, 1), (206, 1), (225, 1), (246, 1), (252, 1), (546, 1), (566, 2), (598, 1), (609, 1), (639, 1), (709, 1), (888, 12), (889, 1), (890, 1), (891, 1), (892, 1), (893, 1), (894, 4), (895, 1), (896, 1), (897, 1), (898, 1), (899, 1), (900, 1), (901, 1), (902, 1), (903, 1), (904, 1), (905, 1), (906, 1), (907, 3), (908, 17), (909, 1)], [(0, 2), (1, 2), (5, 2), (8, 1), (11, 2), (13, 5), (14, 4), (15, 2), (18, 1), (19, 7), (57, 1), (68, 1), (130, 1), (140, 1), (148, 4), (177, 3), (190, 2), (354, 6), (357, 1), (363, 1), (452, 2), (485, 2), (487, 1), (898, 1), (910, 2), (911, 1), (912, 1), (913, 1), (914, 2), (915, 1), (916, 1), (917, 1), (918, 1), (919, 1), (920, 1), (921, 1), (922, 1), (923, 8), (924, 1), (925, 4), (926, 1), (927, 1), (928, 2)], [(31, 1), (165, 1), (331, 1), (354, 1), (361, 1), (362, 2), (363, 1), (364, 1), (898, 1), (923, 1)], [(11, 1), (15, 1), (21, 1), (57, 1), (58, 1), (68, 1), (69, 1), (70, 1), (75, 1), (78, 1), (109, 1), (137, 1), (153, 2), (178, 1), (179, 1), (184, 2), (212, 1), (301, 1), (395, 1), (396, 1), (866, 1), (929, 1), (930, 1), (931, 1), (932, 1), (933, 1), (934, 1), (935, 1), (936, 1), (937, 2), (938, 1), (939, 1), (940, 1), (941, 1), (942, 1), (943, 1), (944, 1), (945, 1), (946, 1), (947, 8), (948, 1), (949, 1), (950, 1), (951, 1), (952, 1), (953, 1), (954, 1), (955, 1), (956, 1), (957, 2), (958, 1), (959, 2), (960, 1), (961, 1), (962, 1), (963, 1), (964, 1), (965, 1), (966, 1)], [(0, 2), (8, 1), (11, 2), (13, 3), (14, 4), (15, 2), (18, 1), (19, 3), (89, 1), (148, 1), (165, 1), (197, 1), (528, 2), (967, 3), (968, 1), (969, 1), (970, 1), (971, 1), (972, 8), (973, 2), (974, 1), (975, 3), (976, 6), (977, 1), (978, 4), (979, 1), (980, 1), (981, 1), (982, 2), (983, 1), (984, 2), (985, 1)], [(10, 1), (31, 1), (49, 1), (143, 5), (193, 1), (197, 2), (213, 1), (263, 1), (302, 3), (573, 1), (629, 1), (902, 2), (986, 1), (987, 1), (988, 2), (989, 1), (990, 1), (991, 3), (992, 3), (993, 4), (994, 1), (995, 1), (996, 3), (997, 16), (998, 1), (999, 1), (1000, 1), (1001, 1), (1002, 1), (1003, 1), (1004, 1), (1005, 1), (1006, 1), (1007, 4), (1008, 9)], [(3, 1), (9, 1), (11, 1), (15, 1), (17, 1), (24, 3), (26, 1), (31, 2), (69, 1), (97, 1), (129, 1), (130, 1), (131, 1), (141, 1), (143, 5), (195, 1), (342, 2), (343, 3), (369, 1), (833, 1), (836, 1), (1009, 1), (1010, 1), (1011, 1), (1012, 1), (1013, 1), (1014, 1), (1015, 1), (1016, 1), (1017, 2), (1018, 1), (1019, 1), (1020, 1), (1021, 1), (1022, 1), (1023, 1)], [(24, 1), (85, 1), (108, 1), (143, 8), (192, 3), (197, 2), (866, 1), (1010, 3), (1017, 5), (1024, 1), (1025, 1), (1026, 1), (1027, 1)], [(24, 1), (85, 1), (108, 1), (143, 8), (192, 3), (197, 2), (866, 1), (1010, 3), (1017, 5), (1024, 1), (1025, 1), (1026, 1), (1027, 1)], [(141, 1), (143, 4), (1015, 1), (1017, 1), (1026, 1), (1028, 1)], [(0, 2), (1, 1), (3, 1), (4, 2), (6, 2), (9, 2), (10, 1), (11, 2), (12, 3), (13, 6), (14, 1), (15, 2), (16, 1), (18, 2), (19, 7), (312, 2), (317, 6), (321, 1), (327, 2), (525, 2), (1016, 1), (1029, 1), (1030, 1), (1031, 1), (1032, 1)], [(0, 4), (11, 1), (13, 4), (14, 1), (15, 1), (18, 1), (19, 9), (140, 3), (167, 1), (177, 6), (317, 7), (321, 3), (525, 1), (921, 7), (1016, 4), (1029, 1), (1032, 1), (1033, 1), (1034, 1), (1035, 7)], [(0, 2), (1, 5), (5, 5), (11, 1), (13, 8), (14, 5), (15, 1), (17, 1), (18, 1), (19, 15), (24, 7), (69, 1), (73, 1), (83, 4), (85, 5), (86, 1), (97, 1), (132, 1), (137, 1), (144, 1), (148, 1), (156, 1), (168, 1), (187, 2), (197, 1), (201, 1), (206, 1), (222, 1), (250, 1), (263, 1), (276, 1), (310, 2), (317, 1), (321, 2), (354, 1), (420, 1), (488, 2), (525, 1), (580, 3), (875, 1), (889, 3), (918, 1), (1000, 2), (1016, 1), (1036, 2), (1037, 1), (1038, 1), (1039, 1), (1040, 1), (1041, 1), (1042, 1), (1043, 2), (1044, 1), (1045, 1), (1046, 1), (1047, 1), (1048, 1), (1049, 1), (1050, 1), (1051, 1), (1052, 2), (1053, 1), (1054, 1), (1055, 1), (1056, 1), (1057, 2), (1058, 1), (1059, 1), (1060, 1), (1061, 1), (1062, 1), (1063, 1), (1064, 1), (1065, 1), (1066, 1), (1067, 2), (1068, 2)], [(0, 2), (8, 1), (11, 2), (13, 3), (14, 4), (15, 2), (18, 1), (19, 3), (89, 1), (98, 1), (159, 6), (313, 1), (353, 1), (624, 1), (1002, 1), (1069, 6), (1070, 1), (1071, 1), (1072, 1), (1073, 1), (1074, 1), (1075, 1), (1076, 7), (1077, 1), (1078, 1), (1079, 1)], [(2, 1), (24, 4), (55, 1), (73, 1), (83, 4), (85, 1), (106, 1), (120, 1), (137, 1), (187, 1), (192, 1), (213, 1), (235, 1), (260, 1), (450, 1), (640, 1), (697, 1), (846, 1), (889, 1), (903, 1), (1080, 1), (1081, 1), (1082, 1), (1083, 1), (1084, 1), (1085, 1), (1086, 1), (1087, 1), (1088, 1), (1089, 2), (1090, 6), (1091, 1), (1092, 1), (1093, 1), (1094, 1)], [(2, 1), (83, 1), (85, 2), (120, 1), (246, 1), (250, 1), (255, 2), (258, 1), (264, 2), (267, 1), (366, 1), (420, 2), (515, 1), (598, 1), (704, 1), (756, 1), (812, 1), (860, 1), (994, 1), (999, 4), (1009, 1), (1095, 1), (1096, 1), (1097, 1), (1098, 9), (1099, 2), (1100, 1), (1101, 1), (1102, 1), (1103, 1), (1104, 1), (1105, 1), (1106, 1), (1107, 1), (1108, 1), (1109, 1), (1110, 1), (1111, 13), (1112, 1), (1113, 1), (1114, 3), (1115, 7), (1116, 1), (1117, 4), (1118, 5), (1119, 20), (1120, 1), (1121, 1), (1122, 1), (1123, 1)], [(0, 1), (2, 1), (44, 20), (49, 2), (69, 1), (74, 5), (84, 2), (86, 1), (106, 1), (120, 1), (192, 2), (246, 1), (255, 1), (264, 1), (312, 1), (318, 1), (384, 1), (415, 1), (426, 1), (525, 1), (640, 1), (685, 1), (854, 1), (866, 1), (872, 1), (879, 1), (1000, 2), (1124, 1), (1125, 1), (1126, 1), (1127, 1), (1128, 1), (1129, 1), (1130, 2), (1131, 1), (1132, 1), (1133, 1), (1134, 1), (1135, 1), (1136, 1), (1137, 3), (1138, 1), (1139, 3), (1140, 1), (1141, 1), (1142, 1), (1143, 1), (1144, 1), (1145, 1), (1146, 1), (1147, 1), (1148, 5), (1149, 2), (1150, 18), (1151, 1), (1152, 1)], [(0, 3), (2, 4), (11, 1), (15, 1), (24, 3), (26, 1), (49, 1), (55, 1), (56, 2), (58, 1), (73, 1), (75, 1), (83, 3), (106, 1), (120, 3), (153, 1), (187, 2), (246, 1), (260, 1), (261, 1), (318, 1), (442, 1), (724, 1), (731, 1), (903, 1), (1091, 1), (1125, 3), (1127, 1), (1134, 1), (1142, 1), (1145, 2), (1150, 2), (1153, 1), (1154, 1), (1155, 1), (1156, 1), (1157, 1), (1158, 1), (1159, 1), (1160, 1), (1161, 1), (1162, 4), (1163, 1), (1164, 1), (1165, 1), (1166, 1), (1167, 1), (1168, 1), (1169, 1), (1170, 2), (1171, 2), (1172, 1), (1173, 1)], [(2, 2), (17, 2), (44, 2), (75, 1), (76, 1), (97, 1), (106, 1), (120, 1), (153, 5), (160, 1), (178, 1), (179, 1), (184, 1), (192, 2), (312, 4), (318, 2), (371, 1), (1174, 1), (1175, 3), (1176, 1), (1177, 1), (1178, 2), (1179, 2), (1180, 2)], [], [(2, 1), (3, 1), (10, 1), (11, 2), (15, 2), (22, 2), (24, 4), (44, 3), (54, 1), (55, 1), (58, 10), (71, 1), (73, 1), (75, 2), (83, 1), (84, 1), (106, 1), (112, 1), (118, 1), (120, 1), (125, 1), (129, 1), (130, 1), (131, 1), (143, 1), (187, 1), (192, 1), (220, 1), (260, 1), (266, 1), (333, 2), (352, 1), (371, 1), (455, 1), (462, 1), (559, 1), (594, 1), (828, 1), (889, 1), (903, 1), (945, 1), (965, 1), (1034, 1), (1091, 1), (1181, 1), (1182, 1), (1183, 1), (1184, 1), (1185, 1), (1186, 1), (1187, 1), (1188, 3), (1189, 1), (1190, 1), (1191, 1), (1192, 1), (1193, 1), (1194, 1)], [(2, 1), (17, 3), (22, 2), (49, 2), (73, 1), (84, 1), (97, 2), (143, 1), (165, 1), (192, 1), (298, 1), (320, 1), (354, 1), (366, 1), (372, 1), (379, 1), (400, 1), (415, 2), (450, 1), (543, 2), (574, 1), (726, 1), (813, 1), (860, 1), (879, 2), (947, 1), (968, 2), (1017, 1), (1097, 1), (1112, 1), (1183, 1), (1195, 1), (1196, 1), (1197, 1), (1198, 1), (1199, 1), (1200, 3), (1201, 1), (1202, 1), (1203, 1), (1204, 1), (1205, 1), (1206, 1), (1207, 1), (1208, 1), (1209, 1), (1210, 1), (1211, 2), (1212, 1), (1213, 1), (1214, 1), (1215, 1), (1216, 1), (1217, 1), (1218, 1), (1219, 1), (1220, 1), (1221, 1), (1222, 1), (1223, 1), (1224, 2), (1225, 1), (1226, 1), (1227, 2), (1228, 1)], [(2, 4), (26, 1), (49, 1), (69, 1), (84, 1), (100, 1), (120, 3), (366, 1), (372, 1), (574, 3), (947, 1), (1112, 2), (1153, 1), (1198, 1), (1213, 2), (1229, 1), (1230, 1), (1231, 1), (1232, 2), (1233, 1), (1234, 1)], [(2, 1), (84, 1), (120, 1), (192, 1), (209, 1), (574, 1), (1009, 1), (1101, 1), (1112, 1), (1213, 1), (1234, 1), (1235, 1), (1236, 1), (1237, 1), (1238, 1), (1239, 1), (1240, 1)], [(0, 2), (2, 1), (17, 1), (26, 1), (44, 11), (56, 3), (70, 1), (74, 5), (78, 1), (97, 1), (120, 2), (192, 1), (197, 1), (255, 1), (264, 1), (312, 1), (318, 1), (442, 1), (446, 1), (529, 1), (756, 1), (872, 1), (1000, 1), (1057, 1), (1085, 1), (1094, 1), (1125, 2), (1130, 2), (1131, 1), (1143, 1), (1145, 3), (1148, 12), (1150, 9), (1153, 1), (1157, 1), (1241, 1), (1242, 1), (1243, 1), (1244, 1), (1245, 1), (1246, 1), (1247, 1), (1248, 1), (1249, 1), (1250, 1), (1251, 1), (1252, 1), (1253, 2), (1254, 1), (1255, 1), (1256, 1), (1257, 1)], [(2, 2), (120, 2), (122, 1), (192, 1), (555, 1), (609, 1), (697, 1), (1101, 1), (1105, 1), (1109, 1), (1118, 2), (1119, 6), (1258, 2), (1259, 1)], [(2, 2), (22, 6), (54, 2), (84, 2), (106, 1), (112, 1), (120, 2), (255, 1), (262, 7), (264, 1), (318, 1), (446, 1), (639, 3), (697, 2), (837, 1), (1000, 1), (1076, 4), (1198, 1), (1260, 1), (1261, 10), (1262, 1), (1263, 1), (1264, 7), (1265, 2), (1266, 1), (1267, 2), (1268, 4), (1269, 1)], [(2, 1), (52, 1), (58, 1), (73, 1), (120, 1), (132, 2), (192, 1), (235, 1), (255, 4), (264, 4), (267, 3), (276, 1), (338, 1), (363, 1), (420, 1), (426, 1), (515, 1), (598, 1), (609, 1), (724, 1), (740, 1), (813, 1), (825, 1), (866, 1), (1048, 1), (1052, 1), (1054, 1), (1064, 1), (1094, 1), (1150, 1), (1197, 2), (1204, 1), (1270, 1), (1271, 1), (1272, 1), (1273, 1), (1274, 1), (1275, 1), (1276, 1), (1277, 1), (1278, 1), (1279, 1), (1280, 6), (1281, 1), (1282, 1), (1283, 1), (1284, 1), (1285, 1), (1286, 1), (1287, 1), (1288, 1), (1289, 1), (1290, 2), (1291, 1), (1292, 12), (1293, 4), (1294, 1), (1295, 1), (1296, 1), (1297, 1), (1298, 1), (1299, 1), (1300, 20), (1301, 4), (1302, 1), (1303, 1), (1304, 1), (1305, 1), (1306, 1), (1307, 1)], [(2, 1), (52, 1), (58, 1), (73, 1), (120, 1), (132, 2), (192, 1), (235, 1), (255, 4), (264, 3), (267, 3), (276, 1), (338, 1), (363, 1), (420, 1), (426, 1), (515, 1), (598, 1), (609, 1), (724, 1), (813, 1), (825, 1), (866, 1), (1048, 1), (1052, 1), (1054, 1), (1064, 1), (1094, 1), (1150, 1), (1197, 2), (1204, 1), (1270, 1), (1271, 1), (1272, 1), (1273, 1), (1275, 1), (1276, 1), (1277, 1), (1278, 1), (1280, 5), (1281, 1), (1282, 1), (1284, 1), (1285, 1), (1286, 1), (1287, 1), (1288, 1), (1289, 1), (1290, 2), (1291, 1), (1292, 12), (1293, 1), (1296, 1), (1297, 1), (1298, 1), (1299, 1), (1300, 17), (1301, 3), (1303, 1), (1305, 1), (1306, 1), (1307, 1)], [(2, 2), (3, 1), (10, 1), (11, 1), (15, 1), (17, 1), (22, 4), (24, 2), (84, 4), (85, 1), (97, 1), (102, 2), (112, 5), (120, 2), (127, 1), (129, 1), (130, 1), (131, 1), (137, 2), (143, 4), (145, 1), (146, 2), (192, 1), (197, 1), (207, 1), (224, 1), (318, 1), (446, 1), (609, 1), (639, 3), (697, 1), (797, 2), (840, 1), (1017, 1), (1076, 2), (1149, 2), (1165, 4), (1261, 13), (1263, 1), (1267, 2), (1268, 4), (1308, 1), (1309, 1), (1310, 1), (1311, 1), (1312, 1), (1313, 1)], [(20, 2), (45, 1), (53, 3), (72, 1), (82, 4), (87, 1), (113, 5), (117, 4), (125, 1), (141, 1), (145, 1), (153, 1), (165, 2), (223, 1), (355, 1), (370, 1), (471, 1), (607, 1), (740, 1), (795, 2), (930, 3), (936, 3), (951, 2), (966, 1), (969, 2), (980, 1), (998, 1), (1075, 1), (1145, 1), (1173, 1), (1195, 1), (1268, 1), (1314, 2), (1315, 1), (1316, 1), (1317, 1), (1318, 1), (1319, 1), (1320, 1), (1321, 1), (1322, 1), (1323, 2), (1324, 1), (1325, 2), (1326, 1), (1327, 3), (1328, 1), (1329, 1), (1330, 1), (1331, 1), (1332, 2), (1333, 1), (1334, 1), (1335, 2), (1336, 1), (1337, 1), (1338, 1), (1339, 2), (1340, 1), (1341, 2), (1342, 4), (1343, 1), (1344, 1), (1345, 1), (1346, 2), (1347, 1), (1348, 1), (1349, 1), (1350, 1), (1351, 1), (1352, 3), (1353, 1), (1354, 1), (1355, 1), (1356, 1), (1357, 2), (1358, 1), (1359, 2), (1360, 1), (1361, 1), (1362, 1), (1363, 1), (1364, 1), (1365, 2), (1366, 6), (1367, 2), (1368, 1), (1369, 1), (1370, 2), (1371, 1), (1372, 2), (1373, 1), (1374, 2), (1375, 3), (1376, 2), (1377, 1), (1378, 3), (1379, 1), (1380, 1), (1381, 1), (1382, 2), (1383, 1), (1384, 2), (1385, 1), (1386, 1), (1387, 1), (1388, 1), (1389, 1), (1390, 3), (1391, 1), (1392, 2), (1393, 1), (1394, 1), (1395, 1), (1396, 5), (1397, 1), (1398, 1), (1399, 1), (1400, 1), (1401, 1), (1402, 1), (1403, 1), (1404, 1), (1405, 1), (1406, 1), (1407, 1), (1408, 1), (1409, 1), (1410, 1), (1411, 1), (1412, 1), (1413, 1), (1414, 2), (1415, 1), (1416, 1), (1417, 1), (1418, 2), (1419, 1), (1420, 1), (1421, 1), (1422, 1), (1423, 1), (1424, 1), (1425, 1), (1426, 1), (1427, 3), (1428, 1), (1429, 1), (1430, 1), (1431, 1), (1432, 1), (1433, 1), (1434, 1), (1435, 1), (1436, 1), (1437, 1), (1438, 1), (1439, 1), (1440, 3), (1441, 1), (1442, 1), (1443, 1), (1444, 1), (1445, 1), (1446, 1), (1447, 1), (1448, 1), (1449, 1), (1450, 1), (1451, 1), (1452, 1), (1453, 2), (1454, 2), (1455, 4), (1456, 1), (1457, 1), (1458, 1), (1459, 1), (1460, 1), (1461, 1), (1462, 2), (1463, 1), (1464, 1), (1465, 1), (1466, 1), (1467, 1), (1468, 1), (1469, 1), (1470, 1), (1471, 1), (1472, 2), (1473, 1), (1474, 3), (1475, 1), (1476, 1), (1477, 1), (1478, 1), (1479, 1), (1480, 1), (1481, 1), (1482, 1), (1483, 2), (1484, 1), (1485, 1), (1486, 1)], [(9, 1), (22, 1), (24, 8), (55, 1), (60, 1), (67, 1), (73, 1), (83, 3), (85, 1), (203, 2), (206, 1), (231, 1), (242, 2), (258, 1), (267, 2), (477, 1), (509, 1), (697, 1), (889, 2), (995, 1), (1001, 1), (1052, 1), (1076, 1), (1101, 1), (1108, 1), (1116, 1), (1121, 1), (1123, 1), (1142, 1), (1186, 1), (1274, 1), (1487, 1), (1488, 1), (1489, 1), (1490, 1), (1491, 1), (1492, 1), (1493, 1), (1494, 1), (1495, 22), (1496, 1), (1497, 1), (1498, 1), (1499, 1), (1500, 2), (1501, 1), (1502, 1), (1503, 1), (1504, 1), (1505, 1), (1506, 1), (1507, 1), (1508, 1), (1509, 1), (1510, 1), (1511, 1), (1512, 1), (1513, 1), (1514, 1), (1515, 1), (1516, 1), (1517, 1), (1518, 4)], [(22, 1), (24, 1), (26, 1), (44, 1), (71, 1), (112, 1), (118, 1), (150, 1), (161, 1), (266, 1), (547, 1), (825, 1), (947, 1), (1000, 1), (1034, 1), (1268, 1), (1273, 1), (1306, 1), (1519, 1), (1520, 1), (1521, 1), (1522, 1), (1523, 1), (1524, 2), (1525, 1), (1526, 1), (1527, 1), (1528, 1), (1529, 1), (1530, 1), (1531, 2), (1532, 1), (1533, 1), (1534, 1), (1535, 1), (1536, 1), (1537, 1), (1538, 1), (1539, 1), (1540, 1), (1541, 1), (1542, 1), (1543, 1), (1544, 1), (1545, 1)], [(3, 3), (10, 1), (11, 1), (15, 1), (17, 1), (24, 1), (44, 1), (45, 1), (49, 1), (55, 1), (70, 1), (72, 1), (73, 2), (82, 1), (83, 5), (91, 1), (97, 1), (106, 13), (120, 3), (165, 1), (167, 1), (186, 1), (192, 1), (194, 1), (195, 1), (197, 1), (201, 2), (212, 1), (213, 1), (224, 2), (246, 2), (252, 1), (255, 3), (258, 1), (260, 1), (263, 1), (264, 3), (273, 1), (310, 1), (314, 1), (365, 1), (369, 1), (408, 1), (450, 1), (467, 2), (489, 1), (509, 1), (525, 1), (575, 2), (587, 1), (598, 1), (634, 1), (637, 1), (639, 1), (697, 5), (702, 1), (797, 2), (800, 1), (840, 1), (845, 2), (866, 1), (932, 1), (936, 1), (938, 1), (951, 1), (975, 1), (994, 1), (999, 1), (1004, 1), (1011, 2), (1016, 2), (1023, 1), (1028, 3), (1080, 2), (1084, 1), (1087, 1), (1134, 1), (1144, 1), (1150, 2), (1204, 1), (1245, 1), (1273, 1), (1306, 1), (1326, 1), (1332, 1), (1342, 1), (1443, 1), (1495, 4), (1498, 1), (1530, 1), (1546, 1), (1547, 1), (1548, 1), (1549, 2), (1550, 1), (1551, 1), (1552, 1), (1553, 1), (1554, 1), (1555, 1), (1556, 1), (1557, 1), (1558, 1), (1559, 1), (1560, 1), (1561, 1), (1562, 1), (1563, 1), (1564, 1), (1565, 1), (1566, 1), (1567, 1), (1568, 1), (1569, 1), (1570, 1), (1571, 1), (1572, 1), (1573, 1), (1574, 1), (1575, 1), (1576, 1), (1577, 1), (1578, 1), (1579, 1), (1580, 1), (1581, 1), (1582, 2), (1583, 1), (1584, 1), (1585, 1), (1586, 1), (1587, 1), (1588, 1), (1589, 1), (1590, 1), (1591, 1), (1592, 1), (1593, 1), (1594, 1), (1595, 1), (1596, 1), (1597, 1), (1598, 1), (1599, 1), (1600, 1), (1601, 1), (1602, 1), (1603, 1), (1604, 1), (1605, 1), (1606, 1), (1607, 1), (1608, 1), (1609, 1)], [(9, 1), (22, 1), (24, 8), (55, 1), (60, 1), (67, 1), (73, 1), (83, 3), (85, 1), (203, 2), (206, 1), (231, 1), (242, 2), (258, 1), (267, 2), (477, 1), (509, 1), (697, 1), (889, 2), (995, 1), (1001, 1), (1052, 1), (1076, 1), (1101, 1), (1108, 1), (1116, 1), (1121, 1), (1123, 1), (1142, 1), (1186, 1), (1274, 1), (1487, 1), (1488, 1), (1489, 1), (1490, 1), (1491, 1), (1492, 1), (1493, 1), (1494, 1), (1495, 22), (1496, 1), (1497, 1), (1498, 1), (1499, 1), (1500, 2), (1501, 1), (1502, 1), (1503, 1), (1504, 1), (1505, 1), (1506, 1), (1507, 1), (1508, 1), (1509, 1), (1510, 1), (1511, 1), (1512, 1), (1513, 1), (1514, 1), (1515, 1), (1516, 1), (1517, 1), (1518, 4)], [], [(22, 1), (26, 1), (44, 4), (49, 1), (54, 1), (71, 4), (106, 2), (110, 1), (112, 6), (123, 1), (128, 1), (156, 2), (159, 1), (197, 2), (231, 1), (357, 2), (697, 2), (825, 2), (837, 1), (867, 1), (1043, 1), (1134, 1), (1167, 1), (1197, 3), (1204, 1), (1273, 1), (1530, 2), (1610, 1), (1611, 1), (1612, 1), (1613, 1), (1614, 1), (1615, 1), (1616, 1), (1617, 1), (1618, 1), (1619, 1), (1620, 1), (1621, 1), (1622, 2), (1623, 1), (1624, 1), (1625, 1), (1626, 1), (1627, 1), (1628, 1), (1629, 1), (1630, 1), (1631, 1)], [(22, 15), (26, 1), (51, 1), (69, 1), (82, 1), (90, 3), (95, 3), (98, 1), (101, 1), (127, 1), (165, 1), (197, 1), (246, 1), (261, 1), (594, 1), (1153, 1), (1164, 1), (1167, 1), (1226, 1), (1632, 1), (1633, 1), (1634, 1), (1635, 1), (1636, 1), (1637, 1), (1638, 1), (1639, 7), (1640, 1), (1641, 1), (1642, 8), (1643, 4), (1644, 1)], [(22, 1), (26, 1), (49, 1), (69, 1), (164, 7), (165, 1), (197, 2), (212, 1), (273, 1), (697, 1), (1048, 1), (1153, 1), (1198, 1), (1645, 1), (1646, 9), (1647, 1), (1648, 1), (1649, 1), (1650, 3), (1651, 1), (1652, 1), (1653, 1), (1654, 1), (1655, 1), (1656, 1), (1657, 1), (1658, 1), (1659, 1), (1660, 1), (1661, 1), (1662, 1), (1663, 2), (1664, 1), (1665, 1)], [(3, 1), (22, 1), (24, 5), (85, 1), (192, 1), (258, 1), (261, 1), (366, 1), (372, 2), (408, 7), (515, 1), (697, 1), (907, 1), (947, 2), (1101, 1), (1112, 1), (1158, 1), (1235, 1), (1241, 1), (1666, 1), (1667, 1), (1668, 1), (1669, 1), (1670, 1), (1671, 1), (1672, 2), (1673, 2), (1674, 1), (1675, 1), (1676, 1), (1677, 1)], [(17, 2), (22, 3), (49, 3), (54, 4), (71, 2), (97, 2), (106, 2), (112, 1), (241, 11), (266, 4), (446, 1), (639, 3), (697, 2), (828, 1), (1261, 6), (1263, 1), (1267, 2), (1268, 4), (1678, 2), (1679, 1), (1680, 1), (1681, 4)], [(22, 1), (44, 4), (71, 2), (106, 2), (110, 1), (112, 4), (159, 1), (197, 2), (357, 1), (515, 1), (697, 2), (825, 1), (867, 1), (1101, 1), (1134, 1), (1197, 4), (1204, 2), (1273, 1), (1530, 1), (1610, 1), (1611, 1), (1612, 1), (1613, 1), (1615, 1), (1622, 1), (1623, 1), (1626, 1), (1627, 1), (1628, 1), (1631, 1), (1682, 1), (1683, 1), (1684, 1), (1685, 1), (1686, 1)], [(22, 1), (44, 4), (76, 1), (106, 1), (109, 2), (197, 2), (212, 1), (255, 1), (264, 1), (366, 1), (429, 1), (450, 1), (515, 1), (724, 1), (860, 1), (945, 1), (947, 1), (959, 1), (965, 1), (988, 1), (1101, 1), (1126, 2), (1134, 1), (1197, 10), (1204, 1), (1273, 6), (1274, 1), (1295, 1), (1302, 1), (1303, 1), (1306, 2), (1508, 1), (1610, 1), (1612, 4), (1626, 1), (1628, 1), (1631, 4), (1687, 1), (1688, 1), (1689, 2), (1690, 1), (1691, 1), (1692, 1), (1693, 1), (1694, 3), (1695, 1), (1696, 1), (1697, 1), (1698, 1), (1699, 1), (1700, 1)], [(22, 1), (71, 2), (110, 1), (112, 4), (159, 1), (357, 1), (825, 1), (867, 1), (1101, 1), (1530, 1), (1610, 1), (1611, 1), (1613, 1), (1615, 1), (1622, 1), (1623, 1), (1627, 1), (1682, 1), (1683, 1), (1684, 1), (1685, 1), (1686, 1)], [(22, 6), (24, 1), (26, 1), (70, 1), (127, 1), (164, 3), (212, 1), (213, 1), (231, 1), (235, 1), (238, 1), (240, 1), (243, 1), (263, 1), (384, 1), (536, 1), (609, 2), (697, 2), (732, 1), (1153, 1), (1286, 1), (1646, 4), (1650, 1), (1701, 1), (1702, 1), (1703, 2), (1704, 1), (1705, 1), (1706, 1), (1707, 1), (1708, 1), (1709, 1), (1710, 1), (1711, 1), (1712, 1), (1713, 1), (1714, 1)], [(11, 2), (15, 2), (17, 1), (24, 15), (49, 1), (55, 2), (58, 2), (69, 2), (73, 2), (75, 1), (83, 14), (84, 3), (85, 2), (97, 1), (98, 1), (106, 5), (132, 1), (187, 1), (192, 1), (206, 3), (213, 1), (258, 1), (260, 1), (263, 1), (276, 1), (396, 1), (450, 1), (517, 1), (520, 1), (526, 1), (697, 3), (813, 2), (846, 1), (889, 2), (903, 1), (1044, 1), (1054, 1), (1059, 1), (1064, 1), (1091, 4), (1093, 2), (1495, 2), (1654, 1), (1656, 1), (1662, 1), (1715, 10), (1716, 1), (1717, 1), (1718, 2), (1719, 1), (1720, 1), (1721, 1), (1722, 1), (1723, 1), (1724, 1), (1725, 1), (1726, 1), (1727, 1), (1728, 1), (1729, 1), (1730, 2), (1731, 1), (1732, 1), (1733, 2), (1734, 1), (1735, 1), (1736, 1), (1737, 1), (1738, 2), (1739, 1), (1740, 1)], [(3, 1), (10, 1), (11, 1), (15, 1), (22, 2), (24, 10), (40, 1), (44, 1), (49, 9), (55, 1), (56, 2), (73, 2), (83, 5), (84, 1), (85, 1), (102, 1), (106, 1), (110, 1), (112, 3), (118, 1), (129, 1), (130, 1), (131, 1), (143, 1), (146, 1), (187, 2), (207, 1), (261, 2), (446, 1), (639, 2), (740, 2), (1016, 1), (1017, 1), (1060, 1), (1076, 1), (1134, 2), (1158, 2), (1261, 13), (1263, 1), (1267, 3), (1268, 3), (1550, 1), (1632, 1), (1741, 2), (1742, 1), (1743, 1), (1744, 1), (1745, 1), (1746, 1), (1747, 1), (1748, 1), (1749, 1), (1750, 1), (1751, 1), (1752, 1), (1753, 3), (1754, 1), (1755, 1), (1756, 1), (1757, 1), (1758, 1), (1759, 1), (1760, 3), (1761, 1), (1762, 1), (1763, 1), (1764, 1), (1765, 1)], [(24, 1), (44, 1), (49, 5), (73, 1), (84, 3), (112, 1), (197, 1), (261, 1), (446, 1), (639, 3), (1158, 1), (1261, 5), (1263, 1), (1267, 2), (1268, 4), (1741, 7), (1760, 5), (1764, 2), (1766, 1), (1767, 1)], [(3, 1), (10, 1), (11, 1), (15, 1), (22, 2), (24, 9), (55, 1), (56, 1), (73, 1), (83, 2), (84, 2), (91, 1), (96, 1), (112, 1), (125, 1), (129, 1), (130, 1), (131, 1), (143, 1), (146, 1), (207, 1), (213, 1), (235, 1), (446, 1), (587, 1), (639, 4), (834, 2), (835, 1), (1017, 1), (1060, 1), (1150, 1), (1163, 1), (1261, 12), (1263, 1), (1267, 2), (1268, 4), (1312, 1), (1741, 1), (1752, 1), (1763, 1), (1768, 1), (1769, 1), (1770, 1), (1771, 1), (1772, 1), (1773, 2), (1774, 1), (1775, 1), (1776, 1)], [(17, 1), (22, 1), (26, 2), (44, 2), (49, 1), (69, 1), (70, 2), (78, 2), (84, 4), (96, 1), (106, 1), (110, 3), (112, 1), (116, 1), (117, 3), (118, 5), (127, 6), (224, 1), (318, 1), (384, 1), (943, 1), (984, 1), (1198, 1), (1755, 1), (1777, 2), (1778, 1), (1779, 1), (1780, 1), (1781, 1), (1782, 2), (1783, 1), (1784, 1), (1785, 1), (1786, 1)], [(22, 1), (54, 9), (62, 12), (71, 3), (74, 1), (84, 1), (125, 2), (210, 1), (255, 1), (264, 1), (266, 7), (267, 1), (310, 1), (318, 1), (366, 1), (464, 1), (515, 1), (566, 1), (724, 1), (824, 1), (828, 4), (837, 1), (860, 1), (866, 1), (867, 1), (868, 1), (869, 1), (870, 1), (871, 1), (872, 1), (1101, 1), (1787, 1), (1788, 1), (1789, 1), (1790, 1), (1791, 1), (1792, 1)], [(3, 2), (10, 1), (11, 1), (15, 1), (17, 3), (22, 10), (24, 2), (52, 1), (55, 1), (56, 1), (69, 1), (70, 1), (73, 1), (79, 1), (83, 1), (84, 2), (90, 1), (95, 1), (137, 1), (617, 1), (867, 1), (932, 1), (1347, 1), (1703, 1), (1793, 2), (1794, 1), (1795, 1), (1796, 1), (1797, 1), (1798, 1), (1799, 1), (1800, 1)], [(3, 3), (10, 3), (11, 7), (15, 7), (22, 1), (24, 2), (31, 1), (54, 1), (55, 1), (69, 1), (70, 1), (73, 1), (78, 1), (83, 2), (84, 1), (85, 2), (129, 1), (130, 1), (131, 1), (137, 1), (143, 1), (150, 1), (159, 1), (160, 1), (207, 1), (220, 2), (266, 1), (267, 3), (786, 1), (797, 1), (825, 1), (834, 2), (835, 2), (1017, 1), (1101, 1), (1149, 1), (1235, 1), (1545, 1), (1565, 1), (1801, 1), (1802, 1), (1803, 1), (1804, 1), (1805, 1), (1806, 1), (1807, 1), (1808, 1), (1809, 1), (1810, 1), (1811, 1)], [(22, 1), (24, 22), (26, 2), (49, 2), (55, 1), (69, 1), (73, 1), (80, 1), (83, 2), (106, 4), (118, 3), (127, 1), (132, 1), (137, 3), (165, 1), (192, 2), (193, 1), (206, 2), (220, 2), (231, 1), (246, 1), (255, 2), (318, 2), (328, 1), (396, 1), (442, 1), (547, 2), (598, 2), (609, 1), (636, 1), (669, 1), (698, 2), (797, 2), (815, 1), (889, 3), (898, 2), (992, 1), (1000, 1), (1009, 1), (1034, 2), (1052, 1), (1057, 1), (1083, 1), (1153, 3), (1212, 1), (1249, 1), (1305, 2), (1531, 1), (1550, 1), (1607, 1), (1654, 1), (1656, 1), (1657, 9), (1662, 1), (1663, 10), (1703, 1), (1705, 1), (1723, 11), (1786, 1), (1810, 1), (1812, 6), (1813, 1), (1814, 1), (1815, 1), (1816, 2), (1817, 1), (1818, 4), (1819, 1), (1820, 1), (1821, 1), (1822, 5), (1823, 1), (1824, 2), (1825, 1), (1826, 1), (1827, 1), (1828, 1), (1829, 1), (1830, 1), (1831, 1), (1832, 1), (1833, 1), (1834, 1), (1835, 1), (1836, 1), (1837, 2), (1838, 1), (1839, 1), (1840, 1), (1841, 1), (1842, 1), (1843, 1), (1844, 2), (1845, 1), (1846, 13), (1847, 1), (1848, 2), (1849, 1), (1850, 1), (1851, 1)], [(174, 1), (258, 1), (456, 1), (486, 1), (515, 1), (543, 1), (697, 1), (797, 2), (1852, 1), (1853, 4), (1854, 1), (1855, 1), (1856, 1), (1857, 2), (1858, 1), (1859, 1), (1860, 1), (1861, 1)], [(52, 1), (83, 2), (192, 1), (225, 1), (255, 1), (258, 2), (264, 1), (267, 2), (332, 1), (455, 1), (462, 1), (486, 3), (498, 1), (594, 1), (986, 1), (987, 1), (988, 1), (1009, 1), (1097, 1), (1101, 1), (1142, 1), (1183, 1), (1547, 1), (1699, 1), (1711, 1), (1731, 1), (1862, 1), (1863, 1), (1864, 1), (1865, 1), (1866, 2), (1867, 1), (1868, 10), (1869, 1), (1870, 1), (1871, 1), (1872, 1), (1873, 1), (1874, 1), (1875, 1), (1876, 2), (1877, 1), (1878, 1), (1879, 1), (1880, 2), (1881, 2), (1882, 2), (1883, 1), (1884, 1), (1885, 1)], [(174, 1), (456, 1), (486, 1), (515, 1), (543, 1), (697, 1), (797, 3), (1852, 1), (1853, 7), (1856, 1), (1857, 3), (1858, 1), (1859, 1), (1860, 2), (1861, 1), (1886, 1), (1887, 2), (1888, 1), (1889, 1), (1890, 1), (1891, 1), (1892, 1), (1893, 1), (1894, 1), (1895, 1)], [(15, 1), (24, 1), (84, 1), (85, 1), (138, 2), (163, 1), (255, 1), (267, 2), (442, 1), (455, 2), (462, 2), (515, 1), (535, 1), (536, 1), (537, 2), (538, 1), (539, 7), (540, 1), (541, 3), (542, 1), (543, 1), (544, 1), (545, 4), (546, 1), (547, 1), (1101, 1), (1235, 1), (1667, 1), (1802, 1), (1806, 1), (1896, 1), (1897, 1)], [(15, 1), (85, 1), (163, 1), (255, 1), (267, 1), (455, 1), (462, 1), (535, 1), (536, 1), (537, 1), (538, 1), (539, 2), (541, 1), (542, 1), (544, 1), (545, 1), (546, 1), (1101, 1), (1235, 1), (1802, 1), (1806, 1), (1896, 1), (1897, 1)], [], [(0, 2), (4, 1), (17, 1), (24, 1), (26, 1), (31, 1), (49, 1), (69, 1), (70, 2), (91, 1), (96, 1), (97, 1), (105, 1), (120, 2), (137, 2), (165, 1), (192, 1), (246, 1), (255, 2), (264, 2), (310, 1), (316, 1), (350, 2), (420, 1), (455, 2), (456, 1), (485, 1), (519, 2), (525, 3), (528, 1), (563, 1), (595, 1), (859, 1), (867, 1), (932, 1), (936, 2), (967, 1), (980, 3), (1036, 1), (1043, 1), (1125, 1), (1131, 1), (1144, 1), (1183, 5), (1186, 3), (1274, 1), (1443, 1), (1501, 1), (1575, 2), (1594, 8), (1752, 2), (1898, 2), (1899, 1), (1900, 1), (1901, 1), (1902, 1), (1903, 1), (1904, 1), (1905, 1), (1906, 1), (1907, 1), (1908, 1), (1909, 1), (1910, 1), (1911, 1), (1912, 1), (1913, 1), (1914, 1), (1915, 1), (1916, 1), (1917, 1), (1918, 2), (1919, 2), (1920, 1), (1921, 1), (1922, 1), (1923, 1), (1924, 1), (1925, 1), (1926, 4), (1927, 2), (1928, 1), (1929, 2), (1930, 2), (1931, 2), (1932, 1), (1933, 1), (1934, 1), (1935, 1), (1936, 1), (1937, 2), (1938, 1), (1939, 2), (1940, 2), (1941, 1), (1942, 2), (1943, 1), (1944, 1), (1945, 11)], [(26, 1), (49, 1), (55, 2), (73, 3), (83, 11), (85, 1), (106, 1), (137, 2), (143, 3), (213, 2), (221, 1), (235, 3), (263, 3), (318, 1), (328, 1), (435, 1), (442, 1), (455, 1), (462, 1), (496, 1), (536, 1), (598, 3), (637, 1), (638, 1), (762, 2), (797, 1), (812, 2), (902, 7), (992, 15), (1064, 7), (1094, 2), (1153, 1), (1347, 1), (1500, 1), (1623, 1), (1705, 1), (1737, 3), (1838, 5), (1946, 2), (1947, 1), (1948, 1), (1949, 1), (1950, 1), (1951, 1), (1952, 1), (1953, 1), (1954, 1), (1955, 1), (1956, 1), (1957, 1), (1958, 1), (1959, 1), (1960, 1), (1961, 1), (1962, 4), (1963, 1), (1964, 1), (1965, 1), (1966, 1), (1967, 1), (1968, 2), (1969, 1), (1970, 14), (1971, 2), (1972, 1)], [(2, 1), (3, 1), (10, 1), (11, 2), (15, 2), (22, 2), (24, 4), (44, 3), (54, 1), (55, 1), (58, 10), (71, 1), (73, 1), (75, 2), (83, 1), (84, 1), (106, 1), (112, 1), (118, 1), (120, 1), (125, 1), (129, 1), (130, 1), (131, 1), (143, 1), (187, 1), (192, 1), (220, 1), (260, 1), (266, 1), (333, 2), (352, 1), (371, 1), (455, 1), (462, 1), (559, 1), (594, 1), (828, 1), (889, 1), (903, 1), (945, 1), (965, 1), (1034, 1), (1091, 1), (1181, 1), (1182, 1), (1183, 1), (1184, 1), (1185, 1), (1186, 1), (1187, 1), (1188, 3), (1189, 1), (1190, 1), (1191, 1), (1192, 1), (1193, 1), (1194, 1)], [(49, 1), (197, 2), (304, 6), (318, 1), (366, 1), (488, 6), (510, 1), (640, 1), (697, 2), (860, 1), (1052, 1), (1087, 1), (1095, 1), (1101, 1), (1131, 1), (1755, 1), (1843, 2), (1973, 1), (1974, 1), (1975, 1), (1976, 1), (1977, 1), (1978, 1), (1979, 1), (1980, 1), (1981, 1), (1982, 1), (1983, 1), (1984, 1), (1985, 4), (1986, 1), (1987, 1), (1988, 1), (1989, 1), (1990, 1), (1991, 1), (1992, 1), (1993, 9), (1994, 1), (1995, 1), (1996, 1)], [(55, 1), (69, 2), (73, 1), (74, 1), (83, 1), (84, 1), (85, 3), (221, 1), (258, 2), (263, 3), (455, 3), (462, 4), (637, 5), (640, 1), (669, 1), (890, 2), (1095, 1), (1101, 1), (1383, 1), (1515, 1), (1957, 3), (1997, 4), (1998, 1), (1999, 1), (2000, 2), (2001, 2), (2002, 3), (2003, 1), (2004, 1), (2005, 1), (2006, 1), (2007, 1), (2008, 4), (2009, 2), (2010, 1), (2011, 3), (2012, 1), (2013, 1), (2014, 2)], [(55, 1), (69, 1), (73, 1), (74, 1), (83, 1), (84, 1), (85, 1), (221, 1), (455, 2), (462, 2), (637, 3), (1101, 1), (1383, 1), (1515, 1), (1957, 1), (1997, 2), (1998, 1), (1999, 1), (2000, 2), (2002, 2), (2005, 1), (2006, 1), (2007, 1), (2008, 2), (2011, 1), (2012, 1), (2013, 1)], [(230, 1), (455, 1), (462, 1), (486, 1), (509, 1), (1957, 1), (2005, 1), (2011, 1), (2015, 1), (2016, 1), (2017, 1), (2018, 1), (2019, 1), (2020, 1)], [(3, 2), (10, 3), (11, 4), (15, 4), (17, 1), (24, 3), (55, 1), (56, 1), (73, 1), (83, 1), (84, 2), (85, 2), (97, 1), (143, 3), (146, 2), (150, 1), (163, 2), (197, 1), (212, 1), (267, 2), (342, 1), (354, 1), (450, 1), (455, 2), (462, 1), (515, 1), (598, 1), (609, 1), (745, 1), (850, 1), (889, 1), (1101, 1), (1183, 2), (1212, 1), (1235, 1), (1552, 1), (1565, 2), (1613, 1), (1676, 1), (1725, 1), (1732, 1), (1801, 1), (1803, 1), (1804, 1), (1808, 1), (1834, 1), (2021, 1), (2022, 1), (2023, 1), (2024, 1), (2025, 1), (2026, 1), (2027, 1), (2028, 1)], [(49, 3), (68, 1), (69, 2), (70, 1), (78, 1), (106, 2), (140, 1), (456, 1), (472, 2), (479, 1), (484, 1), (489, 1), (496, 1), (507, 2), (539, 7), (541, 4), (543, 1), (594, 1), (617, 1), (624, 1), (883, 1), (1142, 1), (1183, 24), (1196, 14), (1197, 1), (1204, 1), (1219, 9), (1273, 1), (1306, 1), (1436, 3), (1547, 1), (1596, 1), (1681, 4), (1757, 1), (2029, 1), (2030, 1), (2031, 1), (2032, 1), (2033, 1), (2034, 1), (2035, 1), (2036, 1), (2037, 1), (2038, 1), (2039, 1), (2040, 1), (2041, 4), (2042, 1), (2043, 1), (2044, 1), (2045, 1), (2046, 4), (2047, 1), (2048, 2)], [(56, 2), (267, 1), (455, 2), (462, 1), (1101, 1), (1235, 1), (1237, 1), (1238, 1), (1509, 1), (2049, 1), (2050, 1), (2051, 3), (2052, 3), (2053, 1), (2054, 4)], [], [(11, 2), (56, 1), (85, 1), (192, 1), (255, 1), (262, 1), (267, 2), (341, 1), (368, 1), (455, 1), (462, 1), (486, 1), (489, 1), (496, 2), (609, 1), (637, 1), (797, 1), (1009, 1), (1016, 1), (1101, 1), (1183, 1), (1276, 1), (1296, 1), (1564, 2), (1711, 1), (1928, 1), (2055, 1), (2056, 2), (2057, 1), (2058, 1), (2059, 1), (2060, 1), (2061, 1), (2062, 1), (2063, 1), (2064, 2)], [(262, 2), (267, 1), (455, 4), (462, 1), (489, 2), (669, 1), (1101, 1), (1681, 2), (2030, 2), (2065, 5), (2066, 1), (2067, 1), (2068, 1), (2069, 1), (2070, 1), (2071, 1), (2072, 1), (2073, 1)], [(64, 1), (192, 1), (212, 1), (366, 1), (372, 1), (455, 2), (462, 1), (479, 1), (484, 1), (488, 1), (491, 1), (507, 5), (515, 1), (539, 2), (563, 1), (860, 1), (1094, 1), (1183, 1), (1331, 1), (1700, 1), (1889, 2), (1893, 1), (1992, 1), (2006, 1), (2074, 1), (2075, 1), (2076, 1), (2077, 1), (2078, 1), (2079, 1), (2080, 1), (2081, 1), (2082, 1)], [(26, 1), (366, 1), (372, 1), (455, 1), (462, 1), (479, 1), (484, 2), (507, 4), (539, 1), (563, 1), (860, 1), (1094, 1), (1153, 1), (1183, 1), (1596, 1), (2074, 1), (2078, 1), (2082, 1), (2083, 1), (2084, 1), (2085, 1)], [(366, 1), (372, 1), (455, 1), (462, 1), (479, 1), (484, 1), (507, 3), (515, 1), (539, 2), (563, 1), (860, 1), (1094, 1), (1183, 1), (1700, 1), (1889, 1), (1893, 1), (2006, 1), (2074, 1), (2077, 1), (2078, 1), (2079, 1), (2081, 1), (2082, 1)], [(3, 2), (10, 2), (11, 1), (15, 1), (22, 10), (26, 2), (49, 4), (91, 2), (106, 2), (112, 1), (129, 1), (130, 1), (131, 2), (192, 1), (207, 1), (224, 1), (255, 1), (264, 1), (310, 1), (318, 1), (446, 1), (455, 3), (462, 1), (594, 1), (639, 2), (697, 3), (774, 1), (883, 1), (987, 1), (988, 1), (1153, 1), (1183, 2), (1196, 1), (1219, 1), (1245, 2), (1261, 10), (1263, 1), (1267, 4), (1268, 5), (1347, 1), (1582, 3), (1690, 1), (1759, 1), (1762, 1), (1828, 1), (2039, 1), (2048, 1), (2086, 1), (2087, 1), (2088, 2), (2089, 1), (2090, 1), (2091, 1), (2092, 1), (2093, 1), (2094, 1), (2095, 1), (2096, 1), (2097, 1), (2098, 4), (2099, 1)], [(49, 2), (106, 1), (112, 1), (342, 1), (446, 1), (455, 3), (462, 1), (594, 1), (639, 1), (658, 2), (752, 1), (753, 1), (883, 1), (988, 1), (1183, 2), (1196, 1), (1219, 1), (1245, 3), (1261, 4), (1263, 1), (1267, 1), (1268, 2), (1690, 1), (2039, 1), (2048, 1), (2093, 1), (2096, 1), (2100, 1), (2101, 2), (2102, 2), (2103, 2), (2104, 1), (2105, 1), (2106, 4)], [(49, 2), (102, 1), (163, 1), (174, 1), (192, 1), (197, 1), (455, 1), (462, 1), (479, 1), (510, 1), (515, 1), (537, 1), (543, 1), (595, 1), (697, 1), (1101, 1), (1568, 1), (1853, 1), (1964, 1), (1975, 1), (1978, 1), (2000, 1), (2046, 1), (2107, 1), (2108, 4), (2109, 1), (2110, 1), (2111, 2), (2112, 1), (2113, 1), (2114, 1), (2115, 2), (2116, 3)], [(24, 2), (108, 1), (156, 1), (165, 5), (192, 2), (258, 1), (354, 1), (448, 1), (449, 1), (455, 1), (462, 1), (472, 1), (637, 1), (697, 1), (812, 1), (1055, 1), (1094, 1), (1101, 1), (2006, 1), (2010, 1), (2046, 1), (2117, 3), (2118, 1), (2119, 1), (2120, 3), (2121, 1), (2122, 1), (2123, 1), (2124, 1), (2125, 1), (2126, 2), (2127, 1), (2128, 1), (2129, 1), (2130, 4), (2131, 1), (2132, 1), (2133, 1), (2134, 1), (2135, 1), (2136, 1), (2137, 1)], [(52, 1), (55, 1), (73, 1), (83, 1), (192, 2), (195, 1), (213, 5), (235, 5), (255, 1), (258, 1), (263, 1), (264, 1), (332, 1), (456, 1), (594, 1), (987, 1), (1083, 1), (1112, 1), (1122, 1), (1183, 1), (1699, 1), (1716, 1), (1731, 1), (1867, 1), (1868, 9), (1869, 1), (1870, 1), (1873, 1), (1875, 1), (1878, 1), (1879, 1), (2138, 1), (2139, 1), (2140, 1), (2141, 1)], [(192, 2), (197, 1), (366, 1), (372, 2), (455, 2), (462, 1), (479, 9), (484, 1), (507, 1), (546, 1), (566, 1), (756, 1), (860, 1), (947, 1), (1183, 1), (1596, 1), (1701, 1), (1889, 1), (1893, 1), (2006, 1), (2046, 1), (2077, 1), (2080, 1), (2081, 1), (2142, 2), (2143, 1), (2144, 1), (2145, 1), (2146, 1), (2147, 1), (2148, 1), (2149, 1), (2150, 1)], [(192, 2), (197, 1), (366, 1), (372, 2), (455, 2), (462, 1), (479, 9), (484, 1), (507, 1), (515, 1), (546, 1), (566, 1), (756, 1), (860, 1), (947, 1), (1183, 1), (1596, 1), (1701, 1), (1889, 1), (1893, 1), (2006, 1), (2046, 1), (2077, 1), (2080, 1), (2081, 1), (2142, 2), (2143, 1), (2144, 1), (2145, 1), (2146, 1), (2147, 1), (2148, 1), (2150, 1)], [(192, 1), (366, 1), (372, 1), (455, 2), (462, 1), (479, 3), (546, 1), (566, 1), (860, 1), (1183, 1), (1889, 1), (1893, 1), (2006, 1), (2046, 1), (2077, 1), (2081, 1), (2142, 1), (2144, 1), (2148, 1)], [(84, 2), (102, 1), (106, 1), (197, 1), (258, 1), (262, 1), (318, 1), (354, 1), (455, 1), (462, 1), (479, 3), (484, 1), (507, 1), (756, 1), (846, 3), (1093, 1), (1094, 1), (1183, 1), (1198, 1), (1568, 1), (1596, 1), (2046, 1), (2120, 2), (2145, 1), (2151, 1), (2152, 1), (2153, 1), (2154, 1), (2155, 1), (2156, 2)], [(17, 2), (56, 3), (73, 1), (97, 2), (131, 1), (167, 1), (192, 1), (255, 1), (261, 1), (262, 1), (267, 3), (366, 1), (450, 1), (455, 7), (462, 1), (492, 1), (496, 1), (515, 1), (543, 2), (697, 1), (724, 1), (860, 1), (866, 1), (1101, 1), (1142, 1), (1144, 1), (1182, 1), (1183, 7), (1196, 4), (1676, 1), (1684, 1), (1790, 1), (2030, 2), (2048, 1), (2071, 2), (2155, 1), (2157, 1), (2158, 1), (2159, 1), (2160, 1), (2161, 1), (2162, 1), (2163, 1), (2164, 2)], [(3, 1), (10, 2), (11, 2), (15, 2), (24, 3), (29, 1), (56, 3), (106, 2), (129, 1), (130, 1), (131, 1), (157, 1), (250, 1), (455, 1), (598, 1), (1186, 1), (1677, 3), (1754, 1), (2165, 1), (2166, 1), (2167, 1), (2168, 1), (2169, 2), (2170, 1), (2171, 1), (2172, 1), (2173, 1)], [(2, 1), (7, 1), (9, 1), (11, 2), (13, 1), (14, 6), (15, 1), (19, 10), (102, 1), (120, 2), (168, 1), (199, 1), (304, 1), (380, 1), (519, 1), (543, 2), (697, 1), (875, 2), (918, 1), (1000, 4), (1036, 3), (1038, 2), (1047, 1), (1049, 1), (1050, 1), (1051, 1), (1052, 2), (1062, 1), (1066, 1), (1196, 2), (1322, 1), (1564, 1), (1741, 7), (1754, 1), (2060, 1), (2174, 1), (2175, 1), (2176, 1), (2177, 8), (2178, 1), (2179, 1), (2180, 1), (2181, 1), (2182, 6), (2183, 1), (2184, 1), (2185, 1), (2186, 1), (2187, 1)], [(2, 4), (17, 1), (26, 1), (31, 2), (49, 1), (97, 1), (120, 4), (393, 1), (609, 1), (697, 2), (1162, 9), (2188, 1), (2189, 1), (2190, 1), (2191, 1)], [(2, 5), (11, 2), (15, 2), (26, 1), (44, 5), (68, 1), (74, 1), (120, 5), (137, 1), (212, 1), (301, 1), (362, 2), (396, 1), (400, 1), (525, 1), (607, 1), (669, 1), (671, 1), (797, 1), (857, 1), (1048, 1), (1145, 1), (1148, 4), (1150, 4), (1162, 2), (1172, 1), (1243, 1), (1253, 1), (1300, 2), (1763, 1), (1796, 1), (1895, 1), (2192, 2), (2193, 1), (2194, 1), (2195, 1), (2196, 1), (2197, 6), (2198, 4), (2199, 1), (2200, 1), (2201, 1), (2202, 9), (2203, 1), (2204, 3), (2205, 1), (2206, 1), (2207, 1), (2208, 2), (2209, 2), (2210, 2), (2211, 1), (2212, 1), (2213, 1), (2214, 1), (2215, 4), (2216, 1), (2217, 1), (2218, 2), (2219, 3), (2220, 1), (2221, 1), (2222, 1), (2223, 1)], [(0, 1), (1, 1), (3, 2), (5, 1), (8, 1), (10, 3), (11, 3), (13, 2), (14, 6), (15, 3), (18, 1), (19, 3), (20, 1), (22, 1), (24, 5), (31, 1), (73, 1), (83, 3), (85, 2), (106, 1), (110, 1), (117, 1), (129, 1), (130, 2), (131, 1), (141, 1), (143, 1), (156, 1), (160, 3), (164, 1), (178, 1), (179, 1), (184, 1), (192, 3), (204, 1), (205, 1), (207, 1), (208, 1), (213, 1), (220, 1), (229, 1), (231, 1), (233, 1), (234, 1), (235, 1), (255, 2), (301, 1), (348, 1), (350, 1), (362, 1), (493, 1), (697, 1), (705, 1), (812, 1), (944, 2), (1016, 1), (1043, 1), (1065, 1), (1071, 1), (1102, 1), (1375, 1), (1412, 1), (1542, 1), (1566, 1), (1589, 1), (1754, 1), (1996, 1), (2170, 1), (2224, 1), (2225, 1), (2226, 1), (2227, 1), (2228, 1), (2229, 1), (2230, 1), (2231, 1), (2232, 1), (2233, 1), (2234, 1), (2235, 1), (2236, 1), (2237, 1), (2238, 1), (2239, 1), (2240, 1), (2241, 1), (2242, 1), (2243, 2)], [(10, 1), (20, 1), (22, 4), (26, 1), (31, 1), (38, 21), (54, 1), (150, 1), (301, 1), (609, 1), (852, 1), (1000, 1), (2244, 5), (2245, 1), (2246, 1), (2247, 4), (2248, 1), (2249, 1), (2250, 4)], [(0, 4), (3, 3), (10, 3), (11, 2), (15, 2), (17, 1), (20, 1), (22, 13), (24, 5), (31, 1), (44, 15), (49, 9), (52, 2), (56, 1), (66, 1), (69, 1), (79, 2), (90, 2), (91, 17), (95, 3), (96, 17), (97, 1), (112, 10), (156, 2), (262, 2), (276, 1), (333, 2), (629, 10), (666, 1), (684, 1), (752, 12), (753, 12), (825, 1), (1083, 3), (1125, 4), (1203, 1), (1255, 1), (1268, 1), (1347, 1), (1582, 5), (1593, 1), (2081, 4), (2095, 7), (2251, 2), (2252, 1), (2253, 6), (2254, 1), (2255, 1), (2256, 1), (2257, 1), (2258, 5), (2259, 2), (2260, 1), (2261, 2), (2262, 1), (2263, 2), (2264, 2), (2265, 1), (2266, 1), (2267, 1), (2268, 2), (2269, 2), (2270, 5), (2271, 1), (2272, 1), (2273, 2), (2274, 1), (2275, 1), (2276, 1), (2277, 1)], [(10, 3), (20, 1), (22, 1), (26, 1), (31, 2), (44, 1), (54, 14), (62, 4), (71, 16), (72, 1), (74, 1), (86, 2), (125, 2), (240, 4), (245, 1), (266, 1), (685, 2), (827, 1), (846, 1), (852, 2), (861, 2), (898, 1), (2278, 1), (2279, 1), (2280, 1), (2281, 1), (2282, 1), (2283, 2), (2284, 2)], [(0, 1), (1, 1), (4, 1), (5, 1), (11, 1), (13, 1), (14, 7), (15, 1), (17, 1), (19, 3), (20, 1), (22, 1), (49, 1), (97, 1), (199, 1), (206, 1), (301, 1), (313, 1), (354, 1), (697, 3), (986, 1), (1000, 1), (1094, 1), (1186, 1), (1195, 1), (1216, 1), (1265, 1), (1390, 1), (1504, 1), (1587, 4), (1588, 8), (2285, 2), (2286, 1), (2287, 1), (2288, 1), (2289, 1), (2290, 1), (2291, 1), (2292, 1)]]

In [26]:
corpus_tfidf = tf_idf(corpus)

In [27]:
texts = [['depot', 'hd', 'report', 's', 'p', 'dow', 'nasdaq', 's', 'p', 'street', 'hd', 'hd', 'zacks', 'consensus', 'estimate', 'zacks', 'consensus', 'hd', 'zacks', 'rank', 'zacks', 'rank', 'strong', 'buy', 'strong', 'sell', 'zacks', 'consensus', 'eps', 'hd', 'zacks', 'rank', 'hold', 'hd', 'forward', 'forward', 'hd', 'hd', 'peg', 'peg', 'hd', 'peg', 'zacks', 'industry', 'rank', 'industry', 'rank', 'zacks', 'rank'], ['music', 'entertainment', 'nyse', 'china', 'rmb', 'rmb', 'rmb', 'ad', 'tencent', 'music', 'getty', 'tencent', 'music', 'tencent', 'music', 'spotify', 'nyse', 'apple', 'nasdaq', 'aapl', 'music', 'sirius', 'xm', 'nasdaq', 'siri', 'pandora', 'spotify', 'apple', 'music', 'pandora', 'music', 'china', 'wesing', 'tencent', 'music', 'social', 'tencent', 'music', 'arppu', 'maus', 'subscription', 'rmb', 'mobile', 'qoq', 'yoy', 'arppu', 'rmb', 'getty', 'arppu', 'mobile', 'qoq', 'yoy', 'mau', 'arppu', 'music', 'eps', 'tencent', 'music', 'apple', 'music', 'music', 'china', 'tencent', 'music', 'maus'], ['apple', 'aapl', 'report', 'supreme', 'court', 'apple', 'war', 'impact', 'apple', 'president', 'donald', 'trump', 'trump', 'friday', 'trump', 'build', 'united', 'tariff', 'trump', 'friday', 'china', 'apple', 'china', 'china', 'apple', 'xiaomi', 'apple', 'greater', 'china', 'hong', 'kong', 'taiwan', 'greater', 'china', 'apple', 'apple', 'china', 'morgan', 'stanley', 'm', 'report', 'katy', 'huberty', 'x', 'apple', 'aapl', 'caseto', 'greater', 'china', 'apple', 'ceo', 'tim', 'cook', 'service', 'spotify', 'spot', 'report', 'apple', 'music', 'apple', 'amazon', 'amzn', 'report', 'prime', 'netflix', 'nflx', 'report', 'hulu', 'disney', 'dis', 'report', 'apple', 'goldman', 'sachs', 'g', 'report', 'apple', 'apple', 'jefferies', 'supreme', 'court', 'monday', 'apple', 'brett', 'kavanaugh', 'supreme', 'court', 'apple', 'apple', 'kavanaugh', 'apple', 'apple', 'netflix', 'apple', 'apple', 'aapl', 'monday', 'apple', 'service', 'apple', 'zacks', 'conesus', 'estimate', 'apple', 'aapl', 'tuesday', 'apple', 'zacks', 'rank', 'hold', 'apple', 'zackswould', 's', 'p', 's', 'p'], ['monday', 'wall', 'street', 'china', 'china', 'apple', 'nasdaq', 'aapl', 'uber', 'nyse', 'scripps', 'nasdaq', 'ssp', 'apple', 'china', 'apple', 'apple', 'supreme', 'court', 'app', 'store', 'apple', 'ipo', 'friday', 'uber', 'scripps', 'friday', 'scripps', 'newsy'], ['supreme', 'court', 'apple', 'nasdaq', 'aapl', 'monday', 'app', 'store', 'apple', 'app', 'store', 'apple', 'court', 'apple', 'app', 'store', 'apple', 'supreme', 'court', 'applefirst', 'apple', 'apple', 'apple', 'app', 'store', 'apple', 'apple', 'nasdaq', 'nflx', 'spotify', 'nyse', 'app', 'store', 'apple', 'europe', 'apple', 'app', 'store', 'apple', 'apple', 'app', 'store', 'apple', 'apple', 'apple', 'app', 'store', 'apple', 'craig', 'lachance', 'apple', 'apple', 'developer', 'apple', 'app', 'store', 'apple', 'android', 'apple', 'apple', 'apple', 'mac', 'apple', 'store', 'apple', 'apple', 'apple', 'app', 'store', 'apple', 'app', 'store', 'app', 'store', 'supreme', 'court', 'apple', 'apple'], ['amid', 'president', 'trump', 'china', 'apple', 'nasdaq', 'aapl', 'trump', 'china', 'friday', 'cupertino', 'trump', 'cnbc', 'apple', 'x', 'apple', 'apple', 'apple', 'jpmorgan', 'apple', 'morgan', 'stanley', 'katy', 'huberty', 'apple', 'x', 'apple', 'apple', 'india', 'china'], ['s', 'p', 'dow', 'wall', 'street', 'facebook', 'fb', 'report', 'facebook', 'china', 'facebook', 'facebook', 'instagram', 'whatsapp', 'amazon', 'ceo', 'mark', 'zuckerberg', 'zacks', 'consensus', 'estimate', 'fb', 'fb', 'eps', 'fb', 'fb', 'zacks', 'rank', 'buy', 'facebook', 'fb', 'zacks', 'consensus', 'eps', 'facebook', 'googl', 'report', 'google', 'alphabet', 'gmail', 'youtube', 'china', 'great', 'firewall', 'google', 'google', 'amazon', 'amzn', 'report', 'prime', 'alphabet', 'pixel', 'home', 'alphabet', 'googl', 'eps', 'zacks', 'rank', 'hold', 'google', 'zacks', 'consensus', 'eps', 'nflx', 'report', 'los', 'gatos', 'china', 'netflix', 'china', 'netflix', 'amazon', 'hulu', 'disney', 'dis', 'report', 'apple', 'aapl', 'report', 't', 't', 'report', 'better', 'nflx', 'eps', 'netflix', 'zacks', 'rank', 'hold', 'nflx', 'tuesday', 'netflix', 'nflx', 'zackswould', 's', 'p', 's', 'p'], ['united', 'china', 'trump', 'china', 'dow', 'jones', 'industrial', 'average', 'monday', 'tuesday', 'getty', 'president', 'trump', 'china', 'monday', 'trump', 'treasury', 'china', 'china', 'china', 'china', 'apple', 'nasdaq', 'aapl', 'china', 'apple', 'china', 'china'], ['wall', 'street', 'china', 'june', 'donald', 'trump', 'dvd', 'etf', 'fear', 'trade', 'clash', 'klaus', 'baader', 'societe', 'generale', 'china', 'gdp', 'china', 'united', 'america', 'china', 'morgan', 'stanley', 'united', 'etf', 'apple', 'aapl', 'report', 'apple', 'china', 'apple', 'trump', 'zacks', 'rank', 'hold', 'vgm', 'score', 'cat', 'report', 'caterpillar', 'china', 'zacks', 'rank', 'vgm', 'score', 'b', 'dow', 'jones', 'etf', 'tuesday', 'boeing', 'company', 'ba', 'report', 'china', 'global', 'time', 'united', 'china', 'boeing', 'zacks', 'rank', 'vgm', 'score', 'daniel', 'midland', 'company', 'adm', 'report', 'zacks', 'rank', 'vgm', 'score', 'd', 'forget', 'trade', 'fear', 'invest', 'defensive', 'sector', 'etf', 'msci', 'global', 'agriculture', 'producer', 'etf', 'vegi', 'report', 'msci', 'acwi', 'select', 'agriculture', 'producer', 'investable', 'market', 'index', 'canada', 'norway', 'japan', 'italy', 'etf', 'aum', 'vector', 'agribusiness', 'etf', 'moo', 'report', 'aum', 'mvis', 'global', 'agribusiness', 'index', 'moo', 'phlx', 'semiconductor', 'etf', 'soxx', 'report', 'aum', 'etf', 'phlx', 'sox', 'semiconductor', 'sector', 'index', 'zacks', 'rank', 'trade', 'tension', 'etf', 'area', 'select', 'sector', 'spdr', 'xli', 'report', 'etf', 'aum', 'industrial', 'select', 'sector', 'index', 'etf', 'zacks', 'etf', 'rank', 'strong', 'buy', 'medium', 'etf', 'zacks', 'fund', 'newsletter', 'etf'], ['china', 'dow', 'june', 'china', 'china', 'boeing', 'ba', 'report', 'apple', 'aapl', 'report', 'march', 'tuesday', 'boeing', 'apple', 'zacks', 'rank', 'hold', 'april', 'march', 'federal', 'open', 'market', 'committee', 'fomc', 'fed', 'fed', 'president', 'trump', 'fed'], ['tuesday', 'china', 'dow', 'june', 'china', 'china', 'boeing', 'ba', 'report', 'apple', 'aapl', 'report', 'march', 'tuesday', 'boeing', 'apple', 'zacks', 'rank', 'hold', 'april', 'march', 'federal', 'open', 'market', 'committee', 'fomc', 'fed', 'fed', 'president', 'trump', 'fed', 'vickerysenior', 'editorquestions', 'click', 'today', 'zackswould', 's', 'p', 's', 'p'], ['jan', 'monday', 'china', 'fox', 'news', 'larry', 'kudlow', 'dow', 'jones', 'industrial', 'average', 'dji', 's', 'p', 'nasdaq', 'composite', 'index', 'cboe', 'volatility', 'index', 'vix', 'nyse', 'nasdaq', 'benchmark', 'perform', 'dow', 'caterpillar', 'cat', 'report', 'apple', 'aapl', 'report', 'boeing', 'ba', 'report', 'zacks', 'rank', 'hold', 'zacks', 'rank', 'strong', 'buy', 's', 'p', 's', 'p', 'jan', 's', 'p', 'technology', 'select', 'sector', 'spdr', 'fund', 'xlk', 'monday', 'nasdaq', 'microsoft', 'msft', 'report', 'president', 'trump', 'oval', 'office', 'monday', 'hungarian', 'prime', 'minister', 'viktor', 'orban', 'president', 'xi', 'jinping', 'june', 'group', 'tariff', 'goodsthe', 'chinese', 'finance', 'ministry', 'china', 'united', 'united', 'united', 'china', 'china', 'asian', 'friday', 'trump', 'china', 'united', 'say', 'tariff', 'china', 'hurt', 'partiesin', 'fox', 'news', 'white', 'house', 'larry', 'kudlow', 'united', 'china', 'china', 'kudlow', 'china', 'zackswould', 's', 'p', 's', 'p'], ['apple', 'nasdaq', 'aapl', 'apple', 'channel', 'hbo', 'showtime', 'starz', 'epix', 'apple', 'apple', 'channel', 'apple', 'app', 'channel', 'app', 'apple', 'apple', 'apple', 'channel', 'hbo', 'apple', 'tv', 'channel', 'apple', 'app', 'apple', 'samsung', 'tv', 'roku', 'amazon', 'apple', 'channel', 'apple', 'hbo', 'apple', 'apple', 'apple', 'apple', 'apple', 'amazon', 'prime', 'channel', 'apple', 'channel', 'channel', 'apple', 'app', 'apple', 'app', 'channel', 'apple', 'apple', 'apple', 'apple', 'channel', 'apple', 'channel', 'app', 'channel'], ['washington', 'reuters', 'supreme', 'court', 'monday', 'apple', 'apple', 'app', 'store', 'apple', 'apple', 'cupertino', 'president', 'donald', 'trump', 'apple', 'monday', 'justice', 'brett', 'kavanaugh', 'apple', 'mark', 'rifkin', 'apple', 'apple', 'app', 'store', 'supreme', 'court', 'kavanaugh', 'monopoly', 'apple', 'kavanaugh', 'apple', 'apple', 'app', 'store', 'kavanaugh', 'trump', 'apple', 'justice', 'neil', 'gorsuch', 'gorsuch', 'trump', 'apple', 'facebook', 'app', 'store', 'spotify', 'apple', 'apple', 'app', 'store', 'rifkin', 'apple', 'developer', 'apple', 'app', 'store', 'apple', 'apple', 'robert', 'pepper', 'chicago', 'california', 'apple', 'texas', 'california', 'new', 'york', 'apple', 'chamber', 'commerce', 'oakland', 'san', 'circuit', 'court', 'apple'], ['washington', 'reuters', 'supreme', 'court', 'monday', 'apple', 'apple', 'app', 'store', 'apple', 'apple', 'cupertino', 'president', 'donald', 'trump', 'apple', 'monday', 'justice', 'brett', 'kavanaugh', 'apple', 'mark', 'rifkin', 'apple', 'apple', 'app', 'store', 'supreme', 'court', 'kavanaugh', 'monopoly', 'apple', 'kavanaugh', 'apple', 'apple', 'app', 'store', 'kavanaugh', 'trump', 'apple', 'justice', 'neil', 'gorsuch', 'gorsuch', 'trump', 'apple', 'facebook', 'app', 'store', 'spotify', 'apple', 'apple', 'app', 'store', 'rifkin', 'apple', 'developer', 'apple', 'app', 'store', 'apple', 'apple', 'robert', 'pepper', 'chicago', 'california', 'apple', 'texas', 'california', 'new', 'york', 'apple', 'chamber', 'commerce', 'oakland', 'san', 'circuit', 'court', 'apple'], ['apple', 'monday', 'tv', 'apple', 'hulu', 'netflix', 'apple', 'apple', 'comcast', 'corp', 'apple', 'hbo', 'starz', 'showtime', 'tastemade', 'apple', 'apple', 'apple', 'apple', 'apple', 'app', 'store', 'factset', 'apple', 'samsung', 'tv', 'vizio', 'lg', 'sony', 'tv', 'airplay', 'apple', 'siri', 'netflix', 'apple', 'netflix', 'apple', 'apple', 'spectrum', 't', 'directv', 'sony', 'playstation', 'vue', 'apple', 'steven', 'spielberg', 'oprah', 'winfrey'], ['washington', 'mai', 'reuters', 'weg', 'sammelklage', 'da', 'supreme', 'court', 'washington', 'montag', 'entscheidung', 'instanz', 'sammelklage', 'die', 'kläger', 'apple', 'markt', 'apps', 'monopol', 'preise', 'künstlich', 'zu', 'überhöhen', 'apple', 'klage'], ['automatic', 'data', 'processing', 'adp', 'report', 'zacks', 'rank', 'buy', 'zacks', 'zacks', 'consensus', 'estimate', 'eps', 'zacks', 'wall', 'street', 'zacks', 'adp', 'powerful', 'force', 'impacting', 'stock', 'pricesthe', 'adp', 'power', 'estimate', 'revisionsas', 'zacks', 'rank', 'zacks', 'rank', 'zacks', 'rank', 'strong', 'buy', 'zacks', 'rank', 'strong', 'sell', 'zacks', 'rank', 'zacks', 'rank', 'strong', 'buy', 'estimate', 'revision', 'adpfor', 'june', 'adp', 'zacks', 'consensus', 'estimate', 'wall', 'street', 'zacks', 'buy', 'zacks', 'rank', 'adp', 'zacks', 'rank'], ['corporation', 'kmb', 'report', 'zacks', 'rank', 'buy', 'restructuring', 'force', 'program', 'yield', 'resultsthe', 'global', 'restructuring', 'program', 'global', 'restructuring', 'program', 'focus', 'force', 'personal', 'care', 'strategy', 'january', 'upthe', 'consumer', 'staple', 'stock', 'thesethe', 'estee', 'lauder', 'company', 'el', 'report', 'zacks', 'rank', 'buy', 'zacks', 'rank', 'strong', 'buy', 'company', 'mkc', 'report', 'zacks', 'rank', 'mill', 'gi', 'report', 'zacks', 'rank', 'zackswould', 's', 'p', 's', 'p'], ['johnson', 'johnson', 'zytiga', 'court', 'appeal', 'federal', 'circuit', 'tuesday', 'zytiga'], ['oklahoma', 'monday', 'johnson', 'johnson', 'teva', 'pharmaceutical', 'cleveland', 'county', 'district', 'judge', 'thad', 'balkman', 'norman', 'oklahoma'], ['company', 'mmm', 'report', 'zacks', 'rank', 'sell', 'price', 'performance', 'earnings', 'estimate', 'revision', 'apr', 'zacks', 'consensus', 'estimate', 'company', 'price', 'company', 'safety', 'industrial', 'transportation', 'electronics', 'carlisle', 'company', 'incorporated', 'csl', 'report', 'federal', 'signal', 'corporation', 'f', 'report', 'honeywell', 'international', 'hon', 'report', 'carlisle', 'company', 'zacks', 'rank', 'strong', 'buy', 'federal', 'signal', 'honeywell', 'international', 'zacks', 'rank', 'buy', 'zacks', 'rank', 'carlisle', 'company', 'federal', 'signal', 'honeywell', 'zackswould', 's', 'p', 's', 'p'], ['nyse', 'june', 'mmm', 'dividend', 'scorecard', 'yield', 'chart', 'dividend', 'growth'], ['mcdonald', 'mcd', 'big', 'data', 'chief', 'information', 'officer', 'wall', 'street', 'journal', 'dynamic', 'yield', 'mcdonald', 'ceo', 'march'], ['boston', 'beer', 'nyse', 'sam', 'dogfish', 'head', 'brewery', 'inbev', 'nyse', 'head', 'sam', 'calagione', 'boston', 'beer', 'image', 'getty', 'boston', 'beer', 'dogfish', 'head', 'boston', 'beer', 'inbev', 'molson', 'coors', 'revenue', 'ttm', 'head', 'delaware', 'boston', 'beer', 'sam', 'adam', 'truly', 'hard', 'seltzer', 'twisted', 'tea', 'angry', 'inbev', 'beer', 'inbev', 'sam', 'adam', 'minneapolis', 'head', 'head', 'boston', 'beer', 'time', 'boston', 'beer', 'sam', 'adam', 'head', 'boston', 'beer', 'head', 'boston', 'beer'], ['craft', 'brew', 'alliance', 'nasdaq', 'brew', 'cba', 'kona', 'brewing', 'cba', 'kona', 'widmer', 'brewing', 'redhook', 'brewery', 'october', 'mountain', 'brewery', 'cisco', 'brewer', 'wynwood', 'brewing', 'company', 'kona', 'hawaii', 'cba', 'ceo', 'andy', 'thomas', 'brexit', 'australia', 'craft', 'brew', 'alliance', 'ncaa', 'march', 'madness', 'kona', 'inbev', 'nyse', 'bud', 'cba', 'strs', 'april', 'cba', 'kona', 'strs', 'wynwood', 'la', 'rubia', 'american', 'constellation', 'brand', 'nyse', 'corona', 'modelo', 'iri', 'constellation', 'brewing', 'brew', 'alliance', 'la', 'rubia', 'puerto', 'rico', 'southern', 'caribbean', 'cba', 'pre', 'aperitivo', 'spritz', 'craft', 'brew', 'alliance'], ['juan', 'guaido', 'monday', 'carlos', 'vecchio', 'guaido', 'southern', 'command', 'guaido', 'craig', 'faller', 'command', 'guaido', 'twitter', 'southern', 'command', 'venezuela', 'venezuelan', 'vecchio', 'faller', 'guaido', 'president', 'donald', 'trump', 'venezuela', 'guaido', 'venezuela', 'national', 'assembly', 'president', 'united', 'guaido', 'venezuelan', 'president', 'nicolas', 'maduro', 'vice', 'president', 'delcy', 'rodriguez', 'vecchio', 'venezuela', 'rodriguez', 'guaido', 'caracas', 'saturday', 'donald', 'trump', 'venezuela', 'cuba', 'maduro'], ['exxon', 'mobil', 'corporation', 'xom', 'report', 'newport', 'wale', 'santoprene', 'santoprene', 'chemical', 'santoprene', 'pensacola', 'newport', 'fawley', 'exxonmobil', 'fawley', 'united', 'kingdom', 'rank', 'stock', 'considerthe', 'zacks', 'rank', 'hold', 'hess', 'he', 'report', 'cactus', 'whd', 'report', 'usa', 'compression', 'partner', 'lp', 'usac', 'report', 'hess', 'zacks', 'rank', 'strong', 'buy', 'cactus', 'usa', 'compression', 'zacks', 'rank', 'buy', 'zacks', 'rank', 'compression', 'zackswould', 's', 'p', 's', 'p'], ['zacks', 'oil', 'gas', 'integrated', 'international', 'united', 'gulf', 'mexico', 'asia', 'south', 'america', 'africa', 'australia', 'industry', 'rank', 'bleak', 'prospectsthe', 'zacks', 'oil', 'gas', 'integrated', 'international', 'zacks', 'oil', 'energy', 'zacks', 'industry', 'rank', 'zacks', 'industry', 'rank', 'zacks', 'rank', 'lag', 's', 'p', 'outperforms', 'sectorthe', 'zacks', 'oil', 'gas', 'integrated', 'international', 'zacks', 's', 'p', 'zacks', 'oil', 'energy', 's', 'p', 'price', 'performanceindustry', 'current', 'valuationsince', 'enterprise', 'tax', 'depreciation', 'amortization', 'ebitda', 's', 'p', 'enterprise', 'ebitda', 'ratiobottom', 'linethe', 'united', 'zacks', 'rank', 'strong', 'buy', 'zacks', 'rank', 'buy', 'zacks', 'rank', 'hold', 'zacks', 'rank', 'sa', 'repyy', 'report', 'madrid', 'spain', 'zacks', 'consensus', 'estimate', 'zacks', 'rank', 'repyyypf', 'sociedad', 'anonima', 'ypf', 'report', 'buenos', 'aire', 'argentina', 'ypf', 'sociedad', 'zacks', 'rank', 'zacks', 'consensus', 'estimate', 'ypfexxon', 'mobil', 'corporation', 'xom', 'report', 'irving', 'exxonmobil', 'bp', 'report', 'zacks', 'bproyal', 'dutch', 'shell', 'report', 'zacks', 'consensus', 'estimate', 'eps', 'hague', 'netherlands', 'zacks', 'rank', 'zackswould', 's', 'p', 's', 'p'], ['exelixis', 'nasdaq', 'exel', 'twitter', 'nyse', 'exxonmobil', 'nyse', 'budwell', 'exelixis', 'food', 'drug', 'exelixis', 'peg', 'getty', 'exelixis', 'squibb', 'merck', 'cabozantinib', 'exelixis', 'stoffel', 'twitter', 's', 'p', 'twitter', 'trailing', 'compound', 'data', 'cagr', 'remember', 'twitter', 'facebook', 'twitter', 'gregg', 'brewer', 'exxonmobil', 'exxon', 'clinton', 'reagan', 'george', 'bush', 'simply', 'exxon', 'ttm', 'exxon', 'exxon', 'wall', 'street', 'exxon', 'add', 'wall', 'street', 'exxon'], ['exxon', 'mobil', 'corp', 'tuesday', 'rovuma', 'lng', 'mozambique', 'east', 'african', 'lng', 'italy', 'eni', 'qatar', 'papua', 'new', 'guinea', 'russia', 'united', 'rovuma', 'lng', 'area', 'mozambique', 'exxon', 'lng', 'lng', 'mozambique', 'rovuma', 'venture', 'exxon', 'mobil', 'eni', 'china', 'national', 'petroleum', 'corp', 'rovuma', 'lng', 'mozambique', 'area', 'lpg'], ['exxon', 'mobil', 'corp', 'tuesday', 'rovuma', 'lng', 'mozambique', 'east', 'african', 'lng', 'italy', 'eni', 'qatar', 'papua', 'new', 'guinea', 'russia', 'united', 'rovuma', 'lng', 'area', 'mozambique', 'exxon', 'lng', 'lng', 'mozambique', 'rovuma', 'venture', 'exxon', 'mobil', 'eni', 'china', 'national', 'petroleum', 'corp', 'rovuma', 'lng', 'mozambique', 'area', 'lpg'], ['tim', 'conway', 'carol', 'burnett', 'tuesday', 'conway', 'doris', 'day', 'monday', 'peggy', 'lipton', 'mod', 'squad', 'twin', 'peak', 'saturday', 'kingdom', 'jim', 'fowler', 'wednesday', 'fowler', 'johnny', 'carson', 'peter', 'mayhew', 'star', 'war', 'chewbacca', 'april', 'john', 'singleton', 'boyz', 'hood', 'fast', 'furious', 'april', 'troy', 'dean', 'shafer', 'diy', 'network', 'nashville', 'flipped', 'april', 'basketball', 'hall', 'famer', 'nba', 'john', 'havlicek', 'boston', 'april', 'stefanie', 'sherk', 'april', 'demian', 'bichir', 'sherk', 'csi', 'cyber', 'valentine', 'day', 'bichir', 'sherk', 'ernest', 'fritz', 'hollings', 'south', 'carolina', 'april', 'south', 'carolina', 'rapper', 'nipsey', 'hussle', 'march', 'los', 'angeles', 'police', 'department', 'hussle', 'grammys', 'freeda', 'foreman', 'george', 'foreman', 'march', 'harris', 'county', 'sheriff', 'office', 'tebben', 'lewis', 'harris', 'county', 'institute', 'forensic', 'actor', 'luke', 'perry', 'beverly', 'hill', 'march', 'cnn', 'golden', 'katherine', 'helmond', 'bos', 'soap', 'february', 'apa', 'cnn', 'peter', 'tork', 'monkees', 'february', 'acclaimed', 'british', 'academy', 'award', 'albert', 'finney', 'february', 'john', 'dingell', 'congress', 'capitol', 'hill', 'february', 'frank', 'robinson', 'major', 'league', 'baseball', 'february', 'hall', 'famer', 'world', 'series', 'baltimore', 'oriole', 'james', 'ingram', 'r', 'b', 'heart', 'january', 'ingram', 'fatima', 'ali', 'bravo', 'chef', 'january', 'kevin', 'barnett', 'rel', 'january', 'twentieth', 'century', 'fox', 'television', 'fox', 'entertainment', 'masazo', 'nonaka', 'april', 'january', 'nhk', 'tony', 'mendez', 'cia', 'ben', 'affleck', 'argo', 'january', 'christy', 'fletcher', 'argo', 'iran', 'academy', 'award', 'john', 'coughlin', 'january', 'center', 'safesport', 'figure', 'skating', 'olympic', 'committee', 'broadway', 'carol', 'channing', 'january', 'channing', 'tony', 'dolly', 'levi', 'broadway', 'hello', 'dolly', 'alfred', 'newman', 'navajo', 'code', 'talker', 'january', 'navajo', 'world', 'war', 'ii', 'bob', 'einstein', 'smother', 'comedy', 'hour', 'larry', 'david', 'curb', 'enthusiasm', 'january', 'einstein', 'super', 'dave', 'osborne'], ['plenty', 'vermont', 'remote', 'worker', 'grant', 'program', 'january', 'joan', 'goldstein', 'vermont', 'department', 'economic', 'development', 'beth', 'dow', 'vermont', 'denver', 'east', 'dow', 'dow', 'dow', 'bennington', 'vermont', 'denver', 'bennington', 'vermont', 'vermont', 'goldstein', 'goldstein', 'goldstein', 'teddy', 'martin', 'new', 'york', 'city', 'new', 'york', 'new', 'orleans', 'martin', 'teddy', 'vermont', 'brattleboro', 'january', 'goldstein'], ['league', 'adl', 'united', 'jew', 'poway', 'california', 'jonathan', 'greenblatt', 'adl', 'ceo', 'san', 'diego', 'county', 'jew', 'october', 'tree', 'life', 'pittsburgh', 'jewish', 'adl', 'greenblatt', 'adl', 'greenblatt', 'tuesday', 'baraboo', 'wisconsin', 'milwaukee', 'hitler', 'jew', 'california', 'new', 'york', 'new', 'jersey', 'massachusetts', 'adl', 'congress', 'adl', 'george', 'selim', 'adl'], ['nye', 'tonight', 'john', 'oliver', 'sunday', 'new', 'deal', 'alexandria', 'ed', 'markey', 'nye', 'new', 'deal', 'oliver', 'green', 'new', 'deal', 'united', 'green', 'new', 'deal', 'cheesecake', 'factory', 'nye', 'suv', 'john', 'mentos', 'diet', 'coke'], ['national', 'law', 'enforcement', 'officer', 'memorial', 'fund', 'nleomf', 'steve', 'groeninger', 'cnn', 'cnn', 'savannah', 'georgia', 'police', 'officer', 'kelvin', 'ansari', 'sgt', 'ansari', 'army', 'biloxi', 'mississippi', 'police', 'officer', 'robert', 'mckeithen', 'officer', 'mckeithen', 'biloxi', 'police', 'department', 'mooresville', 'north', 'carolina', 'police', 'officer', 'jordan', 'harris', 'sheldon', 'sheldon', 'april', 'cowlitz', 'county', 'washington', 'deputy', 'sheriff', 'justin', 'derosier', 'deputy', 'derosier', 'vancouver', 'washington', 'derosier', 'cowlitz', 'county', 'sheriff', 'office', 'march', 'el', 'paso', 'county', 'texas', 'deputy', 'sheriff', 'peter', 'herrera', 'deputy', 'herrera', 'cnn', 'kvia', 'march', 'kittitas', 'county', 'washington', 'deputy', 'sheriff', 'ryan', 'thompson', 'deputy', 'thompson', 'thompson', 'march', 'mchenry', 'county', 'illinois', 'deputy', 'sheriff', 'jacob', 'keltner', 'deputy', 'sheriff', 'kelter', 'marshal', 'service', 'kelter', 'nleomf', 'march', 'midland', 'texas', 'police', 'officer', 'nathan', 'heidelberg', 'officer', 'heidelberg', 'nleomf', 'february', 'sullivan', 'county', 'tennessee', 'sheriff', 'sgt', 'steve', 'hinkle', 'sgt', 'hinkle', 'hinkle', 'february', 'puerto', 'rico', 'police', 'agent', 'alfred', 'agent', 'puerto', 'rico', 'february', 'nypd', 'detective', 'brian', 'simonsen', 'detective', 'simonsen', 'queen', 'new', 'york', 'february', 'milwaukee', 'police', 'officer', 'matthew', 'rittner', 'officer', 'rittner', 'milwaukee', 'february', 'virginia', 'state', 'trooper', 'lucas', 'dowell', 'trooper', 'dowell', 'farmville', 'virginia', 'february', 'clermont', 'county', 'ohio', 'det', 'brewer', 'det', 'brewer', 'cincinnati', 'january', 'mobile', 'alabama', 'police', 'officer', 'sean', 'tuder', 'officer', 'tuder', 'january', 'birmingham', 'alabama', 'police', 'sgt', 'wytasha', 'carter', 'sgt', 'carter', 'january', 'davis', 'california', 'police', 'officer', 'natalie', 'corona', 'officer', 'corona', 'davis', 'january', 'provo', 'utah', 'police', 'officer', 'joseph', 'shinners', 'officer', 'shinners', 'bed', 'bath', 'salt', 'lake', 'tribune'], ['disney', 'nyse', 'management', 'bob', 'iger', 'disney', 'disney', 'svod', 'svod', 'disney', 'disney', 'disney', 'disney', 'disney', 'disney', 'disney', 'disney', 'disney', 'disney', 'dvd', 'disney', 'dvd', 'dvd', 'disney', 'disney', 'dvd', 'dvd', 'disney', 'disney', 'disney', 'dvd', 'disney'], ['netflix', 'nasdaq', 'nflx', 'disney', 'nyse', 't', 'nyse', 'warnermedia', 'comcast', 'nasdaq', 'cmcsa', 'nbcuniversal', 'pwc', 'netflix', 't', 'comcast', 'office', 'netflix', 'netflixthough', 'netflix', 'netflix', 'netflix', 'netflix', 'netflix', 'disney', 't', 'comcast', 'netflix', 'netflix', 'netflix', 'warnermedia', 'netflix', 'netflix', 'netflix', 'netflix', 't', 'hbo', 'hbo', 'netflix'], ['dow', 'jones', 'industrial', 'average', 'djindices', 's', 'p', 'snpindex', 'monday', 'china', 'president', 'trump', 'walt', 'disney', 'nyse', 'comcast', 'nasdaq', 'cmcsa', 'comcast', 'hulu', 'ralph', 'lauren', 'nyse', 'getty', 'comcast', 'hulu', 'comcast', 'disney', 'class', 'comcast', 'comcast', 'hulu', 'disney', 'january', 'hulu', 'hulu', 'comcast', 'hulu', 'comcast', 'hulu', 'nbcuniversal', 'hulu', 'lauren', 'europe', 'great', 'ralph', 'lauren'], ['fox', 'disney', 'nyse', 'dis', 'hulu', 't', 'nyse', 'comcast', 'nasdaq', 'cmcsa', 't', 'hulu', 'tuesday', 'disney', 'comcast', 'disney', 'hulu', 'disney', 'comcast', 'disney', 'hulu', 'january', 'comcast', 'hulu', 'disney', 'hulu', 'comcast', 'hulu', 't', 'disney', 'comcast', 'nbcuniversal', 'comcast', 'hulu', 'comcast', 'disney', 'comcast', 'comcast', 'nbcuniversal', 'hulu', 'nbcuniversal', 'hulu', 'comcast', 'hulu', 'live', 'nbc', 'hulu', 'disney', 'ceo', 'bob', 'iger', 'hulu', 'walt', 'disney', 'company', 'disney', 'espn', 'disney', 'hulu', 'disney', 'hulu', 'disney', 'disney'], ['disney', 'nyse', 'hulu', 'hulu', 't', 'comcast', 'nasdaq', 'cmcsa', 't', 'comcast', 'hulu', 'disney', 'ceo', 'bob', 'iger', 'house', 'mouse', 'hulu', 'comcast', 'disney', 'hulu', 'disney', 'comcast', 'hulu', 'disney', 'hulu', 't', 'hulu', 'disney', 'hulu', 'disney', 'comcast', 'disney', 't', 'hulu', 'hulu', 'comcast', 'comcast', 'comcast', 'hulu', 'ott', 'disney', 'disney', 'comcast', 'disney', 'comcast', 'nbcuniversal', 'hulu', 'century', 'fox', 'fox', 'corporation', 'disney', 'disney', 'comcast', 'hulu', 'disney', 'hulu', 'ott'], ['disney', 'nyse', 'march', 'black', 'panther', 'captain', 'marvel', 'disney', 'century', 'fox', 'disney', 'disney', 'avatar', 'disney', 'skywalker', 'star', 'war', 'skywalker', 'aladdin', 'toy', 'story', 'lion', 'king', 'frozen', 'fox', 'dark', 'phoenix', 'june', 'disney', 'disney', 'fox', 'disney', 'avatar', 'fox', 'star', 'war', 'disney', 'box', 'disney', 'fox', 'disney', 'disney', 'disney', 'hulu', 'disney', 'disney', 'disney', 'disney'], ['los', 'angeles', 'reuters', 'skywalker', 'december', 'walt', 'disney', 'star', 'war', 'hbo', 'game', 'throne', 'tuesday', 'disney', 'star', 'war', 'december', 'star', 'war', 'star', 'war', 'skywalker', 'december', 'tuesday', 'chief', 'executive', 'bob', 'iger', 'david', 'benioff', 'wei', 'game', 'disney', 'skywalker', 'star', 'war', 'star', 'war', 'star', 'war', 'jedi', 'rian', 'johnson'], ['walt', 'disney', 'hulu', 'comcast', 'corp', 'netflix', 'hulu', 'comcast', 'disney', 'january', 'comcast', 'hulu', 't', 'comcast', 'comcast', 'hulu', 'disney', 'hulu', 'netflix', 'apple', 'disney', 'hulu', 'trip', 'miller', 'gullane', 'capital', 'partner', 'disney', 'disney', 'fox', 'comcast', 'hulu', 'disney', 'tuesday', 'disney', 'chief', 'executive', 'bob', 'iger', 'india', 'comcast', 'nbcuniversal', 'nbcuniversal', 'hulu', 'comcast', 'nbcuniversal', 'nbcuniversal', 'hulu', 'nbcu', 'hulu', 'disney', 'new', 'york', 'stock', 'exchange', 'comcast', 'nasdaq', 'hulu', 't', 'hulu', 'hulu', 'netflix', 'prime', 'video'], ['walt', 'disney', 'hulu', 'comcast', 'corp', 'netflix', 'hulu', 'comcast', 'disney', 'january', 'comcast', 'hulu', 't', 'comcast', 'comcast', 'hulu', 'disney', 'hulu', 'netflix', 'apple', 'disney', 'hulu', 'trip', 'miller', 'gullane', 'capital', 'partner', 'disney', 'disney', 'fox', 'comcast', 'hulu', 'disney', 'tuesday', 'disney', 'chief', 'executive', 'bob', 'iger', 'india', 'comcast', 'nbcuniversal', 'nbcuniversal', 'hulu', 'comcast', 'nbcuniversal', 'nbcuniversal', 'hulu', 'nbcu', 'hulu', 'disney', 'new', 'york', 'stock', 'exchange', 'comcast', 'nasdaq', 'hulu', 't', 'hulu', 'hulu', 'netflix', 'prime', 'video'], ['caesar', 'entertainment', 'corp', 'tuesday', 'walt', 'disney', 'espn', 'linq', 'hotel', 'casino', 'la', 'vega', 'espn', 'daily', 'wager', 'caesar', 'espn', 'tuesday', 'supreme', 'court', 'nevada'], ['walt', 'disney', 'comcast', 'corp', 'hulu', 'disney', 'comcast', 'january', 'comcast'], ['caesar', 'entertainment', 'czr', 'disney', 'nyse', 'espn', 'linq', 'hotel', 'casino', 'la', 'vega', 'vega', 'espn', 'espn', 'espn', 'espn', 'caesar', 'entertainment', 'espn', 'business', 'development', 'mike'], ['comcast', 'cmcsa', 'hulu', 'nbcuniversal', 'alex', 'sherman', 'comcast', 'disney', 'plus', 'dis', 'comcast', 'comcast', 'hulu', 'hulu'], ['comcast', 'nasdaq', 'cmcsa', 'hulu', 'cnbc', 'disney', 'nyse', 'fox', 'fox', 'foxa', 'warnermedia', 'nyse', 'hulu', 'comcast', 'hulu', 'comcast', 'hulu', 'nbc', 'hulu', 'live', 'nbc'], ['shadow', 'war', 'inside', 'russia', 'china', 'defeat', 'america', 'jim', 'sciutto', 'copyright', 'harper', 'collins', 'stephen', 'su', 'bob', 'anderson', 'fbi', 'stephen', 'su', 'su', 'bin', 'china', 'united', 'canada', 'su', 'anderson', 'su', 'lockheed', 'martin', 'boeing', 'globemaster', 'pentagon', 'su', 'anderson', 'su', 'boeing', 'america', 'su', 'america', 'china', 'office', 'trade', 'representative', 'united', 'china', 'ip', 'ustr', 'china', 'china', 'china', 'china', 'america', 'beijing', 'united', 'anderson', 'stephen', 'su', 'anderson', 'united', 'china', 'anderson', 'chinese', 'mit', 'stanford', 'anderson', 'united', 'anderson', 'su', 'su', 'canada', 'china', 'united', 'justice', 'department', 'china', 'february', 'su', 'united', 'california', 'su', 'fbi', 'su', 'fbi', 'overwhelmed', 'anderson', 'fbi', 'su', 'su', 'china', 'china', 'bob', 'anderson', 'su', 'america', 'china', 'united', 'united', 'anderson', 'china', 'anderson'], ['international', 'paper', 'company', 'ip', 'report', 'zacks', 'consensus', 'estimate', 'enterprise', 'value', 'ebitda', 'international', 'paper', 'surprise', 'international', 'paper', 'zacks', 'consensus', 'estimate', 'ebitda', 'industrial', 'packaging', 'rank', 'score', 'combination', 'international', 'paper', 'zacks', 'rank', 'hold', 'vgm', 'score', 'v', 'value', 'g', 'growth', 'm', 'momentum', 'vgm', 'score', 'b', 'zacks', 'rank', 'strong', 'buy', 'buy', 'equity', 'roe', 'international', 'paper', 'roe', 'roe', 'roe', 'paper', 'company', 'price', 'consensusinternational', 'paper', 'company', 'international', 'paper', 'company', 'quotestocks', 'considera', 'basic', 'israel', 'chemical', 'icl', 'report', 'arconic', 'arnc', 'report', 'arch', 'coal', 'arch', 'report', 'zacks', 'rank', 'zacks', 'rank', 'chemical', 'coal', 'zacks', 's', 'p', 's', 'p'], ['international', 'paper', 'nyse', 'june', 'ip', 'dividend', 'scorecard', 'yield', 'chart', 'dividend', 'growth'], ['monday', 'financial', 'select', 'sector', 'spdr', 'etf', 'nysearca', 'spdr', 's', 'p', 'regional', 'banking', 'etf', 'nysearca', 'kre', 'bank', 'america', 'bac', 'citigroup', 'c', 'jpmorgan', 'jpm', 'goldman', 'sachs', 'g', 'well', 'fargo', 'wfc', 'morgan', 'stanley', 'm', 'comerica', 'cma', 'keycorp', 'key', 'cit', 'group', 'cit', 'financial', 'rf', 'pnc', 'financial', 'pnc', 'fifth', 'fitb', 'huntington', 'bancshares', 'hban', 'financial', 'syf', 'alliance', 'data', 'ad', 'capital', 'cof', 'discover', 'financial', 'dfs', 'american', 'express', 'axp', 'brighthouse', 'financial', 'bhf', 'financial', 'pru', 'principal', 'financial', 'pfg', 'metlife', 'met'], ['eli', 'lilly', 'company', 'lly', 'report', 'fda', 'cyramza', 'hcc', 'fda', 'cyramza', 'hcc', 'reach', 'cyramza', 'hcc', 'amgen', 'amgn', 'report', 'bayry', 'report', 'nexavar', 'afp', 'reach', 'cyramza', 'lilly', 'hcc', 'afp', 'hcc', 'afp', 'cyramza', 'nsclc', 'hcc', 'fda', 'cyramza', 'cyramza', 'iii', 'relay', 'egfr', 'march', 'lilly', 'pfs', 'cyramza', 'nsclc', 'egfr', 'asco', 'june', 'lilly', 'zacks', 'rank', 'hold', 'merck', 'mrk', 'report', 'zacks', 'rank', 'buy', 'zacks', 'rank', 'strong', 'buy', 'merck', 'zackswould', 's', 'p', 's', 'p'], ['kratos', 'defense', 'security', 'solution', 'nasdaq', 'ktos', 'uavs', 'kratos', 'kratos', 'kratos', 'march', 'valkyrie', 'valkyrie', 'kratos', 'valkyrie', 'kratos', 'defense', 'ceo', 'eric', 'demarco', 'kratos', 'washington', 'capitol', 'hill', 'valkyrie', 'kratos', 'demarco', 'air', 'force', 'air', 'force', 'demarco', 'house', 'armed', 'committee', 'valkyrie', 'valkyrie', 'kratos', 'gremlin', 'gremlin', 'defense', 'advanced', 'research', 'agency', 'march', 'valkyrie', 'boeing', 'nyse', 'boeing', 'airpower', 'teaming', 'bat', 'valkyrie', 'pentagon', 'boeing', 'bat', 'boeing', 'bat', 'pentagon', 'kratos', 'valkyrie', 'demarco', 'boeing', 'kratos', 'uavs', 'uavs', 'kratos', 'uav', 'kratos', 'uavs', 'kratos', 'kratos', 'gremlin', 'kratos', 'space', 'force', 'elon', 'musk', 'spacex'], ['monday', 'china', 'dow', 'jones', 'industrial', 'average', 'djindices', 's', 'p', 'snpindex', 'boeing', 'nyse', 'ba', 'max', 'teva', 'pharmaceutical', 'industry', 'nyse', 'getty', 'boeing', 'china', 'max', 'wall', 'street', 'journal', 'boeing', 'federal', 'aviation', 'administration', 'faa', 'boeing', 'china', 'airbus', 'boeing', 'teva', 'pharmaceutical', 'teva', 'novartis', 'sandoz', 'mylan', 'pfizer', 'connecticut', 'attorney', 'general', 'william', 'tong', 'copaxone'], ['boeing', 'max', 'april', 'mcas', 'march', 'max', 'boeing', 'max', 'max', 'march', 'boeing', 'boeing', 'capital', 'max', 'april', 'boeing', 'india', 'jet', 'airway', 'airbus', 'se', 'airbus', 'april', 'airbus', 'boeing', 'boeing', 'boeing', 'united', 'china'], ['boeing', 'max', 'april', 'mcas', 'march', 'max', 'boeing', 'max', 'max', 'march', 'boeing', 'boeing', 'capital', 'max', 'april', 'boeing', 'india', 'jet', 'airway', 'airbus', 'se', 'airbus', 'april', 'airbus', 'boeing', 'boeing', 'boeing', 'united', 'china'], ['faa', 'boeing', 'ba', 'max', 'mcas', 'wsj', 'boeing', 'boeing', 'boeing'], ['general', 'mill', 'gi', 'report', 's', 'p', 'dow', 'nasdaq', 'cheerio', 'yoplait', 'consumer', 'staple', 's', 'p', 'gi', 'eps', 'zacks', 'consensus', 'gi', 'zacks', 'rank', 'zacks', 'rank', 'strong', 'buy', 'strong', 'sell', 'eps', 'gi', 'zacks', 'rank', 'buy', 'gi', 'forward', 'forward', 'gi', 'peg', 'peg', 'food', 'miscellaneous', 'peg', 'food', 'consumer', 'staple', 'zacks', 'industry', 'rank', 'industry', 'rank', 'zacks', 'rank', 'zacks', 'com'], ['momentum', 'zacks', 'momentum', 'style', 'score', 'zacks', 'style', 'score', 'general', 'mill', 'gi', 'report', 'momentum', 'style', 'score', 'b', 'momentum', 'style', 'style', 'score', 'zacks', 'rank', 'general', 'mill', 'zacks', 'rank', 'buy', 'zacks', 'rank', 'strong', 'buy', 'buy', 'b', 'zacks', 'set', 'market', 'gi', 'momentum', 'style', 'cheerio', 'yoplait', 'gi', 'zacks', 'food', 'general', 'mill', 's', 'p', 'gi', 'gi', 'outlookthe', 'zacks', 'momentum', 'style', 'score', 'zacks', 'rank', 'gi', 'gi', 'buy', 'momentum', 'score', 'b', 'general'], ['immediate', 'releasechicago', 'il', 'analyst', 'blog', 'zacks', 'equity', 'research', 'water', 'company', 'msex', 'report', 'consolidated', 'communication', 'holding', 'cnsl', 'report', 'gas', 'ogs', 'report', 'chef', 'warehouse', 'chef', 'report', 'general', 'mill', 'gi', 'report', 'monday', 'analyst', 'stock', 'wall', 'street', 'bloodbaththe', 'united', 'china', 'president', 'trump', 'chinese', 'beijing', 'brace', 'china', 'retaliation', 'trump', 'china', 'washington', 'trade', 'treasury', 'secretary', 'steven', 'mnuchin', 'china', 'commerce', 'ministry', 'trump', 'america', 'beijing', 'trump', 'beijing', 'people', 'daily', 'communist', 'party', 'china', 'china', 'trade', 'war', 'international', 'monetary', 'fund', 'united', 'march', 'china', 'awaystocks', 'november', 'october', 'november', 'pullback', 'food', 'solid', 'choiceswe', 'zacks', 'rank', 'strong', 'buy', 'buy', 'water', 'companyowns', 'zacks', 'rank', 'zacks', 'consensus', 'estimate', 'communication', 'united', 'zacks', 'rank', 'zacks', 'consensus', 'estimate', 'gas', 'united', 'zacks', 'rank', 'zacks', 'consensus', 'estimate', 'zacks', 'rank', 'chef', 'warehouse', 'united', 'canada', 'zacks', 'rank', 'zacks', 'consensus', 'estimate', 'mill', 'zacks', 'rank', 'zacks', 'consensus', 'estimate', 'latest', 'stock', 'today', 'medium', 'contactzacks', 'investment', 'inherent', 'zacks', 'investment', 'research', 'zacks', 'rank', 's', 'p', 'visit'], ['qualcomm', 'incorporated', 'qcom', 'report', 'hmd', 'global', 'oy', 'nokia', 'hmd', 'global', 'qualcomm', 'hmd', 'global', 'hmd', 'global', 'qualcomm', 'qualcomm', 'hmd', 'global', 'android', 'qualcomm', 'hmd', 'global', 'qualcomm', 'takequalcomm', 'zacks', 'rank', 'hold', 'qualcomm', 'viasat', 'vsat', 'report', 'harris', 'corporation', 'hr', 'report', 'motorola', 'solution', 'msi', 'report', 'zacks', 'rank', 'buy', 'zacks', 'rank', 'strong', 'buy', 'zackswould', 's', 'p', 's', 'p'], ['depot', 'ken', 'langone', 'chinasquawk', 'boxpresident', 'donald', 'trump', 'china', 'republican', 'ken', 'langone', 'cnbc', 'thursday', 'langone', 'home', 'trade', 'representative', 'robert', 'lighthizer', 'china', 'langone', 'squawk', 'box', 'trump', 'china', 'wednesday', 'china', 'april', 'beijing', 'united', 'langone', 'god', 'trump', 'vice', 'president', 'joe', 'biden', 'white', 'house', 'america', 'langone', 'invemed', 'trump', 'chris', 'christie', 'john', 'kasich'], ['add', 'solar', 'roof', 'tesla', 'panasonic', 'tesla', 'new', 'york', 'roof', 'tesla', 'panasonic', 'corp', 'buffalo', 'panasonic', 'custom', 'tesla', 'panasonic', 'solar', 'roof', 'tesla', 'chief', 'executive', 'elon', 'musk', 'roof', 'tesla', 'tesla', 'roof', 'united', 'tesla', 'panasonic', 'solar', 'roof', 'solar', 'roof', 'tesla', 'musk', 'tesla', 'musk', 'solarcity', 'lyndon', 'rive', 'alberto', 'canal', 'tesla', 'panasonic', 'buffalo', 'department', 'commerce', 'october', 'trump', 'administration', 'buffalo', 'tesla', 'california', 'reuters', 'august', 'tesla', 'solarcity', 'tesla', 'solarcity', 'home', 'depot', 'tesla', 'panasonic', 'tesla', 'musk', 'panasonic', 'ev', 'tesla', 'shanghai', 'october', 'buffalo', 'panasonic', 'ftz', 'canal', 'buffalo', 'united', 'buffalo', 'buffalo', 'panasonic', 'riverbend', 'solarcity', 'solarcity', 'new', 'york', 'panasonic', 'buffalo', 'riverbend', 'pamm', 'lent', 'empire', 'state', 'development', 'tesla', 'panasonic', 'riverbend', 'tesla', 'buffalo', 'panasonic', 'tesla', 'nichola', 'groom', 'additional', 'makiko', 'yamazaki', 'tokyo', 'editing', 'richard', 'valdmanis', 'brian', 'thevenot'], ['streetwalmart', 'amazon', 'april', 'amazon', 'prime', 'tuesday', 'phoenix', 'la', 'vega', 'southern', 'california', 'april', 'prime', 'amazon', 'zip', 'january', 'walmart', 'amazon', 'new', 'york', 'marc', 'lore', 'walmart', 'walmart', 'amazon', 'prime', 'lore', 'amazon', 'prime', 'walmart', 'target', 'walmart', 'tuesday', 'amazon', 'amazon', 'john', 'bonno', 'alixpartners', 'amazon', 'walmart', 'target', 'amazon', 'walmart', 'laura', 'kennedy', 'kantar', 'amazon', 'ceo', 'jeff', 'bezos', 'doug', 'mcmillon', 'ceo', 'image', 'cnbca', 'ubs', 'walmart', 'amazon', 'ubs', 'walmart', 'walmart', 'amazon', 'rbc', 'capital', 'walmart', 'star', 'war', 'walmart', 'walmart', 'lore', 'walmart', 'sucharita', 'kodali', 'forrester', 'research', 'amazon', 'amazon', 'zappos', 'walmart', 'amazon', 'kodali', 'coresight', 'research', 'walmart', 'target', 'best', 'buy', 'home', 'depot', 'coresight', 'kodali', 'monday', 'amazon', 'amazon', 'amazon', 'food', 'walmart', 'target', 'cnbc', 'michael', 'bloom', 'amazon', 'prime', 'walmart', 'target', 'consumer', 'retail', 'digital', 'original', 'video'], ['uniqlo', 'san', 'francisco', 'california', 'january', 'paul', 'morris', 'bloomberg', 'getty', 'imagestariffs', 'home', 'depot', 'lowe', 'best', 'buy', 'dollar', 'tree', 'tractor', 'supply', 'depot', 'friday', 'best', 'buy', 'lowe', 'tractor', 'supply', 's', 'p', 'retail', 'etf', 'xrt', 'china', 'macy', 'ceo', 'jeff', 'gennette', 'president', 'donald', 'trump', 'china', 'goldman', 'sachs', 'christopher', 'prykull', 'pryull', 'trump', 'friday', 'nike', 'china', 'census', 'trade', 'representative', 'robert', 'lighthizer', 'trump', 'walmart', 'home', 'depot', 'lowe', 'trade', 'partnership', 'jonathan', 'gold', 'national', 'retail', 'federation', 'home', 'depot', 'lowe', 'best', 'buy', 'walmart', 'cnbc', 'michael', 'bloom'], ['consumer', 'consumer', 'discretionary', 'select', 'sector', 'spdr', 'xly', 'vanguard', 'consumer', 'discretionary', 'index', 'etf', 'vcr', 'january', 'april', 'cnbc', 'kensho', 'wall', 'street', 'discretionary', 'january', 'april', 'vanguard', 'etf', 'blackrock', 'consumer', 'service', 'etf', 'iyc', 'etf', 'xly', 'vcr', 'street', 'amazon', 'sachs', 'amazon', 'home', 'depot', 'depot', 'mcdonald', 'etf'], [], ['hashmat', 'zubair', 'amazon', 'baltimore', 'maryland', 'april', 'rclodagh', 'kilcoyne', 'reutersas', 'china', 'goldman', 'sachs', 'amazon', 'apple', 'goldman', 'sachs', 'tuesday', 'david', 'kostin', 'goldman', 'president', 'donald', 'trump', 'china', 'beijing', 'dow', 'jones', 'industrial', 'average', 's', 'p', 'nasdaq', 'trade', 'representative', 'robert', 'lighthizer', 'goldman', 's', 'p', 'goldman', 'amazon', 'google', 'microsoft', 'apple', 'china', 'johnson', 'johnson', 'exxon', 'mobil', 'facebook', 'disney', 'home', 'depot', 't', 'mcdonald', 'goldman', 'netflix', 'comcast', 'well', 'fargo', 'kostin', 'goldman', 'goldman', 'china', 'goldman', 'procter', 'gamble', 'intel', 'chevron', 'boeing', 'kostin', 'broadcom', 'honeywell', 'nvdia', 'goldman', 'brown', 'set', 'nbcuniversal', 'cnbc'], ['reportcheck', 'adp', 'clorox', 'cv', 'health', 'cv', 'cv', 'lauder', 'la', 'mer', 'gps', 'garmin', 'worldwide', 'street', 'hilton', 'humana', 'medicare', 'coors', 'wall', 'street', 'brand', 'yum', 'kfc', 'pizza', 'hut', 'apple', 'wall', 'street', 'apple', 'amgen', 'amgen', 'neulasta', 'international', 'mondelez', 'oreo', 'micro', 'amd', 'gross', 'resort', 'wynn', 'massachusetts', 'ceo', 'matt', 'maddox', 'ceo', 'steve', 'wynn', 'maddox', 'boeing', 'tuesday', 'max', 'depot', 'chief', 'financial', 'officer', 'carol', 'tome', 'august', 'cfo', 'executive', 'vice', 'president', 'richard', 'mcphail', 'vega', 'sand', 'la', 'vega', 'sand', 'macau', 'april', 'june', 'petroleum', 'anadarko', 'chevron', 'occidental', 'petroleum', 'berkshire', 'hathaway'], ['home', 'depot', 'louisville', 'kentucky', 'monday', 'sharrett', 'bloomberg', 'getty', 'imageshome', 'depot', 'tuesday', 'chief', 'financial', 'officer', 'carol', 'tomé', 'richard', 'mcphail', 'carol', 'cfo', 'carol', 'craig', 'menear', 'ceo', 'home', 'depot', 'home', 'depot', 'mcphail', 'richard', 'menear'], ['april', 'home', 'depot', 'tuesday', 'carol', 'tomé', 'richard', 'mcphail', 'finance', 'control', 'administration', 'vibhuti', 'sharma', 'bengaluru', 'editing', 'maju', 'samuel'], ['prime', 'lineas', 'amazon', 'walmart', 'target', 'amazon', 'thursday', 'amazon', 'prime', 'friday', 'walmart', 'amazon', 'prime', 'amazon', 'walmart', 'target', 'national', 'retail', 'federation', 'amazon', 'prime', 'morgan', 'stanley', 'brian', 'nowak', 'amazon', 'amazon', 'prime', 'kolbaskowo', 'poland', 'friday', 'sadowski', 'bloomberg', 'getty', 'imageswhile', 'walmart', 'target', 'wall', 'street', 'walmart', 'target', 'january', 'amazon', 'march', 'target', 'target', 'shipt', 'new', 'york', 'target', 'target', 'shipt', 'drive', 'order', 'pickup', 'walmart', 'twitter', 'friday', 'stay', 'walmart', 'target', 'stacey', 'widlitz', 'sw', 'retail', 'advisor', 'god', 'walmart', 'target', 'amazon', 'best', 'buy', 'kohl', 'home', 'depot', 'amazon', 'november', 'april', 'coresight', 'research', 'coresight', 'walmart', 'target', 'best', 'buy', 'home', 'courtney', 'reagan', 'target', 'consumer', 'retail', 'digital', 'original', 'video'], ['april', 'tesla', 'wednesday', 'home', 'depot', 'tesla', 'mw', 'tesla', 'solarcity', 'tesla', 'home', 'depot', 'solarcity', 'mw', 'february', 'tesla', 'tesla', 'nichola', 'groom', 'los', 'angeles', 'editing', 'rosalba'], ['money', 'jim', 'cramerdo', 'texas', 'instrument', 'cnbc', 'jim', 'cramer', 'tuesday', 'texas', 'instrument', 'mad', 'money', 'cramer', 'tuesday', 'texas', 'instrument', 'texas', 'instrument', 'cramer', 'new', 'york', 'texas', 'instrument', 'cfo', 'rafael', 'lizardi', 'cramer', 'texas', 'instrument', 'lam', 'research', 'apple', 'january', 'qualcomm', 'apple', 'wednesday', 'qualcomm', 'apple', 'apple', 'qualcomm', 'qualcomm', 'apple', 'disney', 'chipotle', 'home', 'depot', 'cramer', 'texas', 'instrument', 'wednesday', 'cramer', 'cramer', 'money', 'jim', 'apple', 'walt', 'disney', 'home', 'depot', 'lam', 'cramer', 'cramer', 'hit', 'cramer', 'twitter', 'facebook', 'mad', 'money'], ['add', 'sears', 'land', 'end', 'new', 'york', 'april', 'sears', 'corp', 'chairman', 'eddie', 'lampert', 'esl', 'investment', 'treasury', 'secretary', 'steven', 'mnuchin', 'thursday', 'sears', 'lampert', 'february', 'sears', 'diehard', 'kenmore', 'sears', 'lampert', 'sears', 'chapter', 'october', 'lampert', 'walmart', 'home', 'depot', 'esl', 'president', 'kunal', 'kamlani', 'bruce', 'berkowitz', 'fairholme', 'capital', 'management', 'sears', 'seritage', 'growth', 'property', 'sears', 'lampert', 'yale', 'university', 'sears', 'esl', 'lampert', 'goldman', 'esl', 'lampert', 'kalmani', 'esl', 'treasury', 'department', 'sears', 'sears', 'roebuck', 'kmart', 'holding', 'lampert', 'sears', 'hoffman', 'estate', 'lampert', 'sears', 'land', 'sears', 'hometown', 'lampert', 'tommy', 'hilfiger', 'leonard', 'green', 'land', 'esl', 'sears', 'seritage', 'sears', 'lampert', 'seritage', 'july', 'warren', 'buffett', 'berkshire', 'hathaway', 'berkshire', 'white', 'new', 'york', 'sears', 'land', 'seritage', 'sears', 'kmart', 'sears', 'corp', 'lampert', 'bankruptcy', 'court', 'southern', 'district', 'new', 'york', 'sears', 'corp', 'jonathan', 'stempel', 'new', 'york', 'steve', 'orlofsky', 'marguerita', 'choy', 'lisa', 'shumaker'], ['add', 'new', 'york', 'april', 'sears', 'corp', 'eddie', 'lampert', 'esl', 'investment', 'treasury', 'secretary', 'steven', 'mnuchin', 'thursday', 'sears', 'lampert', 'february', 'sears', 'diehard', 'kenmore', 'sears', 'lampert', 'sears', 'chapter', 'october', 'lampert', 'walmart', 'home', 'depot', 'esl', 'president', 'kunal', 'kamlani', 'bruce', 'berkowitz', 'fairholme', 'capital', 'management', 'sears', 'seritage', 'growth', 'property', 'sears', 'lampert', 'yale', 'university', 'sears', 'esl', 'lampert', 'goldman', 'esl', 'lampert', 'kalmani', 'esl', 'treasury', 'department', 'sears', 'sears', 'roebuck', 'kmart', 'holding', 'lampert', 'sears', 'hoffman', 'estate', 'lampert', 'sears', 'land', 'sears', 'hometown', 'lampert', 'seritage', 'sears', 'lampert', 'seritage', 'july', 'warren', 'buffett', 'berkshire', 'hathaway', 'berkshire', 'white', 'new', 'sears', 'kmart', 'sears', 'corp', 'lampert', 'bankruptcy', 'court', 'southern', 'district', 'new', 'york', 'sears', 'corp', 'jonathan', 'stempel', 'new', 'york', 'steve', 'orlofsky', 'marguerita', 'choy'], ['boeing', 'nike', 'money', 'jim', 'cramercnbc', 'jim', 'cramer', 'tuesday', 'dow', 'jones', 'industrial', 'average', 'nasdaq', 'composite', 's', 'p', 'mad', 'money', 'boeingboeing', 'max', 'boeing', 'wednesday', 'boeing', 'appleapple', 'cramer', 'january', 'united', 'qualcomm', 'apple', 'cramer', 'apple', 'tuesday', 'qualcomm', 'apple', 'north', 'america', 'china', 'nike', 'china', 'cramer', 'north', 'america', 'europe', 'europe', 'cramer', 'nike', 'caterpillarcramer', 'ubs', 'february', 'caterpillar', 'tuesday', 'ubs', 'caterpillar', 'cat', 'cramer', 'depothome', 'depot', 'cramer', 'tuesday', 'home', 'depot', 'cramer', 'belowfive', 'march', 'april', 'wall', 'street', 'nbc', 'news', 'cramer', 'youtube', 'facebook', 'facebook', 'facebook', 'cramer', 'boeing', 'nike', 'money', 'jim', 'facebook', 'apple', 'home', 'cramer', 'cramer', 'hit', 'cramer', 'twitter', 'facebook', 'mad', 'money'], ['amgn', 'csco', 'moneythe', 'fast', 'money', 'najarian', 'seymour', 'finerman', 'nathan', 'retail', 'etf', 'pete', 'aal', 'amd', 'amrn', 'amgn', 'ashr', 'baba', 'csco', 'cve', 'dvn', 'eca', 'fcx', 'fxi', 'gdx', 'huya', 'intc', 'jef', 'jnpr', 'knx', 'ktos', 'lpl', 'mmm', 'mrvl', 'nke', 'nlsn', 'oih', 'orcl', 'oxy', 'pags', 'pin', 'qep', 'qqq', 'tmus', 'twtr', 'ual', 'xlnx', 'xme', 'pete', 'aapl', 'abbv', 'bac', 'dis', 'fb', 'ful', 'gm', 'goo', 'hal', 'ibm', 'intc', 'kmi', 'ko', 'len', 'lulu', 'luv', 'lvs', 'mpc', 'mrk', 'msft', 'nflx', 'nke', 'pep', 'pfe', 'qcom', 'tgt', 'tpx', 'ual', 'ups', 'upwk', 'usb', 'xom', 'pete', 'ge', 'dan', 'nathan', 'wfc', 'long', 'csco', 'ea', 'june', 'sept', 'dis', 'june', 'sept', 'tim', 'seymour', 'amzn', 'aapl', 'acbff', 'acrgf', 'amza', 'acb', 'apc', 'aph', 'ba', 'baba', 'bac', 'bidu', 'bx', 'c', 'ccj', 'cgc', 'clf', 'cmg', 'cnttf', 'crlbf', 'cron', 'csco', 'cweb', 'curlf', 'dal', 'dis', 'dpz', 'dvye', 'dyme', 'eem', 'eufn', 'ewm', 'fb', 'fdx', 'fxi', 'ge', 'gild', 'gm', 'googl', 'gtbif', 'gtii', 'gwph', 'hal', 'hexo', 'hrvof', 'hvt', 'hyydf', 'intc', 'ithuf', 'jd', 'khrnf', 'kro', 'kshb', 'leaf', 'lnth', 'mat', 'mcd', 'mjne', 'mo', 'mo', 'mpel', 'mpx', 'mrmd', 'nke', 'ogi', 'orgmf', 'otc', 'pak', 'phm', 'pypl', 'rh', 'rl', 'sbux', 'sq', 'stz', 't', 'ter', 'tif', 'tgod', 'tnybf', 'trssf', 'trst', 'twtr', 'ua', 'ual', 'vale', 'viab', 'vod', 'x', 'xrt', 'yndx', 'tim', 'iwm', 'race', 'spy', 'tsla', 'tim', 'cgc', 'hexo', 'cron', 'aph', 'tim', 'green', 'organic', 'dutchman', 'heaven', 'kushco', 'dionymed', 'tikun', 'olam', 'cctv', 'canndescent', 'karen', 'finerman', 'antm', 'c', 'cbs', 'cpri', 'fb', 'fdx', 'fl', 'fnac', 'goog', 'googl', 'glng', 'gmlp', 'jpm', 'lyv', 'spy', 'spy', 'tbt', 'uri', 'wifi', 'iwm', 'karen', 'finerman', 'aal', 'bac', 'bot', 'bitcoin', 'bitcoin', 'cash', 'ethereum', 'c', 'cat', 'cbs', 'cpri', 'dal', 'dvye', 'dxj', 'eem', 'epi', 'eww', 'ewz', 'dvye', 'fb', 'fl', 'gm', 'gmlp', 'glng', 'goog', 'googl', 'jpm', 'low', 'lyv', 'kfl', 'ma', 'mtw', 'prcp', 'sedg', 'taco', 'tgt', 'wifi', 'wfm', 'karen', 'finerman', 'fb', 'karen', 'finerman', 'goog', 'googl', 'karen', 'finerman', 'spy', 'bitcoin', 'ethereum', 'trust'], ['huawei', 'huawei', 'huawei', 'asian', 'huawei', 'trump', 'huawei', 'entity', 'list', 'china', 'zte', 'corp', 'huawei', 'huawei', 'huawei', 'huawei', 'qualcomm', 'intel', 'corp', 'micron', 'technology', 'apple', 'china', 'huawei', 'zte', 'south', 'korea', 'samsung', 'beijing', 'asian', 'huawei', 'huawei', 'huawei', 'huawei', 'china', 'huawei', 'huawei', 'asia', 'korea', 'samsung', 'sk', 'hynix', 'china', 'luxshare', 'precision', 'industry', 'zte', 'chairman', 'eric', 'xu', 'reuters', 'china', 'zte', 'china', 'wan', 'gang', 'china', 'talksthe', 'huawei', 'huawei', 'huawei', 'united', 'doh', 'nh', 'investment', 'security', 'trump', 'china', 'wednesday', 'president', 'donald', 'trump', 'huawei', 'huawei', 'huawei', 'chinese', 'sijia', 'jiang', 'hong', 'kong', 'josh', 'horwitz', 'shanghai', 'park', 'heekyong', 'yang', 'seoul', 'michael', 'martina', 'cate', 'cadell', 'beijing', 'makiko', 'yamazaki', 'tokyo', 'writing', 'miyoung', 'kim', 'editing', 'himani', 'sarkar'], ['momo', 'production', 'digitalvision', 'getty', 'china', 'revolution', 'invest', 'nobel', 'laureate', 'daniel', 'kahneman', 'faang', 'facebook', 'apple', 'amazon', 'netflix', 'google', 'emily', 'chiang', 'arlington', 'emc', 'succession', 'warren', 'buffett', 'saujani', 'set', 'medical', 'research', 'benjamin', 'franklin', 'danqin', 'fang', 'evensky', 'financial', 'wealth', 'management', 'disclaimer', 'danqin', 'fang', 'money', 'lesson', 'know', 'age', 'grow', 'comcast', 'venture', 'acorn'], ['dow', 's', 'p', 'nasdaq', 'china', 'huawei', 'cnbc', 'wednesday', 'cnbc', 'dow', 'walmart', 'wmt', 'amat', 'baidu', 'bidu', 'nvidia', 'nvda', 'pinterest', 'pin', 'cnbc', 'walmart', 'cnbc', 'cisco', 'system', 'csco', 'dow', 'wednesday', 'hathaway', 'amazon', 'amzn', 'wednesday', 'warren', 'buffett', 'cnbc', 'cnbc', 'faang', 'et', 'april', 'philadelphia', 'fed', 'cnbc', 'news', 'todaypresident', 'donald', 'trump', 'huawei', 'cnbc', 'trump', 'huawei', 'cnbc', 'france', 'huawei', 'president', 'macron', 'cnbc', 'trump', 'usa', 'today', 'new', 'york', 'city', 'mayor', 'blasio', 'nbc', 'news', 'iowa', 'south', 'vice', 'president', 'joe', 'biden', 'new', 'york', 'new', 'york', 'gov', 'andrew', 'cuomo', 'jim', 'chanos', 'biden', 'axios', 'axios', 'attorney', 'general', 'william', 'barr', 'wall', 'street', 'journal', 'robert', 'mueller', 'congress', 'house', 'justice', 'department', 'trump', 'baron', 'conrad', 'black', 'wsj', 'washington', 'post', 'guatemalan', 'el', 'paso', 'trump', 'iran', 'persian', 'gulf', 'ny', 'time', 'eu', 'barclays', 'citigroup', 'c', 'morgan', 'jpm', 'mufg', 'royal', 'bank', 'scotland', 'pg', 'e', 'pcg', 'california', 'camp', 'california', 'wsj', 'north', 'carolina', 'juul', 'north', 'carolina', 'attorney', 'general', 'josh', 'stein', 'cnbc', 'federal', 'fda', 'ap', 'meat', 'bynd', 'wednesday', 'tim', 'hortons', 'meat', 'ipo', 'cnbc', 'spacex', 'ceo', 'elon', 'musk', 'starlink', 'mar', 'cnbc', 'jeff', 'koons', 'christie', 'wsj', 'stock', 'watchxpo', 'logistics', 'xpo', 'sne', 'march', 'sony', 'home', 'group', 'home', 'wednesday', 'reuters', 'kohl', 'ks', 'food', 'flo', 'wonder', 'bread', 'dd', 'home', 'kbh', 'rbc', 'capital', 'market', 'nflx', 'black', 'mirror', 'june', 'miley', 'cyrus', 'topher', 'grace', 'yahya', 'ii', 'cbs', 'big', 'bang', 'theory', 'usa', 'today'], ['huawei', 'huawei', 'huawei', 'asian', 'huawei', 'trump', 'huawei', 'entity', 'list', 'china', 'zte', 'corp', 'huawei', 'huawei', 'huawei', 'huawei', 'qualcomm', 'intel', 'corp', 'micron', 'technology', 'apple', 'china', 'huawei', 'zte', 'south', 'korea', 'samsung', 'beijing', 'asian', 'huawei', 'huawei', 'huawei', 'huawei', 'china', 'huawei', 'huawei', 'asia', 'korea', 'samsung', 'sk', 'hynix', 'china', 'luxshare', 'precision', 'industry', 'zte', 'chairman', 'eric', 'xu', 'reuters', 'china', 'zte', 'china', 'wan', 'gang', 'china', 'talksthe', 'huawei', 'huawei', 'huawei', 'united', 'doh', 'nh', 'investment', 'security', 'trump', 'china', 'wednesday', 'president', 'donald', 'trump', 'huawei', 'huawei', 'huawei', 'chinese', 'sijia', 'jiang', 'hong', 'kong', 'josh', 'horwitz', 'shanghai', 'park', 'heekyong', 'yang', 'seoul', 'michael', 'martina', 'cate', 'cadell', 'beijing', 'makiko', 'yamazaki', 'tokyo', 'writing', 'miyoung', 'kim', 'editing', 'himani', 'sarkar'], [], ['faangs', 'facebook', 'apple', 'netflix', 'alphabet', 'faang', 'tiger', 'global', 'management', 'chase', 'coleman', 'facebook', 'exchange', 'commission', 'netflix', 'soros', 'fund', 'management', 'netflix', 'rowe', 'price', 'facebook', 'facebook', 'ceo', 'mark', 'zuckerberg', 'viva', 'technologie', 'parc', 'exposition', 'porte', 'versailles', 'paris', 'getty', 'imagesfaang', 'netflix', 'walt', 'disney', 'faang', 'facebooks', 'paul', 'greene', 'rowe', 'price', 'communication', 'technology', 'fund', 'facebook', 'facebook', 'sec', 'buffett', 'berkshire', 'hathaway', 'amazon', 'march', 'jeff', 'cnbc', 'berkshire', 'amazon', 'wednesday', 'berkshire', 'cnbc', 'amazon', 'todd', 'comb', 'ted', 'weschler', 'amazon', 'march', 'wednesday'], ['tim', 'cook', 'apple', 'apple', 'worldwide', 'developer', 'conference', 'wwdc', 'san', 'jose', 'california', 'monday', 'june', 'david', 'paul', 'morris', 'bloomberg', 'getty', 'imagessteam', 'link', 'apple', 'apple', 'steam', 'link', 'apple', 'valve', 'steam', 'apple', 'app', 'store', 'link', 'apple', 'app', 'store', 'steam', 'app', 'store', 'apple', 'apple', 'steam', 'link', 'valve', 'appletv', 'apple', 'phil', 'schiller', 'valve', 'steam', 'link', 'android', 'march', 'apple', 'apple', 'arcade', 'apple', 'steam', 'link', 'steam', 'valve', 'steam', 'link', 'apple', 'alibaba', 'apple', 'alleysubscribe', 'youtube'], ['japan', 'display', 'mobara', 'chiba', 'prefecture', 'japan', 'monday', 'june', 'ota', 'bloomberg', 'getty', 'japan', 'display', 'apple', 'xr', 'lcd', 'display', 'march', 'ceo', 'yoshiyuki', 'tsukizaki', 'wednesday', 'display', 'valley', 'silicon', 'valley', 'use', 'privacy', 'policy', 'japan', 'display', 'cfo', 'takanobu', 'oshima', 'tpk', 'holding', 'harvest', 'group', 'japan', 'display', 'incj', 'japan', 'display', 'lcd', 'hitachi', 'toshiba', 'sony', 'display', 'march', 'lcd', 'japan', 'display'], ['add', 'china', 'cisco', 'cisco', 'wednesday', 'china', 'united', 'china', 'cisco', 'china', 'chief', 'financial', 'officer', 'kelly', 'kramer', 'dow', 'financial', 'officer', 'kelly', 'kramer', 'cisco', 'china', 'apple', 'reuters', 'cisco', 'ibes', 'refinitiv', 'cisco', 'elazar', 'advisor', 'chaim', 'siegel', 'april', 'cisco', 'akanksha', 'rana', 'bengaluru', 'stephen', 'nellis', 'san', 'francisco', 'editing', 'anil', 'richard', 'chang'], ['roku', 'ceomad', 'money', 'jim', 'cramerroku', 'cnbc', 'jim', 'cramer', 'netflix', 'disney', 'roku', 'ceo', 'anthony', 'wood', 'mad', 'money', 'wednesday', 'roku', 'wall', 'street', 'cramer', 'netflix', 'wood', 'roku', 'disney', 'comcast', 'nbcuniversal', 'apple', 'roku', 'comcast', 'disney', 'apple', 'wood', 'roku', 'roku', 'wood', 'cramer', 'roku', 'wednesday', 'wall', 'street', 'roku', 'roku', 'ceo', 'anthony', 'roku', 'ceo', 'money', 'jim', 'disney', 'apple', 'comcast', 'comcast', 'cnbc', 'cramer', 'cramer', 'hit', 'cramer', 'twitter', 'facebook', 'mad', 'money'], ['add', 'berkshire', 'hathaway', 'faangs', 'facebook', 'apple', 'netflix', 'alphabet', 'faang', 'tiger', 'global', 'management', 'llc', 'chase', 'coleman', 'facebook', 'exchange', 'commission', 'netflix', 'rowe', 'price', 'facebook', 'facebook', 'sec', 'buffett', 'berkshire', 'hathaway', 'amazon', 'march', 'jeff', 'cnbc', 'berkshire', 'amazon', 'wednesday', 'berkshire', 'cnbc', 'amazon', 'todd', 'comb', 'ted', 'weschler', 'amazon', 'march', 'wednesday', 'jennifer', 'ablan', 'editing', 'leslie', 'adler'], ['add', 'amazon', 'warren', 'buffett', 'charlie', 'munger', 'berkshire', 'warren', 'buffett', 'berkshire', 'hathaway', 'march', 'chief', 'executive', 'jeff', 'berkshire', 'march', 'buffett', 'todd', 'comb', 'ted', 'weschler', 'cnbc', 'amazon', 'comb', 'weschler', 'amazon', 'buffett', 'bezos', 'jpmorgan', 'chase', 'haven', 'jpmorgan', 'berkshire', 'vice', 'chairman', 'charlie', 'munger', 'amazon', 'munger', 'berkshire', 'bezos', 'comb', 'weschler', 'berkshire', 'berkshire', 'precision', 'castparts', 'apple', 'berkshire', 'buffett', 'comb', 'weschler', 'berkshire', 'bnsf', 'geico', 'buffett', 'standard', 'poor', 'berkshire', 'delta', 'air', 'pnc', 'financial', 'service', 'group', 'red', 'hat', 'phillips', 'southwest', 'well', 'fargo', 'jonathan', 'stempel', 'new', 'york', 'editing', 'susan', 'thomas', 'lisa', 'shumaker'], ['faangs', 'facebook', 'apple', 'netflix', 'alphabet', 'faang', 'tiger', 'global', 'management', 'llc', 'chase', 'coleman', 'facebook', 'exchange', 'commission', 'netflix', 'rowe', 'price', 'facebook', 'facebook', 'sec', 'jennifer', 'ablan', 'editing', 'leslie', 'adler'], ['billionaire', 'terry', 'gou', 'foxconn', 'technology', 'group', 'scott', 'walker', 'wisconsin', 'east', 'room', 'white', 'house', 'washington', 'wednesday', 'july', 'harrer', 'bloomberg', 'getty', 'image', 'foxconn', 'japan', 'display', 'apple', 'display', 'wednesday', 'japan', 'display', 'oled', 'lcd', 'china', 'japan', 'display', 'bernstein', 'toni', 'sacconaghi', 'february', 'idc', 'february', 'morgan', 'apple', 'apple', 'apple', 'apple', 'channel', 'hbo', 'showtime', 'apple', 'reportsubscribe', 'youtube'], ['bannon', 'cnbc', 'trump', 'china', 'linethere', 'president', 'donald', 'trump', 'china', 'trump', 'steve', 'bannon', 'cnbc', 'wednesday', 'china', 'house', 'trump', 'clinton', 'george', 'bush', 'barack', 'obama', 'china', 'trump', 'donald', 'trump', 'bannon', 'squawk', 'box', 'cnbc', 'steve', 'bannon', 'boxunder', 'trump', 'washington', 'china', 'chinese', 'american', 'huawei', 'chinese', 'china', 'united', 'bannon', 'china', 'huawei', 'wednesday', 'china', 'april', 'beijing', 'united', 'bannon', 'china', 'communist', 'party', 'trump', 'china', 'bannon', 'trump', 'monday', 'china', 'trade', 'representative', 'robert', 'lighthizer', 'china', 'vice', 'premier', 'liu', 'bannon', 'chinese', 'treasury', 'secretary', 'steven', 'mnuchin', 'lighthizer', 'bannon', 'lighthizer', 'lighthizer', 'president', 'trump', 'trump', 'china', 's', 'p', 'tuesday', 'monday', 's', 'p', 'tuesday', 'christmas', 'china', 'trump', 'wall', 'street', 'wednesday', 'senate', 'minority', 'leader', 'chuck', 'schumer', 'goldman', 'sachs', 'china', 'trump', 'trump', 'beijing', 'tuesday', 'goldman', 'ceo', 'lloyd', 'blankfein', 'bannon', 'cnbc', 'singapore', 'yen', 'nee', 'lee', 'reuters', 'sign', 'morning', 'squawk', 'cnbc', 'sign', 'use', 'privacy', 'policy', 'android', 'safari', 'proxima', 'nova', 'semi', 'bold', 'proxima', 'nova', 'bold'], ['remix', 'ceo', 'jim', 'cramerthe', 'ceo', 'china', 'united', 'cnbc', 'jim', 'cramer', 'president', 'donald', 'trump', 'ceo', 'trump', 'mad', 'money', 'china', 'trump', 'friday', 'trump', 'trump', 'ceo', 'people', 'republic', 'cramer', 'china', 'europe', 'china', 'cramer', 'bmw', 'mercedes', 'benz', 'germany', 'ford', 'general', 'china', 'cramer', 'cramer', 'tuesdayadam', 'jeffery', 'cnbccramer', 'dow', 'jones', 'industrial', 'average', 's', 'p', 'nasdaq', 'composite', 'boeing', 'caterpillar', 'max', 'deere', 'friday', 'cramer', 'china', 'cramer', 'narayen', 'ceo', 'adobemark', 'neuling', 'cnbcadobe', 'president', 'ceo', 'shantanu', 'narayen', 'tuesday', 'cramer', 'san', 'francisco', 'ceo', 'china', 'adobe', 'adobe', 'narayen', 'cramericajim', 'cramerscott', 'mlyn', 'cnbcjim', 'cramer', 'china', 'trade', 'desk', 'ceo', 'jeff', 'greensource', 'desk', 'ceo', 'jeff', 'alphabet', 'google', 'amazon', 'facebook', 'apple', 'baidu', 'alibaba', 'tencent', 'trade', 'desk', 'china', 'green', 'green', 'china', 'san', 'francisco', 'qualcomm', 'cramer', 'mad', 'money', 'brink', 'facebook', 'apple', 'cramer', 'cramer', 'hit', 'cramer', 'twitter', 'facebook', 'mad', 'money'], ['ceo', 'money', 'jim', 'crameradobe', 'president', 'ceo', 'shantanu', 'narayen', 'tuesday', 'jim', 'cramer', 'san', 'francisco', 'ceo', 'china', 'mad', 'money', 'adobe', 'adobe', 'narayen', 'tuesday', 'adobe', 'narayen', 'adobe', 'narayen', 'amazon', 'narayen', 'given', 'adobe', 'adobe', 'march', 'tuesday', 'cramer', 'adobe', 'ceo', 'shantanu', 'ceo', 'money', 'jim', 'cramer', 'cramer', 'hit', 'cramer', 'twitter', 'facebook', 'mad', 'money'], ['cramermad', 'money', 'jim', 'cramercnbc', 'jim', 'cramer', 'tuesday', 'dow', 'jones', 'industrial', 'average', 's', 'p', 'nasdaq', 'composite', 'mad', 'money', 'boeing', 'caterpillar', 'max', 'deere', 'friday', 'cramer', 'china', 'cramer', 'president', 'donald', 'trump', 'china', 'people', 'republic', 'china', 'cramer', 'adobe', 'salesforce', 'servicenow', 'vmware', 'splunk', 'twilio', 'workday', 'apple', 'china', 'ipo', 'uber', 'lauren', 'china', 'china', 'macy', 'walmart', 'ralph', 'lauren', 'china', 'trump', 'cramer', 'china', 'cramer', 'congress', 'white', 'house', 'china', 't', 'vodka', 'tequila', 'cramer', 'cramer', 'tuesday', 'jim', 'cramer', 'money', 'jim', 'apple', 'twilio', 'cramer', 'cramer', 'hit', 'cramer', 'twitter', 'facebook', 'mad', 'money'], ['sundar', 'pichai', 'ceo', 'google', 'berlin', 'google', 'germany', 'berlin', 'january', 'koall', 'getty', 'image', 'news', 'getty', 'imagesgoogle', 'alphabet', 'tuesday', 'morgan', 'stanley', 'alphabet', 'apple', 'amazon', 'uber', 'tuesday', 'monday', 'googl', 'tuesday', 'website', 'googl', 'youtube', 'trueview', 'reach', 'google', 'cfo', 'ruth', 'porat', 'youtube', 'youtube', 'porat', 'youtube', 'google', 'youtube', 'facebook', 'amazon', 'morgan', 'stanley', 'street', 'googl', 'tuesday', 'discovery', 'gmail', 'youtube', 'google', 'discover', 'cnbc', 'alphabet'], ['add', 'breakingviews', 'walt', 'disney', 'hulu', 'comcast', 'corp', 'netflix', 'hulu', 'comcast', 'disney', 'january', 'comcast', 'hulu', 't', 'comcast', 'hulu', 'disney', 'hulu', 'netflix', 'apple', 'disney', 'hulu', 'trip', 'miller', 'gullane', 'capital', 'partner', 'disney', 'disney', 'fox', 'comcast', 'hulu', 'disney', 'tuesday', 'disney', 'chief', 'executive', 'bob', 'iger', 'nbcuniversal', 'nbcuniversal', 'hulu', 'comcast', 'nbcuniversal', 'nbcuniversal', 'hulu', 'nbcu', 'hulu', 'disney', 'new', 'york', 'stock', 'exchange', 'comcast', 'hulu', 't', 'hulu', 'netflix', 'prime', 'video', 'suphantha', 'mukherjee', 'editing', 'shailesh', 'kuber', 'jonathan', 'oatis'], ['reportwith', 'apple', 'apple', 'china', 'dow', 's', 'p', 'nasdaq', 'apple', 'tuesday', 'dow', 'tuesday', 'president', 'donald', 'trump', 'apple', 'supreme', 'court', 'monday', 'app', 'store', 'ufc', 'apple', 'street', 'street', 'street', 'wedbush', 'dan', 'ives', 'china', 'foxconn', 'apple', 'frank', 'cappelleri', 'instinet', 'cappelleri', 'morgan', 'apple', 'bank', 'america', 'merrill', 'lynch', 'apple', 'friday', 'apple', 'apple', 'exchange'], ['s', 'p', 'tech', 'microsoft', 'boeing', 'ralph', 'lauren', 'index', 'dow', 's', 'p', 'nasdaq', 'tuesday', 'monday', 'united', 'china', 'president', 'donald', 'trump', 'summit', 'microsoft', 'corp', 'apple', 'trump', 'jerry', 'lucas', 'ubs', 'global', 'wealth', 'management', 'united', 'china', 's', 'p', 'et', 'dow', 'jones', 'industrial', 'average', 's', 'p', 'nasdaq', 'composite', 's', 'p', 'dow', 'max', 'morgan', 'stanley', 'disney', 'comcast', 'corp', 'ralph', 'lauren', 'corp', 's', 'p', 'north', 'america', 'nyse', 's', 'p', 'nasdaq', 'sruthi', 'shankar', 'amy', 'caren', 'daniel', 'bengaluru', 'editing', 'shounak', 'dasgupta', 'arun', 'koyyur'], ['lee', 'ai', 'china', 'silicon', 'valley', 'new', 'world', 'september', 'gaboury', 'patrick', 'mcmullan', 'getty', 'imagesthe', 'china', 'january', 'monday', 'silicon', 'valley', 'sinovation', 'venture', 'google', 'china', 'lee', 'sinovation', 'silicon', 'valley', 'chris', 'evdemon', 'president', 'donald', 'trump', 'china', 'treasury', 'department', 'committee', 'foreign', 'investment', 'cfius', 'trump', 'administration', 'cfius', 'applovin', 'cfius', 'cnbc', 'april', 'patientslikeme', 'china', 'heywood', 'patientslikeme', 'national', 'summit', 'detroit', 'june', 'harrer', 'bloomberg', 'getty', 'imageschinese', 'foreign', 'china', 'rhodium', 'china', 'cfius', 'vcs', 'vcs', 'matt', 'sheehan', 'macropolo', 'paulson', 'institute', 'sheehan', 'china', 'california', 'ai', 'valley', 'silicon', 'valley', 'use', 'privacy', 'policy', 'evdemon', 'ceo', 'sinovation', 'north', 'america', 'cnbc', 'evdemon', 'linkedin', 'basis', 'set', 'venture', 'dropbox', 'lan', 'xuezhao', 'basis', 'set', 'website', 'evdemon', 'ceo', 'sinovation', 'north', 'america', 'sinovation', 'beijing', 'april', 'lee', 'china', 'angela', 'bao', 'sinovation', 'silicon', 'valley', 'sinovation', 'beijing', 'evdemon', 'beijing', 'valley', 'lee', 'google', 'sinovation', 'china', 'lee', 'microsoft', 'research', 'china', 'china', 'silicon', 'valley', 'new', 'world', 'order', 'china', 'silicon', 'valley', 'china', 'apple', 'microsoft', 'google', 'lee', 'foxconn', 'silicon', 'valley', 'pacific', 'november', 'bloomberg', 'lee', 'china', 'bloomberg', 'china', 'america', 'china', 'ai', 'silicon', 'january', 'defense', 'department', 'sinovation', 'china', 'technology', 'transfer', 'strategy', 'sinovation', 'sinovation', 'sinovation', 'wonder', 'workshop', 'fictiv', 'china', 'baidu', 'peter', 'fang', 'american', 'china', 'sinovation', 'mobike', 'megvii', 'vipkid', 'lee', 'taiwan', 'tennessee', 'columbia', 'university', 'carnegie', 'mellon', 'university', 'lee', 'china', 'ai', 'china', 'ip', 'ai', 'lee', 'lee', 'cnbc', 'february', 'ip', 'ai', 'cnbc', 'sally', 'shin', 'chinese', 'chinese', 'sanctuarysubscribe', 'youtube'], ['add', 'oslo', 'statfjord', 'north', 'sea', 'reuters', 'equinor', 'wednesday', 'equinor', 'north', 'sea', 'equinor', 'equinor', 'exxonmobil', 'spirit', 'energy', 'norway', 'petroleum', 'directorate', 'gwladys', 'fouche', 'terje', 'solsvik'], ['new', 'york', 'trump', 'reuters', 'environmental', 'protection', 'agency', 'epa', 'exxon', 'mobil', 'chevron', 'corp', 'carl', 'icahn', 'renewable', 'fuel', 'standard', 'epa', 'obama', 'epa', 'epa', 'advanced', 'biofuels', 'association', 'august', 'court', 'appeal', 'sinclair', 'oil', 'epa', 'epa', 'epa', 'andrew', 'wheeler', 'april', 'gunasekara', 'epa', 'office', 'air', 'radiation', 'epa', 'scott', 'pruitt', 'pruitt', 'rins', 'rins', 'reuters', 'sinclair', 'sinclar', 'michael', 'abboud', 'rin', 'epa', 'trump', 'administration', 'rin', 'valero', 'energy', 'corp', 'pbf', 'energy', 'cvr', 'energy', 'cvr', 'jarrett', 'renshaw', 'editing', 'david', 'gregorio'], ['add', 'oslo', 'statfjord', 'north', 'sea', 'wednesday', 'equinor', 'north', 'sea', 'statfjord', 'equinor', 'equinor', 'morten', 'eek', 'eek', 'north', 'sea', 'equinor', 'equinor', 'equinor', 'equinor', 'exxonmobil', 'spirit', 'energy', 'norway', 'petroleum', 'directorate', 'terje', 'solsvik', 'victoria', 'klesty', 'jason', 'neely', 'jane', 'merriman', 'alexander', 'smith'], ['add', 'lng', 'exxon', 'mobil', 'corp', 'tuesday', 'rovuma', 'lng', 'mozambique', 'east', 'african', 'lng', 'italy', 'eni', 'qatar', 'papua', 'new', 'guinea', 'russia', 'united', 'rovuma', 'lng', 'area', 'lng', 'lng', 'mozambique', 'rovuma', 'venture', 'exxon', 'mobil', 'eni', 'china', 'national', 'petroleum', 'corp', 'rovuma', 'lng', 'mozambique', 'area', 'lpg', 'shanti', 's', 'nair', 'bengaluru', 'editing', 'arun', 'koyyur', 'anil'], ['exxon', 'mobil', 'corp', 'rovuma', 'lng', 'mozambique', 'east', 'african', 'lng', 'italy', 'eni', 'qatar', 'papua', 'new', 'guinea', 'russia', 'united', 'shanti', 's', 'nair', 'bengaluru', 'editing', 'arun', 'koyyur'], [], ['chairman', 'ceo', 'pinterest', 'ben', 'silbermann', 'new', 'york', 'stock', 'exchange', 'nyse', 'ipo', 'april', 'new', 'york', 'eisele', 'afp', 'getty', 'imageshere', 'wall', 'street', 'monday', 'piper', 'wendy', 'wendy', 'wen', 'overweight', 'october', 'wen', 'risk', 'citi', 'pinterest', 'pinterest', 'pinterest', 'mgmt', 'arpu', 'fcf', 'cagr', 'pin', 'buy', 'baird', 'pinterest', 'pinterest', 'pinterest', 'internet', 'pinterest', 'outperform', 'merck', 'amid', 'mrk', 'mrk', 'cagr', 'june', 'investor', 'day', 'mrk', 'morgan', 'zoom', 'morgan', 'zoom', 'docusign', 'overweight', 'december', 'zm', 'analyst', 'focus', 'list', 'piperjaffray', 'zoom', 'zoom', 'unified', 'communication', 'fortune', 'cio', 'digital', 'native', 'uber', 'zoom', 'overweight', 'bank', 'america', 'zoom', 'america', 'zoom', 'zoom', 'buy', 'po', 'zoom', 'zoom', 'zoom', 'po', 'saas', 'hsbc', 'exxonmobil', 'exxon', 'exxon', 'piperjaffray', 'wingstop', 'wingstop', 'wing', 'overweight', 'ebitda', 'kb', 'home', 'kb', 'home', 'kbh', 'ow', 'kbh', 'dhi', 'california', 'tol', 'tyson', 'food', 'tyson', 'food', 'ow', 'pt', 'best', 'idea', 'calavo', 'food', 'pt', 'tsn', 'rbc', 'intel', 'intel', 'intel', 'eps', 'deere', 'deere', 'neutral', 'na', 'large', 'ag', 'oem', 'c', 'f', 'construction', 'wirtgen', 'china', 'bv', 'large', 'ag', 'c', 'f', 'isi', 'chevron', 'chevron', 'chevron', 'chevron', 'chevron', 'cvx'], ['deputy', 'secretary', 'defense', 'patrick', 'shanahan', 'bosnia', 'herzegovina', 'minister', 'defense', 'marina', 'pendes', 'pentagon', 'defense', 'president', 'donald', 'trump', 'secretary', 'defense', 'patrick', 'shanahan', 'pentagon', 'boeing', 'senate', 'james', 'mattis', 'country', 'president', 'trump', 'patrick', 'shanahan', 'secretary', 'defense', 'white', 'house', 'sarah', 'huckabee', 'sander', 'acting', 'secretary', 'shanahan', 'department', 'defense', 'sander', 'donald', 'trump', 'acting', 'defense', 'secretary', 'patrick', 'shanahan', 'cabinet', 'white', 'house', 'washington', 'january', 'young', 'reutersshanahan', 'defense', 'president', 'trump', 'senate', 'national', 'defense', 'strategy', 'airman', 'shanahan', 'pentagon', 'trump', 'thursday', 'white', 'defense', 'sec', 'lunch', 'shanahan', 'shanahan', 'america', 'pentagon', 'shanahan', 'boeing', 'defense', 'pentagon', 'shanahan', 'defense', 'department', 'ig', 'shanahan', 'boeing', 'mattis', 'mattis', 'secretary', 'defense', 'patrick', 'shanahan', 'deputy', 'secretary', 'defense', 'pentagon', 'washington', 'thursday', 'harrer', 'bloomberg', 'getty', 'imagesin', 'mattis', 'america', 'marine', 'trump', 'defense', 'central', 'command', 'iraq', 'shanahan', 'senate', 'trump', 'washington', 'trump', 'organization', 'apprentice', 'trump', 'state', 'rex', 'tillerson', 'ceo', 'exxon', 'mobil', 'tillerson', 'state', 'department', 'trump', 'shanahan', 'pentagon', 'trump', 'united', 'north', 'korea', 'middle', 'east', 'mexico', 'iran', 'cnbc', 'dan', 'mangan'], ['hashmat', 'zubair', 'amazon', 'baltimore', 'maryland', 'april', 'rclodagh', 'kilcoyne', 'reutersas', 'china', 'goldman', 'sachs', 'amazon', 'apple', 'goldman', 'sachs', 'tuesday', 'david', 'kostin', 'goldman', 'president', 'donald', 'trump', 'china', 'beijing', 'dow', 'jones', 'industrial', 'average', 's', 'p', 'nasdaq', 'trade', 'representative', 'robert', 'lighthizer', 'goldman', 's', 'p', 'goldman', 'amazon', 'google', 'microsoft', 'apple', 'china', 'johnson', 'johnson', 'exxon', 'mobil', 'facebook', 'disney', 'home', 'depot', 't', 'mcdonald', 'goldman', 'netflix', 'comcast', 'well', 'fargo', 'kostin', 'goldman', 'goldman', 'china', 'goldman', 'procter', 'gamble', 'intel', 'chevron', 'boeing', 'kostin', 'broadcom', 'honeywell', 'nvdia', 'goldman', 'brown', 'set', 'nbcuniversal', 'cnbc'], ['siemens', 'aim', 'gas', 'power', 'september', 'profitability', 'share', 'ceo', 'munich', 'wednesday', 'gas', 'power', 'january', 'march', 'siemens', 'siemens', 'chief', 'executive', 'joe', 'kaeser', 'siemens', 'healthineers', 'gas', 'power', 'siemens', 'union', 'investment', 'christoph', 'niesel', 'spain', 'gamesa', 'bosch', 'kaeser', 'siemens', 'thyssenkrupp', 'abb', 'gas', 'power', 'kaeser', 'siemens', 'wednesday', 'march', 'infront', 'gas', 'power', 'germany', 'september', 'siemens', 'digital', 'smart', 'infrastructure', 'gas', 'power', 'siemens', 'kaeser', 'john', 'revill', 'additional', 'han', 'seidenstuecker', 'editing', 'georgina', 'prodhan', 'edmund', 'blair'], ['iran', 'baghdad', 'exxon', 'mobil', 'petrochina', 'prime', 'minister', 'adel', 'abdul', 'mahdi', 'tuesday', 'iraq', 'nahr', 'bin', 'umar', 'artawi', 'abdul', 'mahdi', 'iran', 'london', 'monday', 'united', 'iraq', 'iran', 'baghdad', 'washington', 'exxon', 'mobil', 'petrochina', 'abdul', 'mahdi', 'reuters', 'opec', 'exxon', 'mobil', 'united', 'iran', 'washington', 'baghdad', 'mobil', 'petrochina', 'abdul', 'mahdi', 'artawi', 'nahr', 'bin', 'umar', 'president', 'donald', 'trump', 'iran', 'tehran', 'washington', 'united', 'iraq', 'iraqi', 'reuters', 'monday', 'ahmed', 'rasheed', 'additional', 'john', 'davison', 'writing', 'ahmed', 'aboulenein', 'editing', 'louise', 'heaven', 'peter', 'graff'], ['baghdad', 'iraqi', 'prime', 'minister', 'adel', 'abdul', 'mahdi', 'tuesday', 'exxon', 'mobil', 'united', 'iran', 'london', 'monday', 'iraq', 'iran', 'baghdad', 'exxon', 'mobil', 'petrochina', 'abdul', 'mahdi', 'iran', 'president', 'donald', 'trump', 'tehran', 'ahmed', 'rasheed', 'writing', 'ahmed', 'aboulenein', 'editing', 'louise', 'heaven'], ['iraqi', 'pm', 'say', 'iraq', 'close', 'signing', 'initial', 'agreement', 'exxon', 'mobil', 'petrochina', 'energy', 'megaproject', 'south'], ['china', 'boeing', 'caterpillar', 'italy', 'exxon', 'chevron', 'dow', 's', 'p', 'nasdaq', 'add', 'wall', 'street', 'tuesday', 'united', 'china', 'vice', 'premier', 'liu', 'united', 'beijing', 'tuesday', 'president', 'donald', 'trump', 'sunday', 'friday', 'china', 'matt', 'forester', 'bny', 'mellon', 'lockwood', 'king', 'prussia', 'boeing', 'caterpillar', 'boeing', 'barclays', 's', 'p', 'nasdaq', 'commission', 'italy', 'et', 'dow', 's', 'p', 'nasdaq', 'exxon', 'mobil', 'corp', 'chevron', 'corp', 's', 'p', 'refinitiv', 'international', 'group', 'regeneron', 'pharmaceutical', 'labor', 'department', 'march', 'february', 'et', 'amy', 'caren', 'daniel', 'shreyashi', 'sanyal', 'bengaluru', 'editing', 'shounak', 'dasgupta'], ['michael', 'wirth', 'ceo', 'jeffery', 'cnbcchevron', 'occidental', 'petroleum', 'anadarko', 'occidental', 'chevron', 'anadarko', 'chevron', 'occidental', 'anadarko', 'chevron', 'chevron', 'occidental', 'warren', 'buffett', 'berkshire', 'hathaway', 'occidental', 'anadarko', 'occidental', 'anadarko', 'africa', 'french', 'occidental', 'occidental', 'chevron', 'cnbcsquawk', 'streeton', 'monday', 'cnbc', 'david', 'faber', 'anadarko', 'occidental', 'chevron', 'oxy', 'oxy', 'chevron', 'chevron', 'oxy', 'andrew', 'dittmar', 'm', 'chevron', 'monday', 'morgan', 'stanley', 'chevron', 'chevron', 'anadarko', 'b', 'chevron', 'total', 'chevron', 'cnbc', 'chevron', 'ceo', 'mike', 'wirth', 'anadarko', 'boxone', 'chevron', 'anadarko', 'chevron', 'basin', 'chevron', 'anadarko', 'colorado', 'dj', 'chevron', 'gulf', 'mexico', 'chevron', 'chevron', 'anadarko', 'chevron', 'anadarko', 'mozambique', 'africa', 'chevron', 'lng', 'ceo', 'anadarkosquawk', 'boxthe', 'mozambique', 'lng', 'total', 'total', 'lng', 'qatargas', 'royal', 'dutch', 'shell', 'petronas', 'wood', 'mackenzie', 'anadarko', 'mozambique', 'lng', 'total', 'lng', 'nicholas', 'browne', 'wood', 'mackenzie', 'wood', 'mackenzie', 'chevron', 'anadarko', 'ultramajor', 'bp', 'exxonmobil', 'shell', 'mozambique', 'lng', 'chevron', 'frank', 'harris', 'wood', 'mackenzie', 'lng'], ['exxon', 'mobil', 'corp', 'friday', 'liza', 'phase', 'guyana', 'guyana', 'friday', 'phase', 'guyana', 'exploration', 'exxon', 'liza', 'phase', 'liza', 'phase', 'payara', 'arundhati', 'sarkar', 'bengaluru', 'editing', 'maju', 'samuel'], [], ['general', 'electric', 'baker', 'hughes', 'friday', 'february', 'gulf', 'mexico', 'oklahoma', 'north', 'dakota', 'energy', 'information', 'administration', 'eia', 'texas', 'new', 'mexico', 'exxon', 'mobil', 'corp', 'chevron', 'corp', 'permian', 'iran', 'cowen', 'e', 'p', 'cowen', 'e', 'p', 'united', 'simmons', 'piper', 'jaffray', 'simmons', 'april', 'scott', 'disavino', 'editing', 'marguerita', 'choy'], ['houston', 'exxon', 'mobil', 'corp', 'baytown', 'texas', 'baytown', 'baytown', 'basin', 'gulf', 'coast', 'darren', 'wood', 'exxon', 'baytown', 'exxon', 'basin', 'beaumont', 'texas', 'baytown', 'exxon', 'gulf', 'initiative', 'wood', 'erwin', 'seba', 'editing', 'peter', 'cooney'], ['cash', 'share', 'add', 'london', 'dutch', 'shell', 'thursday', 'exxon', 'mobil', 'chevron', 'bp', 'gmt', 'exxon', 'shell', 'bg', 'group', 'april', 'shell', 'jefferies', 'jason', 'boostnet', 'cc', 'shell', 'chief', 'executive', 'officer', 'ben', 'van', 'beurden', 'shell', 'lng', 'integrated', 'gas', 'lng', 'ron', 'bousso', 'jason', 'neely', 'susan', 'fenton'], ['royal', 'dutch', 'shell', 'torzhok', 'rudakov', 'bloomberg', 'getty', 'imagesroyal', 'dutch', 'shell', 'thursday', 'exxon', 'mobil', 'chevron', 'bp', 'shell', 'chief', 'executive', 'officer', 'ben', 'van', 'beurden', 'cc', 'shell', 'lng'], ['lng', 'add', 'london', 'dutch', 'shell', 'thursday', 'exxon', 'mobil', 'chevron', 'bp', 'shell', 'chief', 'executive', 'officer', 'ben', 'van', 'beurden', 'cc', 'shell', 'lng', 'ron', 'bousso', 'jason', 'neely', 'susan', 'fenton'], ['remix', 'apple', 'money', 'jim', 'cramerapple', 'wednesday', 'cnbc', 'jim', 'cramer', 'mad', 'money', 'apple', 'apple', 'wednesday', 'apple', 'cramer', 'apple', 'apple', 'apple', 'january', 'apple', 'cramer', 'apple', 'carmer', 'new', 'york', 'stock', 'news', 'agency', 'getty', 'imagesthe', 'cramer', 'dow', 'jones', 'industrial', 'average', 's', 'p', 'nasdaq', 'composite', 'dow', 'jones', 'nasdaq', 'cramer', 'read', 'meat', 'tiskilwa', 'illinois', 'april', 'acker', 'bloomberg', 'getty', 'imagesbeyond', 'meat', 'burger', 'carmer', 'ipo', 'meat', 'cramer', 'lyft', 'ipo', 'rosensweig', 'ceo', 'cheggscott', 'mlyn', 'cnbcwhen', 'chegg', 'ceo', 'dan', 'rosensweig', 'mad', 'money', 'wednesday', 'rosensweig', 'rosensweig', 'cramer', 'mad', 'money', 'kohl', 'ceo', 'michelle', 'go', 'exxon', 'mobil', 'exxon', 'ceo', 'mike', 'wirth', 'chevron', 'cnbc', 'david', 'faber', 'anadarko', 'occidental', 'chevron', 'exxon', 'delta', 'air', 'apple', 'kohl', 'cramer', 'cramer', 'hit', 'cramer', 'twitter', 'facebook', 'mad', 'money'], ['kohl', 'kohl', 'ceo', 'michelle', 'go', 'exxon', 'mobil', 'exxon', 'ceo', 'mike', 'wirth', 'chevron', 'cnbc', 'david', 'faber', 'anadarko', 'occidental', 'chevron', 'exxon', 'delta', 'air', 'allergan', 'arrowhead', 'pharmaceutical', 'arrowhead', 'wayfair', 'bed', 'bath', 'wayfair', 'wayfair', 'wayfair', 'varonis', 'palo', 'alto', 'network', 'panw', 'cramer', 'money', 'jim', 'palo', 'alto', 'network', 'kohl', 'cramer', 'cramer', 'hit', 'cramer', 'twitter', 'facebook', 'mad', 'money'], ['add', 'ceo', 'sonatrach', 'wednesday', 'organization', 'petroleum', 'exporting', 'europe', 'hachichi', 'sonatrach', 'april', 'abdelmoumene', 'ould', 'kaddour', 'algeria', 'labour', 'day', 'ceo', 'algeria', 'france', 'total', 'italy', 'eni', 'spain', 'britain', 'bp', 'norway', 'equinor', 'algeria', 'march', 'exxon', 'mobil', 'algeria', 'hachichi', 'sonatrach', 'hamid', 'ould', 'ahmed', 'editing', 'edmund', 'blair'], ['london', 'wednesday', 'venezuelan', 'exxon', 'mobil', 'angolan', 'mostarda', 'brent', 'angolan', 'mostarda', 'total', 'fpso', 'kaombo', 'sul', 'april', 'state', 'sonangol', 'june', 'mostarda', 'china', 'june', 'mostarda', 'angolan', 'june', 'brent', 'doba', 'blend', 'brent', 'perenco', 'congolese', 'djeno', 'china', 'unipec', 'yombo', 'india', 'ioc', 'june', 'thursday', 'ioc', 'june', 'opec', 'april', 'reuters', 'iran', 'venezuela', 'saudi', 'arabia', 'international', 'monetary', 'fund', 'africa', 'julia', 'payne', 'editing', 'kirsten', 'donovan'], ['april', 'environmental', 'protection', 'agency', 'white', 'house', 'epa', 'april', 'white', 'house', 'epa', 'renewable', 'fuel', 'standard', 'epa', 'white', 'house', 'white', 'house', 'epa', 'federal', 'register', 'epa', 'white', 'house', 'president', 'donald', 'trump', 'epa', 'barack', 'obama', 'epa', 'exxonmobil', 'chevron', 'reuters', 'epa', 'appeal', 'court', 'epa', 'jarrett', 'renshaw', 'chris', 'prentice', 'new', 'york', 'humeyra', 'pamuk', 'washington', 'richard', 'valdmanis', 'david', 'gregorio'], ['update', 'london', 'april', 'bp', 'bp', 'chief', 'executive', 'officer', 'bob', 'dudley', 'gmt', 'bp', 'whiting', 'indiana', 'financial', 'officer', 'brian', 'gilvary', 'exxon', 'mobil', 'chevron', 'bp', 'exxon', 'total', 'royal', 'dutch', 'shell', 'april', 'bp', 'bernstein', 'oswald', 'clint', 'bp', 'bp', 'bp', 'russia', 'rosneft', 'bhp', 'bp', 'march', 'bhp', 'ron', 'bousso', 'jason', 'neely'], ['add', 'london', 'april', 'bp', 'bp', 'chief', 'executive', 'officer', 'bob', 'dudley', 'gmt', 'bp', 'whiting', 'indiana', 'financial', 'officer', 'brian', 'gilvary', 'exxon', 'mobil', 'chevron', 'bp', 'exxon', 'total', 'royal', 'dutch', 'shell', 'april', 'bp', 'bernstein', 'oswald', 'clint', 'bp', 'bp', 'bp', 'russia', 'rosneft', 'bhp', 'bp', 'march', 'bhp', 'ron', 'bousso', 'jason', 'neely'], ['london', 'april', 'bp', 'bp', 'chief', 'executive', 'officer', 'bob', 'dudley', 'exxon', 'mobil', 'chevron', 'total', 'exxon', 'bp', 'russia', 'rosneft', 'bhp', 'ron', 'bousso', 'jason', 'neely'], ['cfo', 'box', 'europebp', 'tuesday', 'reuters', 'bp', 'brian', 'gilvary', 'bp', 'cnbc', 'squawk', 'box', 'europe', 'march', 'january', 'bp', 'box', 'europeprofits', 'international', 'brent', 'west', 'texas', 'intermediate', 'wti', 'brent', 'tuesday', 'wti', 'stateside', 'chevron', 'exxon', 'mobil', 'france', 'total', 'royal', 'dutch', 'shell', 'thursday'], ['add', 'houston', 'april', 'exxon', 'mobil', 'corp', 'chevron', 'corp', 'friday', 'chevron', 'wall', 'street', 'exxon', 'senior', 'vice', 'president', 'jack', 'williams', 'exxon', 'refinitiv', 'eikon', 'rbc', 'capital', 'market', 'chevron', 'anadarko', 'petroleum', 'petroleum', 'corp', 'wednesday', 'chevron', 'chief', 'executive', 'michael', 'wirth', 'friday', 'anadarko', 'anadarko', 'chevron', 'edward', 'jones', 'chevron', 'wall', 'street', 'anadarko', 'exxon', 'basin', 'west', 'texas', 'new', 'mexico', 'williams', 'exxon', 'basin', 'exxon', 'irving', 'exxon', 'friday', 'san', 'ramona', 'chevron', 'jennifer', 'hiller', 'houston', 'editing', 'jonathan', 'oatis', 'tom', 'brown'], ['jeremy', 'siegelclosing', 'bellmoney', 'wharton', 'school', 'jeremy', 'siegel', 'cnbc', 's', 'p', 'nasdaq', 'friday', 'siegel', 'closing', 'friday', 'commerce', 'department', 'gdp', 'exxon', 'mobile', 'intel', 'ford', 'motor', 's', 'p', 'nasdaq', 'friday', 'dow', 'jones', 'industrial', 'average', 'china', 'china', 'china', 'siegel', 'cnbc', 'fred', 'imbert'], ['immediate', 'releasechicago', 'il', 'analyst', 'blog', 'zacks', 'equity', 'research', 'adobe', 'adbe', 'report', 'conocophillips', 'cop', 'report', 'exelon', 'exc', 'report', 'anadarko', 'petroleum', 'apc', 'report', 'home', 'depot', 'hd', 'report', 'wednesday', 'analyst', 'blog', 'analyst', 'adobe', 'conocophillips', 'exelonthe', 'zacks', 'research', 'daily', 'research', 'daily', 'adobe', 'conocophillips', 'exelon', 'adobe', 'zacks', 'software', 'zacks', 'adobe', 'magento', 'marketo', 'creative', 'cloud', 'adobe', 'europe', 'adobe', 'conocophillips', 'zacks', 'conocophillips', 'e', 'p', 'zacks', 'eagle', 'ford', 'delaware', 'bakken', 'conocophillips', 'cagr', 'conocophillips', 'conocophillips', 'zacks', 'electric', 'power', 'industry', 'exelon', 'zacks', 'exelon', 'exelon', 'uranium', 'exelon', 'anadarko', 'petroleum', 'home', 'report', 'click', 'medium', 'contactzacks', 'investment', 'inherent', 'zacks', 'investment', 'research', 'zacks', 'rank', 's', 'p', 'visit'], ['nyse', 'home', 'depot', 'nyse', 'ceo', 'marvin', 'ellison', 'lowe', 'lowe', 'wednesday', 'getty', 'home', 'depot', 'lowe', 'lowe', 'lowe', 'home', 'depot', 'operating', 'margin', 'ttm', 'lowe', 'wednesday', 'lowe', 'home', 'depot', 'lowe', 'february', 'lowe', 'wall', 'street'], ['getty', 'sears', 'holding', 'penney', 'ascena', 'retail', 'group', 'sears', 'dividend', 'aristocrat', 'peter', 'lynch', 'online', 'amazon', 'amazon', 'food', 'victoria', 'secret', 'l', 'brand', 'gap', 'banana', 'republic', 'old', 'navy', 'h', 'm', 'zara', 'uniqlo', 'american', 'walmart', 'walmart', 'walmart', 'walmart', 'tower', 'data', 'costco', 's', 'p', 'costco', 'north', 'america', 'costco', 'amazon', 'prime', 'boxed', 's', 'p', 'costco', 'tj', 'maxx', 'marshall', 'homegoods', 'amazon', 'ross', 'burlington', 'tjx', 'tjx', 'marmaxx', 'marshall', 'tj', 'maxx', 'homegoods', 'marmaxx', 'homegoods', 'tjx', 'target', 'amazon', 'target', 'shipt', 'instacart', 'grand', 'junction', 'drive', 'target', 'target', 'toy', 'r', 'vineyard', 'vine', 'dividend', 'aristocrat', 'place', 'gymboree', 'gymboree', 'child', 'place', 'child', 'place', 'gymboree', 'child', 'gymboree', 'gymboree', 'child', 'child', 'place', 'gymboree', 'crazy', 'gymboree', 'gymboree', 'child', 'gymboree', 'home', 'depot', 'lowe', 'lowe', 'home', 'depot', 'home', 'depot', 'fast', 'home', 'depot', 'home', 'depot'], ['wednesday', 'trump', 'china', 'united', 'china', 'dow', 'jones', 'industrial', 'average', 'dji', 's', 'p', 'nasdaq', 'composite', 'index', 'cboe', 'volatility', 'index', 'vix', 'nyse', 'nasdaq', 'benchmark', 'perform', 'dow', 'verizon', 'vz', 'report', 'boeing', 'ba', 'report', 'zacks', 'rank', 'hold', 'zacks', 'rank', 'strong', 'buy', 's', 'p', 's', 'p', 'communication', 'select', 'sector', 'spdr', 'fund', 'xlc', 'nasdaq', 'alphabet', 'googl', 'report', 'apple', 'aapl', 'report', 'delay', 'auto', 'united', 'china', 'president', 'trump', 'cnbc', 'trump', 'ford', 'f', 'general', 'motor', 'gm', 'white', 'house', 'european', 'union', 'eu', 'japan', 'economic', 'data', 'turn', 'jitteryon', 'april', 'ny', 'empire', 'state', 'index', 'april', 'china', 'china', 'april', 'headlinessouthwest', 'hike', 'dividend', 'announces', 'new', 'buyback', 'plansouthwest', 'luv', 'report', 'zto', 'express', 'earnings', 'lag', 'rise', 'solid', 'revenueszto', 'express', 'cayman', 'zto', 'report', 'zacks', 'consensus', 'estimate', 'radical', 'new', 'technology', 'trillion', 'opportunityimagine', 'microsoft', 'motorola', 'industrial', 'revolution'], ['apple', 'nasdaq', 'aapl', 'spotify', 'nyse', 'spotify', 'spotify', 'spotify', 'spotify', 'getty', 'imagesaccording', 'edison', 'research', 'data', 'nielsen', 'spotify', 'spotify', 'gimlet', 'anchor', 'february', 'parcast', 'parcast', 'gimlet', 'spotify', 'disney', 'marvel', 'gimlet', 'parcast', 'spotify', 'gimlet', 'parcast', 'anchor', 'anchor', 'anchor', 'anchor', 'spotify', 'spotify', 'spotify', 'spotify', 'spotify', 'spotify', 'spotify', 'apple', 'spotify', 'apple', 'daniel', 'ek', 'spotify', 'spotify', 'apple', 'spotify', 'spotify'], ['supreme', 'court', 'apple', 'nasdaq', 'aapl', 'app', 'marketfoolery', 'chris', 'hill', 'motley', 'fool', 'asset', 'management', 'barker', 'nyse', 'bed', 'bath', 'nasdaq', 'bbby', 'motley', 'fool', 'monday', 'marketfoolery', 'chris', 'hill', 'mfam', 'fund', 'barker', 'barker', 'hill', 'wall', 'street', 'supreme', 'court', 'apple', 'app', 'store', 'apple', 'apple', 'apple', 'nasdaq', 'apple', 'apple', 'apple', 'kavanaugh', 'stay', 'let', 'apple', 'china', 'mfam', 'fund', 'barker', 'hill', 'right', 'hill', 'motley', 'fool', 'texas', 'roadhouse', 'china', 'china', 'texas', 'roadhouse', 'china', 'apple', 'dow', 'dow', 's', 'p', 's', 'p', 'dow', 'take', 'uber', 'uber', 'ipo', 'uber', 'ipo', 'ipo', 'friday', 'ipo', 'peak', 'meat', 'ipo', 'uber', 'uber', 'uber', 'facebook', 'ipo', 'lyft', 'uber', 'ipo', 'hey', 'ipo', 'uber', 'uber', 'lyft', 'ron', 'gross', 'motley', 'fool', 'money', 'uber', 'lyft', 'ipo', 'china', 'facebook', 'dara', 'khosrowshahi', 'ceo', 'uber', 'facebook', 'facebook', 'amazon', 'amazon', 'amazon', 'amazon', 'amazon', 'uber', 'facebook', 'ipo', 'facebook', 'amazon', 'uber', 'lyft', 'facebook', 'amazon', 'uber', 'lyft', 'uber', 'lyft', 'hey', 'facebook', 'amazon', 'facebook', 'amazon', 'amazon', 'web', 'lyft', 'uber', 'facebook', 'amazon', 'ipo', 'meat', 'hill', 'ipo', 'uber', 'ipo', 'meat', 'ceo', 'ipo', 'ipo', 'meat', 'ipo', 'meat', 'ipo', 'bed', 'bath', 'ceo', 'ceo', 'steven', 'temares', 'bed', 'bath', 'ceo', 'bed', 'bath', 'ceo', 'bed', 'bath', 'bed', 'bath', 'amazon', 'bed', 'bath', 'amazon', 'best', 'buy', 'joly', 'best', 'buy', 'amazon', 'best', 'buy', 'amazon', 'bed', 'bath', 'barker', 'best', 'buy', 'hill', 'bed', 'bath', 'geek', 'geek', 'squad', 'bed', 'bath', 'raider', 'lost', 'ark', 'ark', 'covenant', 'bed', 'bath', 'hill', 'ceo', 'jcpenney', 'ceo', 'ron', 'johnson', 'ron', 'johnson', 'apple', 'store', 'gosh', 'apple', 'jcpenney', 'apple', 'store', 'bed', 'bath', 'hubert', 'joly', 'ron', 'hill', 'ceo', 'sixer', 'philadelphia', 'barker', 'motley', 'fool', 'dan', 'boyd', 'chris', 'hill'], ['disney', 'nyse', 'netflix', 'nasdaq', 'nflx', 'disney', 'marvel', 'studio', 'star', 'war', 'netflix', 'mickey', 'netflix', 'disney', 'getty', 'tubi', 'tv', 'netflix', 'netflix', 'tubi', 'netflix', 't', 'nyse', 'hbo', 'hbo', 'hbo', 'game', 'netflix', 'netflix', 'bird', 'box', 'netflix', 'game', 'disney', 'disney', 'marvel', 'studio', 'disney', 'ip', 'mandalorian', 'star', 'war', 'disney', 'netflix', 'netflix', 'netflix', 'premium', 'plan', 'netflix', 'hbo', 'disney', 'apple', 'nasdaq', 'aapl', 'comcast', 'nasdaq', 'cmcsa', 't', 'amazon', 'prime', 'disney', 'disney', 'hulu', 'disney', 'hulu', 'hulu', 'netflix', 'hulu', 'netflix', 'netflix', 'disney', 'disney', 'disney'], ['nvidia', 'nvda', 'report', 'wednesday', 'gpu', 'nvidia', 'wall', 'street', 'nvidia', 'overviewnvidia', 'ceo', 'jensen', 'huang', 'intel', 'intc', 'report', 'advanced', 'micro', 'amd', 'report', 'micron', 'mu', 'report', 'lam', 'research', 'lrcx', 'report', 'international', 'data', 'corporation', 'wednesday', 'apple', 'aapl', 'report', 'chinese', 'nvda', 's', 'p', 'nvidia', 'nvidia', 'wednesday', 'trendsnvidia', 'zacks', 'consensus', 'estimate', 'nvda', 'gpu', 'nvidia', 'eps', 'nvidia', 'nvda', 'nvidia', 'linenvidia', 'zacks', 'rank', 'buy', 'thursday', 'zacks', 'report', 'click']]

In [42]:
len(texts)

146

In [45]:
texts = [i for i in texts if i]

In [36]:
count

70

In [35]:
count = 0
for i in corpus_tfidf:
    print(len(i))
    count+=1

20
26
58
18
18
17
46
15
79
22
29
70
14
35
35
29
22
25
33
7
12
32
8
14
25
45
26
33
65
35
23
23
171
24
29
18
108
7
14
27
20
17
24
23
30
30
17
6
14
9
12
39
43
10
59
32
35
36
13
13
6
25
20
77
26
35
50
56
53
24


ValueError: not enough values to unpack (expected 2, got 0)

In [46]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(a) for a in texts]
corpus_tfidf = tf_idf(corpus)
lda = LdaModel(corpus_tfidf,  # list of lists containing tuples (word index, word freq)
                   id2word=dictionary,  # change nums back to words
                   num_topics=5,  # need to set num topics
                   passes=10)

In [47]:
print_lda(lda)

[   (   0,
        '0.011*"zacks" + 0.007*"report" + 0.007*"rank" + 0.007*"china" + '
        '0.004*"boeing" + 0.004*"p" + 0.004*"s" + 0.004*"goldman"'),
    (   1,
        '0.004*"display" + 0.003*"apple" + 0.003*"lowe" + 0.003*"berkshire" + '
        '0.003*"epa" + 0.003*"japan" + 0.002*"roku" + 0.002*"hd"'),
    (   2,
        '0.012*"hulu" + 0.010*"disney" + 0.010*"comcast" + 0.005*"bp" + '
        '0.005*"lng" + 0.004*"huawei" + 0.004*"rovuma" + 0.004*"sears"'),
    (   3,
        '0.010*"cramer" + 0.008*"apple" + 0.005*"app" + 0.004*"store" + '
        '0.004*"shell" + 0.004*"boeing" + 0.004*"money" + 0.003*"max"'),
    (   4,
        '0.005*"netflix" + 0.002*"kratos" + 0.002*"epa" + 0.002*"su" + '
        '0.002*"facebook" + 0.002*"star" + 0.002*"defense" + 0.002*"oklahoma"')]


In [22]:

# print lda model
lda,corpus_tfidf = get_lda(news, 2, 10)
print_lda(lda)

# get topic for single document
# a = "'A version of this article first appeared in the Reliable Sources newsletter. You can sign up for free right here.   This is an incredibly difficult time for Alex Trebek, his family members, and the extended Jeopardy! family that spans the globe. Trebek showed tremendous courage by recording a candid video message to fans about his stage 4 pancreatic cancer diagnosis. He even managed to work in a joke about being under contract for three more years. Trebek was diagnosed earlier this week, and his video was released on Wednesday afternoon.  In a time that is all about what is keeping us apart, we got tough news today about someone who has always brought America together, literally for decades, CNN\'s Chris Cuomo said Wednesday night. I don\'t care what your race, color, creed, gender, or bank account level, you\'ve watched Jeopardy. Since 1984 Alex Trebek has been the smartest guy in our living rooms, teaching us, but more importantly, bringing us together. Trebek\'s show puts facts first, Cuomo said, and we need him, now mo"
# print(get_topic(a))


[   (   0,
        '0.007*"advertis" + 0.005*"amazon" + 0.004*"revenu" + 0.004*"growth" + '
        '0.003*"500" + 0.003*"expect" + 0.003*"ad" + 0.003*"quarterli"'),
    (   1,
        '0.006*"canadian" + 0.006*"privaci" + 0.005*"therrien" + '
        '0.005*"person" + 0.005*"inform" + 0.004*"attorney" + 0.004*"jame" + '
        '0.004*"govern"')]


In [13]:
get_topic(0,corpus_tfidf)

NameError: name 'corpus_tfidf' is not defined

In [18]:
'advertis1' in [t[0] for t in lda.show_topic(0,20)]

False

In [16]:
news_list = [a[1] for a in news]

In [17]:
exp_smooth(0.65,news_list)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\shery\\OneDrive\\Columbia MSBA\\Courses\\2018-19 Spring Semester\\IEOR4524_Analytics_in_Practice\\Project\\github\\dictionaries\\negative.txt'

In [27]:
[t[0] for i in range(2) for t in lda.show_topic(i,20)]

['advertis',
 'amazon',
 'quarterli',
 '500',
 'ad',
 'nasdaq',
 'attorney',
 'index',
 'revenu',
 '3m',
 'growth',
 'industri',
 'york',
 'say',
 'expect',
 'canadian',
 'contact',
 'new',
 'rate',
 'account',
 'therrien',
 'privaci',
 'regul',
 'pick',
 'court',
 'refus',
 'inform',
 'canadian',
 'commission',
 'law',
 'forc',
 'seek',
 'cambridg',
 'broke',
 'acknowledg',
 'order',
 'session',
 'carew',
 'sinad',
 'slightli']

In [18]:
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=True)

NameError: name 'pyLDAvis' is not defined

In [57]:
emotion_analyzer(news)

negative  positive
1    0.047414  0.006466
2    0.018801  0.011751
3    0.010135  0.003378
4    0.027356  0.006079
5    0.020525  0.005701
6    0.020525  0.005701
7    0.020525  0.005701
8    0.020525  0.005701
9    0.009634  0.013487
10   0.009634  0.013487
11   0.009634  0.013487
12   0.014085  0.005282
13   0.009634  0.013487
14   0.006270  0.022727
15   0.011292  0.007528
16   0.014085  0.005282
17   0.006270  0.022727
18   0.011292  0.007528
19   0.014085  0.005282
20   0.000000  0.007463
21   0.006270  0.022727
22   0.011292  0.007528
23   0.014085  0.005282
24   0.000000  0.007463
25   0.006270  0.022727
26   0.011292  0.007528
27   0.000000  0.018405
28   0.000000  0.007463
29   0.007130  0.012478
30   0.000000  0.018405
..        ...       ...
147  0.029557  0.000000
148  0.012690  0.007614
149  0.044280  0.000000
150  0.012690  0.007614
151  0.012690  0.007614
152  0.029557  0.000000
153  0.044280  0.000000
154  0.012690  0.007614
155  0.012690  0.007614
156  0.029557  0.000000
157  0.044280  0.000000
158  0.029557  0.000000
159  0.012690  0.007614
160  0.012690  0.007614
161  0.012690  0.007614
162  0.029557  0.000000
163  0.012690  0.007614
164  0.012690  0.007614
165  0.029557  0.000000
166  0.029557  0.000000
167  0.012690  0.007614
168  0.016514  0.001835
169  0.016514  0.001835
170  0.029557  0.000000
171  0.016514  0.001835
172  0.016514  0.001835
173  0.029557  0.000000
174  0.029557  0.000000
175  0.016514  0.001835
176  0.037037  0.000000

[176 rows x 2 columns]

In [20]:
# news = load_news("aapl",10)

In [69]:
# text = []
# for i in news:
#     text.append(i[1])

In [105]:
# print_lda(noun_lda(text,sw,4,10)[0])

[   (   0,
        '0.014*"qualcomm" + 0.008*"netflix" + 0.008*"foxconn" + 0.008*"huawei" '
        '+ 0.008*"book" + 0.008*"display" + 0.008*"japan" + 0.007*"roku"'),
    (   1,
        '0.012*"acm" + 0.009*"dutch" + 0.008*"march" + 0.008*"qualcomm" + '
        '0.008*"huawei" + 0.007*"sec" + 0.007*"google" + 0.007*"app"'),
    (   2,
        '0.014*"qualcomm" + 0.008*"oled" + 0.007*"eps" + 0.007*"qcom" + '
        '0.007*"nasdaq" + 0.007*"chesler" + 0.006*"pro" + 0.006*"macbook"'),
    (   3,
        '0.010*"intel" + 0.009*"brussels" + 0.009*"thursday" + '
        '0.008*"qualcomms" + 0.008*"qualcomm" + 0.007*"project" + '
        '0.007*"titan" + 0.007*"spotifys"')]


In [2]:
text = """(Repeats to amend headline)* Huawei purchases $70 bln worth components annually* Huawei says sources $11 bln worth of goods from U.S.* Huawei has been stockpiling some parts - supply chain sources* Asian supplier shares fall on news of U.S. blacklistHONG KONG/BEIJING, May 16 (Reuters) - The latest U.S. broadside against Huawei that puts the Chinese firm on an exports blacklist threatens to rattle the global tech supply chain, linked closely to the $105 billion business of the world's top supplier of telecoms network equipment.The Trump administration has said it would add Huawei Technologies and 70 affiliates to its "Entity List" - a move that will likely ban the firm from acquiring U.S. components and technology without government approval, adding another incendiary element to the U.S.-China trade war.The ban is not yet effective.A similar U.S. ban on China's ZTE Corp had almost crippled business for the smaller Huawei rival early last year before the curb was lifted.Such sanctions on Huawei are, however, likely to have ramifications beyond the company itself, analysts said.It would disrupt Huawei's business at a minimum and all but put it out of business in an extreme, while its U.S. suppliers would also be hit, they said.Out of $70 billion Huawei spent for component procurement in 2018, some $11 billion went to U.S. firms including Qualcomm , Intel Corp and Micron Technology Inc, and they could see that revenue disappear.On the other hand, U.S. companies like Apple face the risk of severe retaliation from China, a key market. "This is going to be very messy," a China-based source at a U.S. tech company said.It will be tough for Huawei too, the person said, noting none of its U.S. suppliers "can be replaced by Chinese ones, not within a few years, at least. By then, they are already dead".Revenue for the company, also the world's second-biggest maker of smartphones, touched 721 billion yuan ($105 billion) last year, eight times ZTE's and half the annual sales of South Korea's Samsung Electronics Co.But its business has come under pressure over the past year given mounting international scrutiny, led by U.S. allegations that its equipment could be used by Beijing for spying, a concern the company has said is unfounded.STOCKPILINGA range of Asian and European suppliers would also be hurt if Huawei was forced to curb production, while telecom carriers that rely on Huawei, and have largely resisted U.S. calls to bar the company, would be left scrambling just as countries race to roll out next-generation 5G mobile networks. "Huawei being unable to manufacture network servers, for example, because they can't get key U.S. components would mean they also stop buying parts from other countries altogether," said an executive at a Huawei chip supplier. "They can relatively better manage component sourcing for mobile phones because they have their own component businesses for smartphones. But server and network, it's a different story," the executive said.According to brokerage Jefferies, the sanctions would mean a "nightmare for China's 5G" too. The country, which is targeting a nationwide rollout next year, will very likely slow down its 5G push as a result, it added.However, industry participants pointed out that Huawei had been stockpiling components such as chips to ease disruptions.Its initial target was to build inventories of six to nine months, and it has recently been raised to 12 and, in some cases, 24 months, Jefferies said.Shares in Huawei suppliers fell across in Asia on the news of the U.S. blacklist.South Korea's Samsung dropped 2.4%, SK Hynix fell 3.5%, while China's Luxshare Precision Industry fell as much as 6.1%. Shares in ZTE also tumbled.Huawei has said it is "ready and willing to engage with the U.S. government and come up with effective measures to ensure product security".Its rotating Chairman Eric Xu also told Reuters in a recent interview that "in case of unforeseen events ... we definitely have our contingency plan. What we have prepared has already been used in some of our products in the Chinese market".Huawei has spearheaded China's campaign to develop its own high-end technologies to reduce reliance on imports and such efforts have taken on urgency after U.S. sanctions on ZTE.The ZTE case led to some "benefits" and "external pressures have developed into internal drivers" in China, said Wan Gang, vice chairman of China's parliamentary advisory body.TRADE TALKSThe pain for Huawei's supply chain would be redoubled if the trade war put a damper on the Chinese technology industry. "The bigger concern would be U.S. allies that used to buy Huawei's components may not continue businesses with Huawei, because of fear of possibly upsetting the United States," said Doh Hyun-woo, an analyst at NH Investment & Securities in Seoul.The Trump administration's rhetoric toward China had cooled in recent days after another round of tariffs between the world's top two economies and a selloff on global stock markets.Tensions escalated on Wednesday after U.S. President Donald Trump signed an executive order barring American companies from using telecommunications equipment made by firms deemed to pose a national security risk.While the president's order did not specifically name any country or company, U.S. officials have previously labelled Huawei a "threat". "The U.S. seems to have already decided to nail Huawei down," said the China-based U.S. tech company source. "The problem is that because there doesn't seem to be a prospect for a trade deal in the near future, the U.S. has expedited the process of killing Huawei." ($1 = 6.8776 Chinese yuan)(Reporting by Sijia Jiang in Hong Kong, Josh Horwitz in Shanghai, Ju-min Park and Heekyong Yang in Seoul, Michael Martina and Cate Cadell in Beijing, Makiko Yamazaki in Tokyo; Writing Miyoung Kim; Editing by Himani Sarkar)"""

In [52]:
import gensim.summarization
import re
# import gensim.split_sentences

striptext = text.replace('\n\n', ' ')
striptext = striptext.replace('\n', ' ')
striptext = striptext.replace('U.S.','US')
p = re.compile(r"(?<=\D)(\.)(?=\D)")
striptext = re.sub(p,r'. ',striptext)

#     try:
summary = gensim.summarization.summarize(striptext, word_count=100) 
#     except:
#         summary = text
#     if not summary:
#         summary = text
#     return summary

In [137]:
import re
d = '  3.4 and.I'
d="3029.I have"
p = re.compile(r"(?<=\w)(\.)(?=\D)")
re.sub(p,r'. ',d)

'3029. I have'

In [93]:
t = "(Repeats to amend headline)* Huawei purchases $70 bln worth components annually* Huawei says sources $11 bln worth of goods from US* Huawei has been stockpiling some parts - supply chain sources* Asian supplier shares fall on news of US blacklistHONG KONG/BEIJING, May 16 (Reuters) - The latest US broadside against Huawei that puts the Chinese firm on an exports blacklist threatens to rattle the global tech supply chain. linked closely to the $105 billion business of the world's top supplier of telecoms network equipment."

In [ ]:
sentences = t.split('')

In [94]:
sentences = gensim.summarization.textcleaner.get_sentences(t)

In [135]:
import re
d = """(Reporting by Jonathan Stempel in New York Editing by Susan Thomas and Lisa Shumaker) VIDEO3:5403:54Adobe CEO 'optimistic' about prospects in wake of tariffsMad Money with Jim CramerAdobe President and CEO Shantanu Narayen said Tuesday he's bullish on the company's 2019 outlook — despite US-China trade tensions that have rattled major indexes in recent trading days. flat sector index (Adds details, analyst quote, updates shares)LONDON, May 2 (Reuters) - Royal Dutch Shell reported on Thursday a small drop in first quarter profit to $5.4 billion, but still easily beat forecasts, helped by stronger trading and liquefied natural gas earnings."""
p = re.compile(r"(?<=\()(.*)(?=\))")
re.sub(p,r'',d)

'() - Royal Dutch Shell reported on Thursday a small drop in first quarter profit to $5.4 billion, but still easily beat forecasts, helped by stronger trading and liquefied natural gas earnings.'

In [89]:
s='--'
sum([w.isalnum() for w in s.split(' ')]) == 0

True

In [109]:
d

"(Reporting by Jonathan Stempel in New York Editing by Susan Thomas and Lisa Shumaker) VIDEO3:5403:54Adobe CEO 'optimistic' about prospects in wake of tariffsMad Money with Jim CramerAdobe President and CEO Shantanu Narayen said Tuesday he's bullish on the company's 2019 outlook — despite US-China trade tensions that have rattled major indexes in recent trading days. flat sector index (Adds details, analyst quote, updates shares)LONDON, May 2 (Reuters) - Royal Dutch Shell reported on Thursday a small drop in first quarter profit to $5.4 billion, but still easily beat forecasts, helped by stronger trading and liquefied natural gas earnings."

In [141]:
import re
d = """(Reporting by Jonathan Stempel in New York Editing by Susan Thomas and Lisa Shumaker) VIDEO3:5403:54Adobe CEO 'optimistic' about prospects in wake of tariffsMad Money with Jim CramerAdobe President and CEO Shantanu Narayen said Tuesday he's bullish on the company's 2019 outlook — despite US-China trade tensions that have rattled major indexes in recent trading days. flat sector index (Adds details, analyst quote, updates shares)LONDON, May 2 (Reuters) - Royal Dutch Shell reported on Thursday a small drop in first quarter profit to $5.4 billion, but still easily beat forecasts, helped by stronger trading and liquefied natural gas earnings."""
d="""(Reporting by Nichola Groom in Los Angeles, Editing by Rosalba O'Brien) (Adds details of Sears' restructuring plan, criticism of Lands' End transaction)NEW YORK, April 18 - Sears Holdings Corp sued longtime former Chairman Eddie Lampert, his hedge fund ESL Investments and others like Treasury Secretary Steven Mnuchin, claiming they illegally siphoned billions of dollars of assets from the retailer before it went bankrupt. flat sector index May 2 - Royal Dutch Shell reported on Thursday a small drop in first quarter profit to $5.4 billion, but still easily beat forecasts, helped by stronger trading and liquefied natural gas earnings."""
p = re.compile(r"\([^\.]*\)[^ ]*")
re.sub(p,r'',d)

' YORK, April 18 - Sears Holdings Corp sued longtime former Chairman Eddie Lampert, his hedge fund ESL Investments and others like Treasury Secretary Steven Mnuchin, claiming they illegally siphoned billions of dollars of assets from the retailer before it went bankrupt. flat sector index May 2 - Royal Dutch Shell reported on Thursday a small drop in first quarter profit to $5.4 billion, but still easily beat forecasts, helped by stronger trading and liquefied natural gas earnings.'

In [113]:
d = """I know the quarter's not gonna be that great, but I say [buy].
"Allergan plc: "I'm actually gonna tell you that you need to be concerned .
We need to see that drug against migraines approved, and I don't know if it's going to get approved soon enough to be able to help what I think is a business that I'm concerned about.
I think Wayfair represents a great bargain — the products.
"WATCH: Cramer's lightning roundVIDEO4:2404:24Cramer's lighting round: The quarter won't be good, but buy this stockMad Money with Jim CramerDisclosure: Cramer's charitable trust owns shares of Palo Alto Networks, Kohl's, and Anadarko."""
p = re.compile(r"VIDEO[\d:]*")
print(re.sub(p,r'',d))

I know the quarter's not gonna be that great, but I say [buy].
"Allergan plc: "I'm actually gonna tell you that you need to be concerned .
We need to see that drug against migraines approved, and I don't know if it's going to get approved soon enough to be able to help what I think is a business that I'm concerned about.
I think Wayfair represents a great bargain — the products.
"WATCH: Cramer's lightning roundCramer's lighting round: The quarter won't be good, but buy this stockMad Money with Jim CramerDisclosure: Cramer's charitable trust owns shares of Palo Alto Networks, Kohl's, and Anadarko.


In [130]:
d = """flat sector index (Adds details, analyst quote, updates shares)LONDON, May 2 (Reuters) - Royal Dutch Shell reported on Thursday a small drop in first quarter profit to $5.4 billion, but still easily beat forecasts, helped by stronger trading and liquefied natural gas earnings. Those were partly offset by stronger contributions from trading as well as increased LNG and gas prices compared with the first quarter of 2018.(Reporting by Ron Bousso; editing by Jason Neely and Susan Fenton) essential@ (Adds CEO comment in wake or protests, paragraphs 9-10)ALGIERS, May 1 (Reuters) - Algeria's state energy company Sonatrach wants to develop its partnerships with foreign firms to boost output and exports, its new chief executive said on Wednesday, comments that could reassure investors a week after his appointment."""
p1 = re.compile(r"\w*@")
p2 = re.compile(r"\([\w ,;-]*\)[^ ]*")
p3 = re.compile(r"VIDEO[\d:]*")
text = re.sub(p1,r'',d)
text = re.sub(p2,r'',text)
text = re.sub(p3,r'',text)

In [131]:
text

"flat sector index  May 2  - Royal Dutch Shell reported on Thursday a small drop in first quarter profit to $5.4 billion, but still easily beat forecasts, helped by stronger trading and liquefied natural gas earnings. Those were partly offset by stronger contributions from trading as well as increased LNG and gas prices compared with the first quarter of 2018.   May 1  - Algeria's state energy company Sonatrach wants to develop its partnerships with foreign firms to boost output and exports, its new chief executive said on Wednesday, comments that could reassure investors a week after his appointment."

In [132]:
sentences = gensim.summarization.textcleaner.get_sentences(text)

In [133]:
for i in sentences:
    print(i.strip())

flat sector index  May 2  - Royal Dutch Shell reported on Thursday a small drop in first quarter profit to $5.4 billion, but still easily beat forecasts, helped by stronger trading and liquefied natural gas earnings.
Those were partly offset by stronger contributions from trading as well as increased LNG and gas prices compared with the first quarter of 2018.
May 1  - Algeria's state energy company Sonatrach wants to develop its partnerships with foreign firms to boost output and exports, its new chief executive said on Wednesday, comments that could reassure investors a week after his appointment.


In [152]:
import os
os.path.isfile('./News Analytics  App/news_analytics/lda/lda_a07fe772ad7f925a.html')

True

In [1]:
text = """getElementsByTagName }); }) };XXXXXXX China trade deal could spark a big rally, says Jeremy SiegelClosing BellMoney is still moving into the stock market — and it looks like the trend is here to stay for a while, Wharton School finance professor Jeremy Siegel told CNBC on Friday. Production at the oilfield was not affected by the evacuation and work is continuing normally, overseen by Iraqi engineers, state-owned South Oil Company chief Ihsan Abdul Jabbar said, adding that production remains at 440,000 barrels per day."""

In [5]:
import re
p5 = re.compile(r"getElements.*X")
text = re.sub(p5,r'',text)

In [4]:
text

' China trade deal could spark a big rally, says Jeremy SiegelClosing BellMoney is still moving into the stock market — and it looks like the trend is here to stay for a while, Wharton School finance professor Jeremy Siegel told CNBC on Friday. Production at the oilfield was not affected by the evacuation and work is continuing normally, overseen by Iraqi engineers, state-owned South Oil Company chief Ihsan Abdul Jabbar said, adding that production remains at 440,000 barrels per day.'